## Tool RAG
- Know what you know and what you don't know.


In [1]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [2]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()
# function calling : 질문에 따라 function을 불러옴 (라우팅(=네트워크에서 경로를 선택하는 프로세스))

In [5]:
# data

#1 전공
major_software = """
 {
    "input": "한림대학교 총장 정보 알려줘",
    "output": "한림대학교 총장은 최양희님입니다. 학력은 1960 ~ 1965 강릉 옥천초등학교, 1965 ~ 1968 강릉중학교, 1968 ~ 1971 경기고등학교, 1971 ~ 1975 서울대학교(전자공학) 학사, 1975 ~ 1977 한국과학기술원(전기 및 전자공학) 석사, 1980 ~ 1984 ENST 프랑스국립정보통신대학(전산학) 공학박사를 취득했습니다. 현재 서울대학교 컴퓨터공학부 명예교수이며, 2021년 9월부터 한림대학교 총장으로 일하고 있습니다."
  },
  {
    "input": "한림대학교 교육이념 알려줘",
    "output": "한림대학교의 교육이념은 개인의 성장, 사회의 발전, 국가의 번영, 인류의 행복에 기여하는 것입니다. 교육 목적은 풍부한 인간성과 창조적 지성을 지닌 인재를 양성하고 학술 및 문화의 진흥을 도모하는 것입니다. 교육 목표는 신진 인재를 양성하는 것입니다,"
  },
  {
    "input": "한림대학교 인재상 알려줘",
    "output": "한림대학교의 인재상은 첫 번째로, 인류 보편의 가치를 실천하는 창의적 선진인재입니다. 인류를 위한 보편적 가치를 함양하고, 사회와 국가의 발전에 기여하며, 글로벌 사회와 함께 성장하는 선진인재를 뜻합니다. 또한 설립 이후 대학이 지향해 온 인재상과 교육의 이념을 21세기의 시대적 요구와 접맥하고, 창조적으로 계승함. '풍부한 인간성과 창조적 지성을 지닌 인재', '지성, 감성, 의지, 체력을 구비한 인재', '더불어 성장하는 창의인재' 등 한림 인재상의 창조적 계승하는 사람을 의미합니다.  두 번째로, 창의와 융합, 실용과 조화입니다. 이는 융합적 인재, 자기주도적 인재, 창의적 인재, 실용적 인재, 조화로운 인재를 아우르는 말입니다. 융합적 인재란 인류의 지속 가능성을 위협하는 문제들에 과감하게 도전하기 위해 지식의 경계를 넘어 거침없이 협력하는 창의적 인재입니다. 자기주도적 인재란 인성, 기초 소양교육을 바탕으로 자신의 삶을 주도적으로 개척하고, 자발적 학습을 통해 역량을 개발하는 인재입니다. 창의적 인재란 개방적이고 주체적으로 새로운 과제에 도전하고 해결책을 찾아가는 인재입니다. 실용적 인재란  인류와 사회가 직면한 문제들에 대한 합리적, 효과적, 효율적 해법을 구현하는 인재입니다. 조화로운 인재란 남을 배려하는 정신으로 세상과 소통하고 나눔을 실천하는 인재입니다."
  },
  {
    "input": "2023년 한림대학교 연혁 말해봐",
    "output": "1. 창의미래융합대학원 설립(100% 온라인 석사학위 1년 과정). 2. 디자인·디지털인문예술학과, 디지털마케팅학과 제15대 부총장 최성찬 교수 취임"
  },
  {
    "input": "2022년 한림대학교 연혁 알려줘",
    "output": "1. 대학기관평가인증\n2.정보과학대학 설립(SW·AI·DATA 특화 융합 학사조직)\n3.소프트웨어학부, 인공지능융합학부, 데이터사이언스학부\n4. 나노디그리과정(소단위 전공과정) 제도 도입\n5. Hallym Vision 2030+ 선포\n6. Hallym Vision 2030+ 추진 체계 구축\n7. 3대 선도연구기관 설립·개편\n8. 행정조직 신설·개편(지식미디어센터, 커뮤니티교육원, 인권센터, 안전팀 등)\n9. 3단계 산학연협력 선도대학 육성사업(LINC 3.0) 선정\n10. 지역혁신 선도연구센터(RLRC) 사업 선정\n11. 의료 인공지능 융합인재 양성 사업 선정\n12. 한림대학교 OPEN Station 개소\n13. 한림대학교 도헌학술원 설립\n14. 자기설계융합전공 제도 도입"
  },
  {
    "input": "2021년 한림대학교 연혁 뭐야",
    "output": "1. 첨단분야 학과 신설 및 정원 순증\n2. 인공지능융합학부 신설(입학정원 60명)\n3. 제2대 산학부총장 강일준 교수 취임\n4. 캠퍼스라이프 카운슬링센터 신설(학생생활상담센터 개편)\n5. 제14대 부총장 김인영 교수 취임\n6. 제11대 총장 최양희 박사 취임\n7. 교육부 대학기본역량진단 일반재정지원대학 선정\n8. 교육부 산업계관점 대학평가 최우수(소프트웨어융합대학)"
  },
  {
    "input": "2020년 한림대학교 연혁 알려줘",
    "output": "1. 제10대 총장 김중수 박사 취임\n2. 원격교육지원센터 신설\n3. 4단계 BK21 사업 선정\n4. 한림대학교 기술지주주식회사 설립\n5. 대학원 인공지능융합학과 신설\n6. 융합교육지원센터 신설\n7. 한림 비전관 개관"
  },
  {
    "input": "2019년 한림대학교 연혁 뭐야",
    "output": "1. Smart Campus 구축\n2. 교양교육 혁신\n3. 일송자유교양대학 설립(교양기초교육대학 개편), 자유교양전공 신설\n4. Liberal Arts(자유교양) 중심 교양교육 개편\n5. 간호대학 설립\n6. 대학원 스쿨 설립 학사조직 개편\n7. 데이터과학융합스쿨 설립(수학과, 통계학과 통합․개편)\n8. 나노융합스쿨 설립(물리학과 개편)\n9. 대학혁신지원사업 선정\n10. KOICA 기후변화대응분야 석사학위 연수사업 선정\n11. LINC+ 1단계 평가 최고등급인 매우우수 획득\n12. 교책연구원 한림AI융합연구원 설립\n13. 사색의 정원 준공"
  },
  {
    "input": "2018년 한림대학교 연혁 말해봐",
    "output": "1. 글로벌융합대학 설립\n2. 소프트웨어중심대학 사업 선정\n3. Campus Life Center 개관\n4. 한림 Vision & Action 2018~ 수립\n5. 스쿨 설립 학사조직 개편\n6. 미디어스쿨 설립(미디어커뮤니케이션학부 개편)\n7. 데이터과학융합스쿨 설립(금융정보통계학과, 데이터테크전공 통합․개편)\n8. 나노융합스쿨 설립(응용광물리학과, 융합신소재공학과 통합․개편)\n9. 소프트웨어융합대학 설립(컴퓨터공학과, 융합소프트웨어학과, 전자공학과 통합․개편)\n10. 미래융합스쿨 설립\n11. 교육부 대학기본역량진단 자율개선대학 선정\n12. 도헌글로벌스쿨 개관\n13. 한림대학교 개교 100주년 기념 타임캡슐 매설"
  },
  {
    "input": "2017년 한림대학교 연혁 알려줘",
    "output": "1. 대학기관평가인증\n2. 융합전공 도입․활성화\n3. 융합인재학부 신설\n4. 사회맞춤형 산학협력 선도대학(LINC+) 육성사업 선정\n5. 학생 중심 교육을 위한 학사 제도 개편\n6. 복수전공 필수화, 소속변경 자율화\n7. 제1대 산학부총장 송창근 교수 취임\n8. 제13대 부총장 안동규 교수 취임\n9. 글로벌협력대학원 설립\n10. 비전협력처 신설(비전전략처, 대외협력처 통합․개편)"
  },
  {
    "input": "2016년 한림대학교 연혁 알려주세요",
    "output": "1. 제9대 총장 김중수 박사 취임\n2. 비전전략처 신설\n3. 한림 Vision & Action 2016~2022 선포\n4. 그룹웨어 시스템 구축, 전자결재 시행\n5. 제1회 Intramural League 개최"
  },
  {
    "input": "2015년 한림대학교 연혁 뭐에요",
    "output": "1. 교책연구소 ICT정책연구센터 신설, 국제학부 산하 정보법과학연구소 신설\n2. 제12대 부총장 변용환 교수 취임\n3. 대학구조개혁평가 A등급"
  },
  {
    "input": "2014년 한림대학교 연혁 말해줘",
    "output": "1. 제11대 부총장 송승철 교수 취임\n2. 산학협력특성화대학 신설\n3. 한림대학교 지방대학특성화교육사업단(CK-I) 신설\n4. 한림대학교 심비우스케어교육사업단 설치\n5. 직속기구 ‘한림미래교육평가원’신설\n6. 부속기관 ‘종합연구센터’신설\n7. 기초교육대학 산하‘의사소통교육센터’ 신설\n8. 의과대학 산하‘춘천임상기초협동연구소’, ‘한림바이오아이텍연구소’ 신설"
  },
  {
    "input": "2013년 한림대학교 연혁 말해봐",
    "output": "1. 사회과학대학 산하 ‘글로벌 사회공헌 연구소’ 설치\n2. 부속기관 레크리에이션센터 설치"
  },
  {
    "input": "2012년 한림대학교 연혁 알려줘",
    "output": "1. 제8대 총장 노건일 박사 취임\n2. 제10대 부총장 최수영 교수 취임\n3. 의과대학 산하 ‘자살과 학생 정신건강 연구소’ 설치\n4. 사회과학대학 산하 ‘헬스커뮤니케이션연구소’ 설치"
  },
  {
    "input": "2011년 한림대학교 연혁 알려주세요",
    "output": "1. 한림봉사센터 설치\n2. 학생생활관 신관 신축\n3. 부속교육기관 ‘한림언어청각센터(학교기업)’ 설치\n4. 자연과학대학 산하 ‘척추측만운동연구소’ 설치\n5. 의과대학 산하 ‘세포분화 및 노화 연구소’ 설치 6. 정보전자공과대학 산하 ‘BIT 융합연구소’ 설치"
  },
  {
    "input": "한림대학교 재학생 현황이 어떻게 돼?",
    "output": "한림대학교 재학생은 2023년 4월 1일 기준, 대학 7,594명, 일반대학원 410명, 특수대학원 276명입니다."
  },
  {
    "input": "한림대학교 교직원 현황 말해봐",
    "output": "한림대학교 교직원은  2023년 4월 1일 기준, 전임교원 748명, 비전임교원 617, 직원 262명입니다."
  },
  {
    "input": "한림대학교 조직 현황 알려줘",
    "output": "한림대학교 조직 현황은 2023년 4월 1일 기준, 교육기구, 대학본부, 부속기관, 부설연구소, 산학협력 기관, 의료 및 부속병원이 있습니다. 교육기구는 8개 대학, 4개 스쿨, 1개 일반대학원, 6개 특수대학원이 있습니다. 대학본부는 7개 처, 24개 팀·센터·실이 있습니다. 부속기관에는 15개의 기관, 부설 연구소에는 70개의 연구소가 있습니다. 산합협력 기관은 1개 산학협력단이 있으며, 의료 및 부속병원은 5개 대형 종합병원이 있습니다."
  },
  {
    "input": "한림대학교 학교 시설 현황 말해봐",
    "output": "한림대학교의 교지는 267,695m²의 면적을 가지고 있습니다. 교사 기본 시설은 267,695m², 지원시설은 49,017m², 연구시설은 21,708m²의 면적을 가지고 있습니다."
  },
  {
    "input": "인문학부 소개해줘",
    "output": "인문학은 모든 학문의 근본이 되는 학문이며, 문화의 핵심, 소프트웨어의 중심에는 국어국문, 철학, 사학 즉 인문학이 자리 잡고 있습니다.  \n한림대학교 인문학부는 인문학에 대한 지식을 폭 넓고 다양하게 공부하여, 끊임없이 변화하는 사회에 발 맞추어 나아갈 수 있는 인재를 양성하고 있습니다."
  },
  {
    "input": "인문학부 전공진입 규정이 뭐야",
    "output": "인문학부에 입학한 학생들은 1학년 재학 중 국어국문, 철학, 사학 각 전공에서 탐색과목을 1과목 이상씩 수강해야 합니다. \n세 전공에서 1과목 이상씩 탐색과목 이수를 완료한 인문학부 1학년 학생들은 1학년 2학기말에 희망하는 전공(3순위까지)을 우선적으로 배정한 후, 탐색과목을 제대로 이수하지 않았거나, F학점 받은 인문학부 1학년들을 희망하는 전공에 배정합니다."
  },
  {
    "input": "국어국문학 전공 학과 소개해주세요",
    "output": "국어국문학은 국어에 대한 폭넓은 이해를 통하여 국어 및 국문학의 본질, 그 형성과 역사적 전개 양상, 그 특징 등을 연구하고 분석 검토하여 국어국문학에 대한 바른 이해를 깊게 하며, 참된 한국인으로서의 기본적 학식을 갖추어 인격 도야에 크게 이바지하도록 교육하고 있습니다.\n\n전문적인 지식습득을 위해 강의실에서 이루어지는 강의 외에 문학 탐방 등 현지조사 연구도 활발히 진행하고 있으며, 르네상스 인문학, 영상문예창작 등 다양한 협동과정과 연계하여 학문 영역을 확장하고 있습니다"
  },
  {
    "input": "국어국문학과 교육 목표가 뭐에요",
    "output": "국어국문학전공에서는 다음과 같은 인재를 양성하기 위해 교육하고 있습니다.\n1. 국어학과 국문학의 모든 영역을 균형 있게 공부하여 한국인으로서의 자부심을 가질 수 있는 지성과 교양을 갖춘 인재\n2. 국어국문학과 관련되는 사회의 각 기관에서 유능한 지도자적 역할을 해낼 수 있는 인재\n3. 대학원에 진학하여 학문을 연마할 수 있는 기본적인 자질을 갖춘 인재\n이러한 인재 양성을 위한 기본적인 교과과정 운영 외에 르네상스 인문학, 영상문예창작 등 다양한 협동과정과 연계하여 학문 영역을 확장하고 있습니다."
  },
  {
    "input": "국어국문학 전망과 진로 알려주세요",
    "output": "국어국문학은 언어를 매체로 하여 학문 활동을 수행하는 기초학문이기 때문에, 취업의 문이 비교적 광범위한 편입니다. 본교가 배출한 국어국문학전공 졸업생들은 현재 문학계, 언론계, 금융계, 출판계, 공무원 및 기타 사회의 각 분야에 진출하여 활발한 활동을 하고 있습니다.\n국어국문학 전공 교육과 졸업 후 사회 진출을 연계하기 위하여 직업군별 전공 트랙을 운영하고 있는데 전공 심화 트랙으로 한국어교육, 문화 기획, 문학 창작 트랙이 있으며, 언론사, 출판사, 일반 회사에 취업하거나 공무원 시험을 준비하는 트랙이 있습니다."
  },
  {
    "input": "사학전공 학과 소개해봐",
    "output": "사학전공은 한림대학이 개교한 다음해인 1983년에 첫 신입생을 맞이하였으며, 현재 인문학부에 소속한 독립학과로서, 석사와 박사과정도 설치되어 있습니다. 사학전공은 한국사뿐만 아니라 동양사와 서양사 및 고고학에 대한 폭넓고 균형있는 역사교육을 통하여 보편적인 상식과 역사적 통찰력과 시민적 지성을 갖춘 역사학도와 교양인을 양성함을 목표로 설정하고 있습니다. 이러한 목표달성을 위하여 역사 전 분야에 대한 다양한 강의를 개설하고, 국내외 현장답사 및 각종 역사문화콘텐츠관련 프로젝트 등을 실시하고 있습니다."
  },
  {
    "input": "사학과 전망과 진로에 대해 알려줘",
    "output": "사학전공의 졸업생은 대학의 교수로서 학자의 길로 갈 수도 있고, 국립박물관과 대학박물관, 그리고 문화재 분야의 학예사와 강원문화재연구소와 같은 발굴전문기관으로의 진출도 가능하고, 나아가서 언론계와 문화계 및 교육계 등 사회문화 전 분야에 걸쳐서 폭넓은 직업선택과 활동의 폭이 열려져 있습니다. 사학의 경우, 학부 4년 동안에 인간의 삶과 역사전반에 대한 보편적 상식과 한국사, 동양사, 서양사, 고고학에 이르는 역사의 모든 분야에 대한 교육과 각종 실습과 훈련을 통하여 길러진 지식과 통찰력이 뒷받침됨으로서 개방적이고 다양한 진로선택이 가능합니다."
  },
  {
    "input": "철학 전공 학과 소개해봐",
    "output": "철학은 일상인들의 사고와 개별 학문들에서 중요하게 기능하면서도 그 의미와 본성에 대해 반성되지 않는 개념들에 대해 반성하는 학문입니다. 따라서 우리의 믿음을 구성하고 행위를 결정하는 데 개입하는 모든 중요한 개념과 이유들이 철학의 검토 대상입니다. 또한 그 근본적 개념들이 어떻게 사용되어야 세계와 우리 자신에 대해 올바른 지식을 제공할 수 있는지를 탐구합니다. 철학적 탐구를 통해 비판적이고 합리적 사고능력, 그리고 정당한 규범의식과 가치관이 배양됩니다."
  },
  {
    "input": "철학 전공 전망과 진로가 어떻게 돼?",
    "output": "철학을 공부하면서 학생들은 논리적이고 분석적 사고력과 판단능력을 갖추게 되어 사회 각 분야에서 활동할 역량과 기회를 얻게 됩니다. 철학전공 졸업생들은 학계, 언론계, 출판계, 교육계, 시민운동, 문화예술계 및 기타 사회 각 분야에 진출하여 활발한 활등을 하고 있습니다. 특히 논리 및 논술의 교육, 그리고 인문학적 소양이 필요한 문화 관련 기관에서 활동할 가능성이 많습니다. 철학전공에서는 매년 취업특강을 실시하여 철학전공 출신들이 진출할 수 있는 취업분야에 관한 각종 정보를 제공하고 있습니다."
  },
  {
    "input": "인문대학 소개해줘",
    "output": "인문대학은 설립 초기부터 역사, 철학, 문학, 언어학, 고고학 등의 영역에서 국내 최고 수준의 교수님들을 모시고 학문과 교육의 독특한 전통을 쌓아 왔습니다.교육 과정을 꾸준히 다양 화시키고 교육 방법을 지속적으로 개발해 왔으며, 교육 환경도 획기적으로 변모시킴으로 써 학생들에게 학습 동기를 높여 주고 다양한 진로 개발의 가능성을 넓혀주고 있습니다. 인문 대학 은 1개 학부(3개 전공), 4개 학과로 구성되며, 3개 협동(융합)전공 및 교직과정을 운영하고 있습니다. 인문학 영역에는인문학부(국어국문학, 철학, 사학), 영어영문학과가 있고, 지역학 영역에는 중국학과, 일본학과, 러시아학과가 있습니다. 협동(융합)전공은 한중비즈니스, 디지털인문예술전공, 글로컬융합인문학전공이 있고, 교직 과정에는 철학과가 있습니다."
  },
  {
    "input": "인문대학 협동(융합)전공에 대해 설명해",
    "output": "인문대학 경쟁력 프로그램으로는 협동(융합)전공, 교직과정이 있습니다. 협동(융합)전공에는 한중비지니스, 디지털인문예술전공, 그로컬융합인문학 전공이 있습니다. 한중비지니스 전공에는 중국학과, 경영학과가 있습니다. 디지털인문예술전공에는 영어영문학과, 철학과, 중국학과, 금융통계학과, 컴퓨터공학과가 참여합니다. 글로컬융합인문학전공에는 인문학부(국어국문학전공·철학전공·사학전공), 영어영문학과, 중국학과, 일본학과, 러시아학과, 디지털인문예술전공이 참여합니다."
  },
  {
    "input": "인문대학 교직과정에 대해 설명해",
    "output": "인문대학 경쟁력 프로그램으로는 협동(융합)전공, 교직과정이 있습니다. 교직과정은 인문학부 철학전공이 참여합니다."
  },
  {
    "input": "일본학연구소가 뭐야",
    "output": "일본학연구소는 인문한국(HKPLUS)지원사업-포스트 제국의 문화 권력과 동아시아에 대한 연구 및 교육활동을 진행하며, 교육부(한국연구재단)이 지원합니다. 책임교수는 서정완 교수님입니다."
  },
  {
    "input": "생사학연구소가 뭐야",
    "output": "생사학연구소는 인문한국플러스(HK+)지원사업 - 포스트 코로나 시대의 '뉴노멀 생사학' (메타버스에 기반한 생명 존중 네트워크 구축)에 대한 연구 및 교육활동을 진행합니다.  교육부(한국연구재단)에서 지원하며, 책임교수는 박준식 교수입니다."
  },
  {
    "input": "인문학부 교환학생 프로그램에 대해 말해",
    "output": "인문학부의 교환학생 프로그램은 총 4개 국가가 있습니다. 미국 유타대학교 지원 대상은 영어영문학과입니다. 중국 북경중앙민족대, 상해해양대, 광동외어외무대학, 서안외국어대학, 대련민족대학 지원 대상은 중국학과입니다. 일본  코마자와대학, 돗토리대학, 카나자와대학교, 니가타대학교, 동북복지대학, 도쿄릿쇼대학, 나고야 시립대학, 오사카 가쿠인대학, 쿠루메대학 지원 대상은 일본학과입니다. 일본 니가타대학교, 카나자와대학교 지원 대상은 인문대학입니다. 러시아 블라디보스톡 국립경제서비스대학, 블라디보스톡 국립극동대학교 지원 대상은 러시아학과입니다."
  },
  {
    "input": "인문학부 단기연수 프로그램에 대해 말해줘",
    "output": "인문학부의 단기연수 프로그램은 총 4개 국가가 있습니다. 미국 유타대학교 단기연수 지원 대상은 영어영문학과입니다. 중국 대련이공대학 단기연수 대상은 인문학부, 중국학과, 일본학과, 러시아학과입니다. 일본 돗토리대학 단기연수 지원 대상은 인문학부, 중국학과, 일본학과, 러시아학과입니다. 일본 쿠루메 대학교 지원 대상은 일본학과입니다. 러시아 블라디보스톡 국립경제서비스 대학 단기 연수 지원 대상은 중국학과, 일본학과, 러시아학과입니다."
  },
  {
    "input": "영어영문학과 소개해",
    "output": "영어영문학은 영미의 언어와 문학을 연구함으로써 그 문화적 전통과 성격을 이해하는 학문영역입니다. 미문학과 영문학, 영어학이라 순수 학문적 분야와 교양영어를 포함한 영어구사력 함양이라는 실용적 분야의 두 영역을 조화시켜 교육하고 있습니다.\n\n특히 사회진출에 도움을 줄 수 있도록 실용영어 강좌를 대폭 강화시키고 있으며, 본교에서 지정한 외국대학교의 어학연구소와 일정기간 어학과정을 학생에게 그곳에서 취득한 학점을 본교의 대체과목 학점으로 환산·인정해주는 학점인정 해외현지교육 프로그램을 실시하고 있습니다."
  },
  {
    "input": "영어영문학과 교육목표가 뭐야",
    "output": "영미 문학작품의 이해를 통해 그 문화군의 역사적 전통과 가치관을 접하게 하고, 그것의 바람직한 수용방법을 연구함으로써 한국문학과 문화 전반에 이바지"
  },
  {
    "input": "영어영문학과 전망과 진로를 말해줘",
    "output": "국제화, 세계화 추세에 따라 영어의 필요성이 사회적·국가적으로 더욱 증대됨에 따라, 영어 실력을 탄탄하게 갖춘 사람이 진출할 분야는 방송, 언론계, 관광, 광고업계, 출판계, 금융계, 일반 기업체 및 공무원직에서부터 공인된 영어구사력자격증 제도에 따른 통역, 번역사나 속기사 그리고 개인 자유업에 이르기까지 매우 다양합니다.\n\n교직과목을 선택하여 영어교육 과목을 이수하는 경우, 국가고시를 통해 교사자격증을 취득하여 현직 영어교사로 진출하거나 대학원에 진학하여 더 깊이 전공 분야를 공부한 다음, 영어교육 관련기관이나 출판계, 방송계 등에 취업할 수 있다."
  },
  {
    "input": "영어영문학과 연계전공이 뭐야",
    "output": "영어영문학을 전공하는 학생은 복수 전공을 통해서 여러 분야를 섭렵할 수 있습니다. 자신의 관심사를 쫓아서 또는 졸업 후의 진로를 염두에 두고서 영어영문학과 학문적 전망 및 진로 특성화에 따른 전공을 연계시켜 공부할 수 있습니다."
  },
  {
    "input": "중국학과 소개해봐",
    "output": "중국학과는 중국 어문을 중심으로 중국의 정치·경제·철학·종교·미술·음악·인류학· 지정학 등을 광범위하게 학습하여 중국문화 일반에 대한 폭넓은 이해와 지식을 갖게 하며, 나아가 중국과 중국인을 깊이 있게 이해할 수 있도록 한다는 점에서 다른 어문학 위주의 중어중문학과와 구별됩니다. 이에 다양한 학과목을 개설하여 학생들로 하여금 광범위하고 전문적인 학습을 하도록 하고, 이를 바탕으로 장차 다가올 한중 양국의 관계발전을 주도하여 나갈 수 있는 건전한 지성인이 될 수 있도록 노력하고 있습니다. 특히 어학실력 제고 및 현지체험을 위해 매년 1학기에 2학년을 대상으로하여 한 학기간 중국현지교육을 진행하고 있으며, 매학기마다 자매대학과의 교환학생 교류를 실시하고 있습니다."
  },
  {
    "input": "중국학과 교육목표가 뭐야",
    "output": "중국학과는 중국을 광범위하게 학습하는 지역학과로서 중국에 대한 종합적이고도 전문적인 지식을 갖춘 지역전문가를 양성하는데 그 목표를 두고 있습니다. 현재 중국학과는 여러 분야의 학과목을 개설하여 학생들로 하여금 광범위하고 전문적인 학습을 하도록 하고 이를 바탕으로 한중 양국의 관계발전을 주도하여 나갈 수 있는 건정한 지성인이 될 수 있도록 노력하고 있습니다. 그러한 인력으로서의 자질 제고를 위해 Hallym University in China 라는 독특한 학과 운영으로 1년간의 중국 대학에서의 학습과 생활환경을 제공하고 있습니다. 중국에서의 대학생활과 언어학습, 문화생활의 준비 과정으로 1학년부터 2학년 1학기 까지 3학기 동안의 집중화된 언어교육을 통해 2학년 2학기부터 시작되는 학생들의 중국 현지 대학에서 “중국현지수업” 또 이와 연계된 “중국권 교환학생”, “중국현지인텁쉽” 등의 프로그램이 최고의 효과를 낼 수 있도록 만전을 기하고 있습니다. \n\n뿐만 아니라 한국에서도 중국에서의 생활을 체험할 수 있도록 24시간 중국어 생활이 가능하고 24시간 중국 학생들과 생활 할 수 있는 China Life in Hallym University 라는 학과 운영 방식을 도입하였습니다. 또한 학교와 교수 학생들간의 유기적인 협력관계 속에서 중국학과는 학생들이 중국 역사, 문화, 정치, 경제 전반을 아우르는 학습 과정을 통해 중국과 관련된 전반적이고 통합적인 학습이 가능하도록 교육과정을 조정하고 있습니다. 이러한 교육과정은 학생, 학부모, 학과 교수들과 학교 및 사회 전반의 요구를 바탕으로 학생들에게 최적의 교육 환경을 제공하기 위해 지속적으로 조정되고 있으며 심화되는 경쟁 사회 속에서 적극적으로 대응할 수 있는 인재들을 양성하기 위해 최선의 노력을 다하고 있습니다."
  },
  {
    "input": "중국학과 전망과진로 말해봐",
    "output": "1992년 한·중 수교가 이루어진 후, 양국의 관계가 더욱더 긴밀해짐에 따라 중국문제 전문가에 대한 사회적 수요도 폭발적으로 증가하여 중국학 전공 학생들의 사회진출은 다양한 분야에서 활발하게 이루어지고 있습니다. 졸업 후 진로는 크게 대학원 진학과 취업으로 나누어 볼 수 있는데, 대학원 진학은 중국, 대만 등 국외대학원과 동시통역대학원, 일반대학원, 국제전문가 양성 특수 대학원 등 국내대학원 진학이 있습니다. \n본 학과를 졸업한 학생들은 학부에서 공부했던 것을 토대로 하여 나름대로의 관심분야를 선택하여 학문적으로 심도 있게 공부하고 있습니다.\n한편 취업의 길은 훨씬 다양하게 열려져 있는 편인데, 본 학과에서 습득한 중국어 실력을 바탕으로 국내외 대기업 및 중소기업을 비롯하여 정부기관(외무부, 국가정보원, 각 시·도청의 국제교류팀 등), 항공사, 선박회사, 수출회사, 방송국, 출판사, 광고사, 여행사, 호텔 등과 같은 업계에 진출하여 활발하게 활동하고 있으며, 중국어 강사, 통역사, 관광가이드, 전문번역사가 되어 프리랜서로 활약하고 있는 졸업생들도 있습니다."
  },
  {
    "input": "중국학 관련 자격 알려줘",
    "output": "HSK, CPT, BCT, FELX, 관광통역자격증 등"
  },
  {
    "input": "일본학과 소개해",
    "output": "21세기 우리의 앞날을 헤쳐나가기 위해서는 세계 각 지역에 대한 이해와 연구가 필수적입니다. 그 중에서도 바로 이웃한 일본은 단지 세계 유수의 경제대국일뿐만 아니라, 아시아태평양지역의 강국으로서 재도약하고 있으며, 상호협력을 위해서도, 또 지속적인 쟁점과 갈등의 해결을 위해서도 일본을 잘 아는 인재양성이 절실합니다. 외국어관과 현지교육제도, 일본학연구소와 도서관 등 국내 최고 수준의 다채로운 교육프로그램과 시설 및 우수하고도 열성적인 교수진을 갖춘 한림대 일본학과는, 일본학을 배우는 학생에게는 최적의 선택이 될 것입니다."
  },
  {
    "input": "일본학과 교육 목표가 어떻게 돼?",
    "output": "일본학과는 21세기 국제화 시대에 우리에게 매우 중요한 일본이라는 지역을 다루는 전문 인력을 양성, 배출하는 데에 그 교육목표를 두고 있습니다. 이를 위해서 일본학과에서는 체계적인 일본어 교육을 바탕으로 일본의 사회와 문화, 역사, 문예, 한일관계 등에 대한 핵심적 지식을 폭넓게 습득케 하며, 또한 일본 현지교육 등의 각종 프로그램 및 다양한 학내시설 활용기회를 제공하는 등 국내 최고수준의 학습여건을 마련하여 지역전문가로서의 기본 소양을 갖출 수 있도록 지도합니다."
  },
  {
    "input": "일본학과 전망과 진로가 어떻게 돼?",
    "output": "졸업 후 진로는 매우 다양하다. 학업을 계속하고자 하는 경우 선택하는 대학원 진학은, 일본의 국공립 및 사립대학의 대학원에 진학하는 경우와 국내의 일반대학원, 동시통역대학원, 국제전문가 양성을 위한 특수대학원에 진학하는 경우가 있다. 일본학과 졸업생들은 나름대로의 관심분야를 선택하여 보다 심도 있는 대학원 생활을 보내고 있으며, 이제 박사학위를 취득한 선배들도 배출되고 있습니다.\n한편, 취업의 길은 훨씬 다양하게 열려져 일본학과에서 습득한 일본어 실력 및 일본학에 대한 식견을 바탕으로 일본 지방자치단체의 국제교류원, 국내외 대기업 및 중소기업, 정부 및 지방자치단체(각 시·도청의 국제교류팀), 항공사, 방송국, 연구소, 출판사, 광고사, 여행사, 호텔, 문화이벤트산업 등의 각종 분야에 진출하여 활발하게 활동하고 있으며, 일본어강사, 통역사, 관광가이드, 전문 번역사가 되어 프리랜서로 활약하고 있는 졸업생들도 있습니다."
  },
  {
    "input": "러시아학과 소개해봐",
    "output": "러시아학과는 강원도 유일의 러시아 관련 학과로서, 러시아어 능력을 잘 갖춘, 지식융합형 지역실무인재를 키우는 것을 목표로 합니다. 러시아가 세계에서 가장 넓은 영토와 막대한 천연자원을 보유한 국가로서 우리나라의 산업발전에 매우 중요한 역할을 하고 있다는 점은 주지의 사실입니다. 특히 최근에는 이른바 동북아-환동해경제권의 발흥이라는 시대적 여건변화를 맞이해 한국과 러시아 간의 경협이 완전히 새로운 단계로 진입하고 있으며(한국정부의 유라시아 이니셔티브 정책 추진, 강원도의 신북방정책, 러시아 관련 의료관광산업 성장, 시베리아횡단철도 및 가스관 연결사업 재부상, 한-러간 무비자협정 발효 등), 이에 따라 관련 인력의 수요가 비약적으로 급증하고 있는 것이 현실입니다. 이러한 시대변화에 발맞춰 준비된 실무러시아전문가를 키워내는 것이 본과의 제1의 목표입니다.\n이를 위해 본과는 교육부에서 지원하는 ‘광역경제권 선도산업 인재양성산업’(의료관광인재양성사업; 2009~2012년)을 통해 의료관광통번역, 코디네이터 인재들을 집중적으로 양성한 바 있으며, 작년부터는 교육부 ‘지방대학특성화사업’(동북아지역 융합인재 양성사업; 2014~2019년)에 선정돼 매년 지원되는 1억원 가량의 교육비로, 러시아를 중심으로 중국, 일본을 아우르는 동북아융합인재를 배출해내고 있습니다. 이외에도 대학전체로는 교육부에서 주관하는 ‘잘 가르치는 대학 11’에도 지정된 바 있어, 입학생들에게는 관련 장학금혜택과 교육비지원혜택이 기다리고 있습니다."
  },
  {
    "input": "러시아학과 교육목표가 뭐야?",
    "output": "러시아는 그가 지닌 거대한 잠재력과 막대한 천연자원 그리고 우리나라가 처한 지정학적인 위치 등으로 우리에게는 정치·안보·경제적으로 매우 중요한 국가라고 할 수 있습니다. 그러므로 러시아를 올바르게 이해하는 것은 우리의 여러 가지 정치·경제 정책의 선택에 있어 필수적이고 이를 위해서는 언어·문학·언어학· 역사·철학·사상·문학 등 인문과학 분야와 정치·경제·사회 등 사회과학 분야의 총체적인 연구를 통해서만이 가능합니다. 한림대학교 러시아학과에서는 이러한 시대적 상황에 맞추어 기존의 노어 노문학과 혹은 러시아어과에서 수행하고 있는 언어와 문학 중심의 강의에서 벗어나 러시아어 학습을 바탕으로 문학 및 역사·철학·사상·문화·민속 그리고 정치·경제 등 다양한 분야의 지역학 강좌를 개설함으로써 러시아 전반에 걸친 교육을 실시하고 이에 대한 우리의 이해의 폭을 넓히고 관련 분야에 걸친 전문적인 지식을 갖춘 지성인의 양성을 목표로 합니다."
  },
  {
    "input": "러시아학과 전망과 진로가 어떻게 돼?",
    "output": "러시아학과를 졸업한 학생들은 러시아어라는 흔치 않은 외국어를 구사할 수 있는 능력을 가지고 있기 때문에 취업의 길이 다양하게 열려 있습니다. 국내외 대기업 및 중소기업을 비롯하여 정부출자 기관, 연구소, 방송국의 해외 방송 부문 등에서도 러시아학과 졸업생들을 필요로 하고 있습니다. 또한 통역사 외무고시를 통한 외교관, 특정 공무원직등에도 종사할 수 있는 기회가 많다. 그리고 보다 특화된 전문가가 되고자 하거나, 학문적 접근을 원하는 경우에는 국내 대학원 및 러시아, 구미 각국으로의 유학 등을 통해 학업을 계속하는 방법도 있습니다. \n\n1. 어문학분야 : 지역 전문가가 되기 위해서는 무엇보다도 언어습득이 선행되어야 한다. 단계별 문법 강의와, 러시아인에 의한 회화 강의, 러시아 문헌 강독 등을 단계적으로 진행함으로써 학생들은 러시아어를 자유롭게 구사할 수 있을 것입니다. 이와 동시에 러시아인들의 다양한 삶의 방식, 세계관, 가치관, 문화, 사상 등을 접하게 하고 러시아에 대한 구체적인 이해를 가능하게 하기 위해 러시아문학 강의를 병행하고 있습니다. \n2. 사회과학분야 : 성공적인 러시아 지역전문가가 되기 위해서는 러시아어 습득을 바탕으로 이 지역에 대한 전반적이고 광범위한 지식이 요구됩니다. 이러한 요구에 부응해서 먼저 러시아의 역사를 강의하고, 역사적 인식과 안목을 바탕으로 현재 러시아에서 핫 이슈가 되고 있는 정치, 경제, 사회적 문제들을 검토하고 세계 정치, 경제 무대에서 러시아가 차지하는 의미와 한반도에 미치는 직접적인 영향력의 범위를 검토해 봅니다.\n3. 취업 및 진로 분야 : 의료관광통번역사, 의료관광코디네이터, 경찰청, 국정원, 해양경비안전본부 외국어전문요원, 외교부, 해양수산부 외국어특채, 러시아 관련 대기업, 해외무역 및 러시아현지 채용, 항공 및 여행 관련 서비스 직종, 통번역대학원 및 일반대학원 진학"
  },
  {
    "input": "글로컬융합인문학전공 소개해봐",
    "output": "글로컬 융합 인문학 전공은 특별한 창의적, 융합적 사고와 도전정신이 요구되는 4차 산업혁명 시대를 맞이해, 인문대학의 7개 학과, 30여명의 교수님들이 의욕적으로 참여해 운영하는 융합전공 프로그램입니다. 세계가 기술기반, 실무기반의 사회가 될수록, 오히려 역설적으로 문화 간, 지식체계 간 소통능력이 중요해질 수밖에 없으며, 그런 만큼 인문학적 사유의 힘과 그 가치는 더 빛을 발할 수밖에 없습니다. \n\n이러한 취지에서 본 융합전공은 “열린 한국학, 뿌리 있는 지역학, 세계시민적 소양”이라는 비전을 모토로 세 가지 하위 트랙을 통섭적으로 운영합니다. 이를테면 한국학을 기존의 국학적 시각에서 바라보는 것에 매몰되지 않고 글로벌 시대 타자의 시선에서 바라볼 수 있도록 하며, 한반도를 둘러싼 주변 4강(미국, 중국, 일본, 러시아)의 역사적 쟁점과 정치경제적 이해관계를 한국을 비롯한 어느 한 국가의 입장에 고착되지 않고 종합적으로 바라볼 수 있는 사유의 힘을 키우고자 합니다. 이러한 사유의 훈련을 철학적으로 뒷받침해 줄 세계시민적 소양 역시 본 융합전공에서 추구하는 중요한 비전입니다. 부디 많은 학생들이 본 교육과정을 이수하여, 명실상부한 인문학적 융합교육의 정수를 맛보고, 급변하는 사회에서 가치를 인정받을 수 있는 인재로 성장하기를 바랍니다."
  },
  {
    "input": "글로컬융합인문학 전공 교육 목표가 뭐야",
    "output": "1) 인문지성을 함양하기 위해 보편적 진리를 습득하도록 지도\n2) 열린 미래 세계상을 충족하고 진로 탐색의 가능성을 극대화하도록 전공 트랙별 교육의 유연성 보장\n3) 기초 역량 및 실용(응용·활용) 역량 강화에 초점을 두어 실용 전공 및 순수 전공 학생들에게 전공 진입의 기회를 극대화\n4) 교과·비교과, 학내·대외(지역, 국가, 해외), 학기·방학을 모두 활용하여 기초 역량 및 실용 역량을 강화\n5) 현장에서 활용할 수 있는 인증·자격증을 트랙별로 안내하여 본 전공에 진입하는 학생의 실용 역량을 강화"
  },
  {
    "input": "글로컬융합인문학 전공 인재상이 뭐야",
    "output": "인문지성을 바탕으로 삶·사회·문화의 제반 문제에 능동적, 효과적으로 대응할 수 있는 창의적·실용적·융합적 인재 양성\n1) 창의적 인재: 주체적·개방적·논리적 태도를 통해 독창적이면서 신선한 생각을 하는 인간형\n2) 실용적 인재: 공익적·발전적·합리적 태도를 통해 공통체가 요구하는 제도 문물을 실생활에 구현할 수 있는 인간형\n3) 융합적 인재: 유창적·융통적·자발적 태도를 통해 다원적 가치를 존중하고 한반도와 세계 문화 창달에 이바지할 수 있는 인간형"
  },
  {
    "input": "글로컬융합인문학 전공 졸업 후 진로가 어떤게 있어?",
    "output": "전공에서는 진로에 대한 개방적인 태도를 갖는습니다.\n• 경영: 문화재감정평가사, 마케팅전문가, 브랜드매니져 등\n• 교육: 교수, 교사, 교육학·언어학·역사학·철학·인문과학 연구원, 이러닝 교육설계자, 교재·교구 개발자 등\n• 개인서비스: 통·번역가, 변리사, 노무사, 승무원 등\n• 공공: 공무원, 공기업 종사자, 외교관, 직업상담사, 국제기구 종사자 등\n• 미디어·엔터테인먼트: 작가, 기록물관리사, 문화재보존원, 광고·출판물 기획자 등"
  },
  {
    "input": "사회과학대학 소속 전공에 뭐가 있어?",
    "output": "사회과학대학 소속 전공으로는 심리학과, 사회학과, 사회복지학부, 정치행정학과, 광고홍보학과, 법학과, 경제학과, 청소년학과가 있습니다."
  },
  {
    "input": "심리학과 소개해봐",
    "output": "1994년에 개설되어 최근 20주년을 맞은 한림대 심리학과는 탄탄하고 내실 있는 프로그램으로 현재도 알차게 성장 중입니다. 특히, 사회, 성격, 지각, 인지심리학과 같은 기초 심리학에서부터 범죄, 임상, 임상신경, 조직심리학과 같은 응용심리학, 그리고 심리학의 기본이 되어주는 심리학 실험법과 심리측정에 이르기까지 다양한 심리학의 분야를 폭넓게 다루고 있습니다. \n\n그동안 교수님들을 비롯한 졸업생 및 재학생들이 열심히 노력한 결과, 현재 많은 졸업생들이 학계, 병원, 상담센터, 학교, 재난 현장, 그리고 그 외의 많은 장소에서 심리학 전문가로서의 역할을 해내고 있습니다.\n\n하지만, 심리학의 가장 큰 매력은 다양성에 있습니다. 심리철학과같이 매우 인문학적인 모습에서부터 생리심리학, 신경심리학과같이 자연과학에 가까운 모습을 갖는 다양한 종류의 심리학이 있습니다. 보다 넓은 시야와 열린 마음으로 심리학을 접한다면, 앞서 열거한 분야를 넘어서는 더 창의적이고 다양한 분야에서 자신의 전문성을 개발할 수 있으리라 믿습니다."
  },
  {
    "input": "심리학과 교육 목표가 뭐야",
    "output": "심리학 전공은 기초적인 심리학 분야에 대한 지식과 연구방법론을 습득하고 실생활의 장면에서 심리학 지식들을 활용할 줄 아는 인재를 양성하는 것을 목표로 합니다. \n1. 자신 및 타인의 심리적 건강에 봉사합니다 : 사회가 점점 복잡해지면서 개인의 각종 심리적인 문제가 심각해지고 있습니다. 자신 및 타인의 다양한 심리적 문제 해결에 도움을 주고, 심리적 건강 증진에 봉사할 수 있는 기초를 갖추는 인재를 양성합니다.\n2. 산업체 및 다양한 조직의 발전에 공헌합니다. : 인간에 대한 이해를 바탕으로 이를 학교, 산업과 기타 다양한 장면에서 적용함으로써 이들의 발전에 공헌할 수 있는 인재를 양성합니다. \n3. 강의를 통한 지식 습득과 더불어 실습 중심의 교육을 합니다. : 이를 위하여 실험실을 운영 하여 실험 및 실습을 가능하게 하고, 전공분야별 현장 실습의 기회를 제공하여 습득한 지식을 적절하게 활용할 수 있는 인재를 양성합니다.\n4. 연구 능력 취득과 인재 양성을 합니다. : 연구 방법 및 컴퓨터 활용능력을 학습하여 사회과학 분야의 기본적인 연구 능력을 갖추게 합니다. 연구 방법론의 습득을 통해 졸업 후 연구 기관이나 조사기관에 취업하여 연구에 참여할 수 있는 인재를 양성합니다."
  },
  {
    "input": "심리학과 전망과 진로",
    "output": "심리학은 젊고 성장하고 있는 학문으로서, 발전 가능성이 아주 높은 학문입니다. 우리나라에서도 전문적인 심리학 지식 및 심리학 전공자들에 대한 수요가 점점 증가하고 있으며, 심리학이 사회에 미치는 영향도 계속 증가하고 있는 추세입니다. 심리학 관련 진로는 학위의 종류에 따라 달라지는데, 심리학과 학부 졸업생이 가장 많이 취업하는 곳은 각종 여론조사 기관과 광고 회사, 일반기업체 등입니다. 국가 기관에서 심리학자를 고용하는 경우도 증가하고 있어, 현재 법무부 교정국, 경찰청, 노동청 산하 연구소, 철도청 연구소 등에서 심리학 전공자들이 근무하고 있습니다. \n\n전문가로 활동하기 위해서는 석사 이상의 교육과 수련을 받아야 하는데, 지금 현재 한국에서는 석사 학위 이상인 경우, 임상/상담 심리학자의 수가 가장 많은데, 이들은 주로 병원이나 상담소에서 활동하고 있습니다. 또한 범죄 심리학의 전공한 석사학위자의 경우 경찰청이나 교도소에서의 취업이 가능합니다. 심리학 박사 학위 소지자들의 경우 현재로서 제일 많이 취업하고 있는 곳은 대학교(교수로서)이고 일부는 각종 연구소에 연구원으로 재직하고 있습니다. \n\n그러나 심리학은 다른 학문분야보다 더 많은 다양성과 가능성을 제공하고 있기 때문에, 본인의 능력과 관심에 따라 다양한 분야에서 새로운 영역의 개척이 가능합니다. 범죄심리, 건강심리, 군사심리, 인공지능개발, 정부정책수립, 통일 문제, 교육 문제, 청소년 문제, 도시 교통 문제, 환경 문제 등에 거의 모든 사회 분야에 심리학에 대한 수요가 계속 증가하고 있어서, 학위에 관계없이 사회의 각 분야에서 심리학 관련 취업 가능성이 점차 증대할 것으로 예상됩니다."
  },
  {
    "input": "사회학과 소개좀 해봐",
    "output": "한림대 사회학과는 과학기술과 산업, 문화와 여성·가족, 종교와 다문화, 사회조사분석과 빅데이터, 도시와 환경 등 각 분야에서 독보적인 위치를 차지하고 있는 교수님들과 함께 이론과 방법론을 체계적으로 배우며, 토론과 발표, 현장학습, 조사연구, 독서토론 등 학생들의 자기주도적인 탐구능력 강화를 위한 방식으로 교육이 이루어지고 있으며 동아리와 학생회 활동이 매우 활발한 학과입니다."
  },
  {
    "input": "사회학과 전망과 진로 알려줘봐",
    "output": "종합적이고 비판적이고 당대 사회의 흐름에 민감한 사회학의 특성을 살려 구체적인 분야를 겨냥한 전문가 양성과 더불어 현대사회에 대한 종합적인 이해를 갖춘 교양있고 창의적인 인재를 키우고 있습니다. 최근에는 전통적인 진출분야인 미디어, 리서치회사, 연구소, NGO, 일반기업 등과 함께 청소년시설, 문화재단, 콘텐츠제작업체, 빅데이터 회사, 사회적경제조직 등에 취업하고 있으며 대학원 진학이나 공무원 등 공공부문으로도 일정 수가 진출하고 있습니다. 어느 분야로 진출하든 사회학과 출신으로서의 유니크한 매력을 발휘해 능력을 인정받고 있습니다."
  },
  {
    "input": "사회복지학부 소개해봐",
    "output": "산업화와 현대화에 따른 가족 및 아동문제, 노인문제, 청소년 비행문제 등 많은 사회문제들의 해결방법을 연구하고, 이를 실천하는 학문으로 개인 심리적, 사회구조적인 문제가 속출하는 현대사회에서 반드시 요청되고 그 필요가 확산되어 가는 미래지향적인 학문입니다. 또한 사회복지는 이상적 사회추구를 구체화시키려는 과학적이고 체계적인 학문으로서, 복지에 관련되는 제반이론 교육과 실습교육을 통해 실천 학문으로서의 목적을 달성하는 데에 목적이 있습니다."
  },
  {
    "input": "사회복지학부 교육목표가 뭐임",
    "output": "한림대학교 사회복지학부는 변화하는 사회에 대응하여 사회구성원의 복지 향상을 위해 정부, 공공기관, 국내 및 해외 사회복지기관에 종사 할 사회복지 전문 인력을 양성하는데 교육목적을 두고 있습니다. 한림대학교 사회복지학부는 일찍이 1982년 학과 출범초기부터 현장밀착형 핵심전문인력 양성을 위해 노력해 왔으며, 그 결과 졸업생들이 다양한 사회복지현장에서 중심적 역할을 수행하고 있습니다. 최근에는 세계화 된 환경에서 글로벌 전문 인력으로서의 자질 제고를 위해 어학교육, 영어강의 개설, 해외연수 및 교환학생 기회 제공 등 최선의 교육 환경을 갖추었습니다. 한림대학교 사회복지학부는 최고수준의 교수진과 열정적인 교육, 차별화된 풍부하고 체계적인 교육프로그램, 진지하고 열정적인 학생들의 학습 분위기, 학과경쟁력에 기반 한 교육부 지원 프로그램의 지속적 선정 및 학교의 탄탄한 재정적 지원에 힘입어 계속 발전하고 있습니다."
  },
  {
    "input": "사회복지학부 전망과 진로 알려줘",
    "output": "사회복지학부의 전망과 진로는 사회복지전담공무원, 의료사회복지사, 학교사회복지사, 교육복지사, 정신보건사회복지사, 교정사회복지사, 군사회복지사, 산업 사회복지사, 의료관광 전문인력, 사회복지자원마케터, 아동복지 전문인력 등이 있습니다."
  },
  {
    "input": "정치행정학과 소개",
    "output": "정치행정학과는 1989년에 설립된 정치외교학과가 2006년 9월 1일자로 발전적으로 개편되어 새롭게 출범하는 학과입니다. 기존의 정치학 및 국제정치학과 행정학을 통합적으로 운영함으로써, 학생들에게 학문적인 동시에 실용적으로 다각적인 학습기회를 제공함으로써 다양한 분야로의 사회진출을 뒷받침하기 위한 것입니다.\n기존의 한국의 대부분 대학들의 행정학과는 정치학 교과과정과 경제, 경영 등 여러 학문영역을 적당히 섞어놓거나 그렇지 않으면 실제 학생들의 공직진출과는 무관하게 다소 기술적인 차원의 좁은 ‘행정학’ 영역의 과목들만을 많은 부분 중복적으로 4년 내내 공부하게 함으로써 21세기 한국의 행정가들이 갖추어야 할 폭넓은 식견과 시야를 제공하는 프로그램을 갖추지 못하고 있습니다.\n이에 비하여 한림대 정치행정학과는 학생들의 공직진출에 필수적인 교과목들을 포함한 실용적인 커리큘럼을 갖추되 이와 동시에 민주화이후 더욱 정치화되고 국제화되어가는 한국의 행정과 정치분야의 인재들이 반드시 갖추어야 할 덕목으로서 넓은 사회인문적 시야와 다각적인 전문지식을 제공하는 교과과정을 운영합니다.\n이를 위해 정치행정학과의 교과과정은 불필요하고 지루할 정도로 중복적인 행정학 과목들만으로 이루어진 단순하고 편중된 대부분의 기존 행정학과들의 교과과정을 탈피하여 행정의 상위개념으로서의 정치, 그리고 그 외연으로서의 외교 및 국제질서에 대한 교육을 필수적인 행정학 교육과 함께 제공하도록 짜여져 있습니다.\n또한 이러한 융합적인 정치행정 교과과정을 더욱 보완하기 위하여 우리 학과는 우리 한림대에 이미 있는 다른 학과들에서 제공되는 유관과목들을 우리학과의 전공학점으로 인정하는 제도를 폭넓게 시행합니다.\n법행정학부, 경영대학, 사회대의 사회학과 및 사회복지학과, 그리고 인문대학의 철학과와 사학과에서 개설되는 과목들을 포함해 학생들의 공직진출을 위한 시험준비에 유익한 총 12과목에 대해 정치행정학과의 학점으로 인정함으로써, 학생들의 필요와 목표의 다양성을 권장하고 지원하는 “열린 학과”를 지향하고 있습니다."
  },
  {
    "input": "정치행정학과 전망과 진로 말해",
    "output": "정치행정학과의 교육과정은 졸업생들이 크게 정치, 행정, 외교, 언론 등 크게 네 분야에서 활동할 수 있는 능력을 키우는데 집중됩니다. 정치가, 행정가, 외교관 등 공직에 진출할 수 있으며, 이를 뒷받침하기 위해 행정고시와 외무고시 및 중앙 및 지방의 일반 행정직, 외무직 등의 공무원시험에 도움을 줄 수 있는 커리큘럼과 함께, 일단 그러한 공직에 진출했을 때 이 사회의 정치인과 행정가 그리고 외교관으로서 갖추어야 할 기본지식과 교양을 함양하는데 필요한 교과과정을 제공하는 것이 정치행정학과의 목적입니다.\n또한 정치행정학 교과과정은 학생들이 졸업후 언론인으로서 진출하는데 크게 도움이 될 것입니다. 언론인에게 요구되는 대학교육은 언론 자체에 대한 학습은 아닙니다. 언론인에게 필요한 것은 나름의 전문적 영역에 대한 심도있는 지식입니다. 언론이 이 사회에서 수행하는 역할의 핵심은 정치와 행정 그리고 통일문제 등 대외관계에 대한 관찰과 감시 그리고 비평과 비판입니다. 따라서 한국의 주요 언론매체의 주요 보도영역인 정치, 행정, 그리고 외교분야를 이끌고 있는 언론인들의 대학 전공분야가 정치외교학과 행정 분야 등인 것은 매우 자연스러운 일입니다. 그런가 하면 정치행정학과의 졸업생들은 정치와 행정 및 외교 분야의 대학원에 진학하여 장차 학자 및 교육자로서 활동할 수 있습니다. 본교에도 석사과정 및 박사과정이 개설되어 있으므로 본교 대학원에 진학할 수도 있고, 자신이 원하는 한국내 다른 대학의 석사 및 박사과정 또는 미국과 일본, 중국 등을 포함한 외국의 대학원과정에 진학하여 석사 혹은 박사학위 과정에서 공부할 수 있습니다.\n이처럼 정치행정학과의 졸업생들에게 일차적으로 권장하는 미래는 정치, 행정, 외교, 그리고 언론과 학문 분야에서 이 사회에 기여하고 비전을 제시하는 역할입니다. 이러한 꿈을 학생들이 이룩할 수 있도록 그리고 그 준비를 위해 알찬 캠퍼스생활을 할 수 있도록 정치행정학과는 최선의 노력을 할 것입니다."
  },
  {
    "input": "광고홍보학과 소개해",
    "output": "한림대 광고홍보학과는 국내 대학 최초, 유일하게 2008년, 국제광고협회 (IAA; International Advertising Association)로부터 IAA 인증교육기관 (IAA Accredited Institution)으로 공인을 받습니다. IAA 인증은 현재 미시간주립대학교, 플로리다대학교 등 광고홍보 분야의 주요 대학을 포함하여 전 세계 57개 기관에게만 부여된 것으로, 한림대 광고홍보학과가 IAA인증을 획득했다는 것은 이른바 국제적 기준에 부합되는 교육 프로그램임을 인정받았다는 것을 의미합니다. 광고와 홍보 관련 산업의 실무 경험이 있는 우수한 교수진과 최고의 실험실습 시설을 갖추고 있습닌다. 또한 국내대학 최초, 아시아에서 두 번째로 깐느 국제 광고제 영라이온즈 은상 수상은 물론, 뉴욕 페스티벌, 클리오 국제광고제, 부산국제광고제, 국제광고협회(IAA) 인터애드 학생 경쟁 부분 수상 등 국내 및 해외 공모전에서도 뛰어난 활약을 보였으며, 재학생들의 뛰어난 활약은 교내에 설치된 ‘Wall of Honor 명예의 벽’에 수상자들의 이름을 계시하여 동기부여의 기회를 제공하고 있습니다."
  },
  {
    "input": "광고홍보학과 교육 목표",
    "output": "광고홍보학과의 비전은 아시아 광고홍보 교육의 허브가 되는 것이며, 이를 위한 목표는 국제 경쟁력을 갖추고 실무에 강한 스마트 광고홍보 전문 인재 (star)의 양성입니다. 이를 위한 추진 전략으로는  스마트 광고홍보 특성화, 프로그램의 국제화, 지역 및 산업협력의 강화가 있습니다."
  },
  {
    "input": "광고홍보학과 스마트 광고홍보 특성화가 뭐야",
    "output": "한림대학교 광고홍보학과의 추진 전략중 하나인 스마트 광고홍보 특성화는 스마트 광고홍보 전문 교육 커리큘럼 구축, 스마트 광고홍보 이론, 기획, 실습을 위한 인프라 구축, 디지털 콘텐츠, 컴퓨터공학, 인문학 전공과의 융합 교육, 교과자문위원회의 운영, 우수 신입생 유치를 의미합니다."
  },
  {
    "input": "광고홍보학과 프로그램의 국제화에는 어떤게 있어?",
    "output": "한림대학교 광고홍보학과의 추진 전략중 하나인 프로그램의 국제화는 국제광고협회(IAA)인증 프로 그램에 대한 교육강화 및 홍보, 해외자매대학 및 글로번 기업의 연수 및 인턴십 프로그램 운영, 학생들의 국제공모전 도전에 대한 지원 및 장학 프로그램 마련, 해외 우수학생의 적극 유치 추진을 의미합니다."
  },
  {
    "input": "광고홍보학과 지역 및 산업협력의 강화가 뭐야?",
    "output": "한림대학교 광고홍보학과의 추진 전략중 하나인 지역 및 산업협력의 강화는 산업체 및 지역 전문가의 겸임교수 활용 및 교과자문위원회 참여, 산학협력 프랙티컴(캡스톤) 수업의 강화, 전문가 및 졸업생과 재학생간의 멘토링 프로그램 운영, 인턴십 강화를 통한 현장실무 능력을 의미합니다."
  },
  {
    "input": "광고홍보학과 전망과 진로를 말해줘",
    "output": "졸업 시 정규과목과 일정 학점 이상 취득 시 국내 대학교 유일의 국제광고협회의 공식 자격증인 IAA Diploma 를 수여하며 광고대행사 또는 홍보대행사, 기업의 마케팅/광고/홍보 부서와 조사 분야로의 진출이 활발한 편입니다."
  },
  {
    "input": "법학과 소개해봐",
    "output": "한림대학교 법학과는 1988년 법학부에서 출발하여 그동안 내적 · 외적으로 눈부신 발전을 이루어 왔습니다. 현재 우리 학부에는 다양한 실무 경험과 우수실적 및 학술경력을 소지한 11명의 전임교수님과 3명의 겸임교수님이 교육을 담당하고 계십니다.\n25년이라는 비교적 짧은 역사이지만, 그동안 한림대학교 법학과의 졸업생들은 법조계는 물론 행정직 · 법원직 · 경찰직 등의 공직분야와 금융계 · 실업계 · 학계 등 다양한 분야에 진출하여 그 탁월한 능력을 인정받고 있습니다.\n앞으로도 한림대학교 법학과는 풍부한 인간성과 창조성을 바탕으로 법률지식과 행정실무능력을 두루 갖추어 우리 사회의 발전에 기여할 수 있는 유능한 인재들을 더욱 더 많이 배출하도록 노력하겠습니다. 이를 위해 모든 교수님들의 열성적인 지도와 각종 공직시험 준비를 위한 전문과 정의 운영 및 다양한 장학제도를 마련하고 있습니다.\n한림대학교 법학과의 계속적인 발전을 위해 여러분 모두의 관심과 성원을 부탁드립니다."
  },
  {
    "input": "법학과 교육목표",
    "output": "법학은 법의 숭고한 이념과 다양한 법 분야에 대한 고도의 전문지식을 습득하도록 하여, 법치주의의 신장과 인권 보호에 기여할 우수한 법조인, 법학자, 법 실무가 등 법조 및 공직분야에서 이 시대를 이끌어 갈 인재를 양성함을 목표로 합니다.\n한림대학교 법학과의 법학전공에서는 체계적이고 다양한 법학교육을 통하여 법률과 관련된 문제해결능력과 실무능력을 배양하며, 사회생활의 각 분야에서 필요로 하는 유사 법조직역 전문가와 유능하며 민주적인 공직자를 양성하여 배출하고자 합니다."
  },
  {
    "input": "법학과 전망과 진로 말해",
    "output": "법학전공자들은 법조·공직분야 뿐 아니라 금융계나 민간기업, 학계·연구계 등 매우 다양한 분야로의 진출이 가능합니다.\n한림대학교 법학과를 졸업한 상당수의 학생들은 공무원시험 등을 통해 중앙 및 지방 공무원, 법원·검찰 공무원, 사법경찰관리 등의 공직분야로 진출해 왔습니다. 그 밖에 금융계, 기업계 등 민간 분야에 있어서도 법학과 관련된 다양한 영역으로 활발하게 진출하고 있습니다. 한림대학교 법학과는 각종 공직시험을 준비하는 학생들을 위해 고시반인 ‘한림헌’을 별도로 설치, 운영해 왔으며, 개인 학습공간과 함께 각종 수험정보 및 자료를 제공하고 있습니다."
  },
  {
    "input": "경제학과 소개해봐",
    "output": "경제학은 사람들이 매일 접하는 일상생활의 문제를 다루는 학문으로, 우리가 살아가는 세상을 객관적으로 이해하는 데 매우 유용합니다. 사회과학의 핵심 분야인 경제학의 주요 과제는 한정된 자원을 효율적으로 사용하는 과정에서 발생하는 여러 문제를 탐구하는 것입니다. 효율성과 공평성을 동시에 지향하는 실천적 사회과학으로서 경제학은 지난 250년 동안 이론적 체계가 정교하게 확립되고 실증적 적용 범위가 점차 확대되어 온 결과, 지금은 ‘사회과학의 여왕’으로 자리매김하고 있습니다."
  },
  {
    "input": "경제학과 교육 목표 알려줘",
    "output": "경제학과는 인간의 기본활동인 경제활동을 분석, 연구함으로써 더욱 풍요로운 삶을 영위하도록 하고, 나아가 지구촌 시대의 국제화 및 블록화에 대한 정확한 이해를 도모하는 동시에, 우리나라가 세계경제에서 국가경쟁력을 갖추고 선진복지국가로 발돋움하기 위한 이론적 기초와 실증적 분석능력을 교육하는 것을 목표로 합니다.\n이를 통해 세계경제의 글로벌화와 인류의 복지 및 공존에 이바지해야 한다는 확고한 가치관을 토대로 경제학적 분석능력을 갖추고 경제 및 사회 각 분야에서 활약할 기업가형 직무능력, 혁신가형 도전능력, 전문가형 기술능력 및 연구자형 탐구능력을 두루 갖춘 인재를 육성하고자 합니다."
  },
  {
    "input": "경제학과 인재상이 뭐야",
    "output": "◦ 경제자료 분석, 기업조직 및 전략, 국제무역 등과 관련한 ‘직무능력’을 갖춘 기업가형 인재\n◦ 거시경제, 화폐금융, 환율과 국제금융, 경제성장 등과 관련한 ‘도전능력’을 갖춘 금융혁신가형 인재\n◦ 국가재정, 조세, 환경, 보건, 법경제학 등과 관련한 ‘기술능력’을 갖춘 정책전문가형 인재\n◦ 경제수학 및 통계학적 분석, 미시경제이론, 게임이론 등과 관련한 ‘탐구능력’을 갖춘 연구자형 인재"
  },
  {
    "input": "청소년학과 소개좀",
    "output": "청소년학 전공은 청소년의 삶과 이를 둘러싼 환경에 대한 사회과학적인 이해를 바탕으로 향후 청소년을 지도할 수 있는 역량을 갖춘 전문가를 배출하고자 심리학과, 사회학과, 사회복지학부에서 함께 운영하는 협동전공 과정입니다."
  },
  {
    "input": "청소년학과 교육 목표는?",
    "output": "청소년학 전공은 청소년기에 경험할 수 있는 활동이 점점 더 다양해지는 반면 청소년 문제는 더욱 심각해지는 상황에서 수요가 급증하고 있는 청소년 지도 분야의 전문인력을 양성하는 것을 목표로 합니다."
  },
  {
    "input": "청소년학과 인재상이 뭐야",
    "output": "적극적이고 주체적인 삶을 위해 청소년의 잠재력이 최적으로 발휘될 수 있도록 도와주는 청소년지도사,  청소년기에 발생하는 문제를 진단하고 해결책을 제시할 수 있는 전문적인 지식과 기술을 가진 청소년지도사"
  },
  {
    "input": "청소년학과 목표 역량을 알려줘",
    "output": "청소년지도사 2급 자격증 취득, 청소년 활동 기획, 운영, 관리, 청소년 관련 프로그램 개발과 진행, 부모, 교사들과의 갈등 해결 및 학교 문제에 대한 개입 및 처리 능력"
  },
  {
    "input": "청소년학과 졸업 후 진로는?",
    "output": "청소년 수련관 등 수련시설, 청소년문화의집, 청소년활동진흥센터, YMCA 등 청소년 관련 NGO, 청소년 지도교사, 청소년 캠프 관련 기업체, 지역아동센터, 청소년 상담복지센터, 청소년 보호시설, 보호관찰소 및 소년원, 청소년 일시/중장기 쉼터 등"
  },
  {
    "input": "청소년학과 이수 요건이 뭐야?",
    "output": "1. 청소년학 전공을 이수하기 위해서는 복수전공 교칙에 따라 33학점을 이수해야 함 \n2. 전공 필수: 없음\n3. 부전공 필수: 없음\n4. 졸업논문: 없음\n5. 전공인정: 심리학과, 사회학과, 사회복지학과에서 졸업요건 전공과목으로 이수한 경우 청소년학 전공 과목으로 중복 계산할 수 없음"
  },
  {
    "input": "청소년지도사 자격증 취득기준이 어떻게 돼?",
    "output": "자격증 명칭은 청소년지도사 2급이며, 응시 자격은  2급 청소년지도사 자격검정에 필요한 과목(교과과정표에 표시된 교과목)을 모두 이수한 자입니다. 시험 과목은  필기시험(2급 청소년지도사 자격검정에 필요한 8개 교과목 이수자는 면제), 면접시험이 있으며 합격 기준은  면접 점수 10점 이상 (15점 만점)입니다."
  },
  {
    "input": "경영대학 소속 전공은?",
    "output": "경영대학 소속 전공에는 경영학과, 금융재무학과, 스타트업비즈니스(융합)전공이 있습니다."
  },
  {
    "input": "경영학과 소개 해줄래?",
    "output": "한림대학교 경영학과는 1984년에 설립된 40여 년의 전통을 지닌 학과이며 2007학년도부터 경영학부로 확대하면서 경영학 전공과 의료경영 전공으로 분리되었고 2010년에는 컨벤션관광 경영을 추가하여 학생들의 적성과 진로에 맞는 교육을 실시하였습니다. 그리고 2016학년도 부터는 학제간 융합을 강조하는 사회의 요구에 맞추어 다시 경영학 전공, 의료경영 전공, 컨벤션관광 경영 전공을 경영학과로 통합하여 학생들을 가르치고 있습니다."
  },
  {
    "input": "경영학과 교육목표 부탁해",
    "output": "융합 학문으로 통합된 경영학과는 비즈니스인텔리젼스, 글로벌혁신전략, 의료경영, 회계/세무, 마케팅기획, 관광경영의 6개 트랙이 유기적으로 협력하여 경영학 제 분야의 이론과 실무 교육을 담을 수 있는 교육과정을 설계하고 있습니다. 여기서 인본 경영이란 경쟁이 강조되는 대기업 중심의 고전 경영학 교육을 확장하여 인간의 존엄성과 가치가 중시되며 상생과 공유가 강조되는 자본주의 5.0 시대를 이끌어갈 경영인을 양성한다는 뜻을 의미하고 있습니다. 이를 통해 한림대학교 경영학과는 경쟁 지상주의를 넘어 인간과 생명의 존엄성 회복을 통한 공유와 상생의 경영학 재창조를 시도하고 소통, 상생, 융합을 강조하는 본교의 심비우스적 인재상과의 정합성을 추구하고 있습니다.현재 한림대학교 경영학과가 추구하는 특성화 분야는 경영학 전공 교육을 토대로 기본적인 사업계획서 작성 역량을 함양하고 의료경영과 관광경영 등 웰니스 서비스 분야에서 쌓아온 역량을 기반으로 서비스 창업에 특화된 인재를 양성하는데 있습니다. 또한 경영학과의 교육은 기업이 요구하는 역량을 기반으로 사업기획 역량을 키우는 것을 목표로 하고 있으므로 창업이 아니더라도 CEO를 꿈꾸거나 기업체 취업 및 자영업을 희망하는 학생들에게도 적합한 학과라고 할 수 있습니다"
  },
  {
    "input": "경영학과 전망과 진로 알려줘",
    "output": "사회가 요구하는 융합 교육을 받은 경영학과 졸업생들은 일반 제조업체는 물론 은행, 백화점, 보험회사, 무역회사와 같은 서비스 업체에 취업하기가 용이하며 경제와 산업의 문제를 담당하고 있는 정부부처, 기업 경영 문제를 연구하는 경영 및 산업 연구소에서도 자기의 능력을 마음껏 펼칠 수 있습니다. 최근 우리 사회가 고령화 사회로 접어들고 삶의 질에 관심이 점차 증가하고 있는 점을 감안할 때 요양병원, 너싱홈, 의료기기회사, 관광 시설, 리조트, 호텔, 항공사, 여행사와 같은 웰니스 서비스 분야에도 취업 기회가 확대될 것으로 기대하고 있습니다. 또한, 공인회계사, 세무사와 같은 전문 직종에 진출하는데도 유리한 위치에 있다고 말씀드릴 수 있습니다."
  },
  {
    "input": "금융재무학과 소개해봐",
    "output": "금융재무학과는 기업과 개인의 재무관리 및 투자 그리고 금융기관의 활동을 교육과 연구의 주요 대상으로 삼습니다. 국내 대부분의 대학에서 금융재무학을 경영학의 한 분야로 취급하는 것과는 달리 한림대학교에서는 금융재무학과를 경영대학 내의 독립된 학과로 운영함으로써 현대 경제에서 차지하는 금융과 재무의 중요성에 주목합니다.\n 21세기 들어 우리나라 기업들은 과거 규모 위주의 부실 경영을 탈피하여 수익성에 입각한 새로운 경영방식으로 탈바꿈하고 있으며, 기업들을 지원하기 위한 자율적이고 효율적인 금융기관 경영의 필요성 또한 그 어느 때보다 강조되고 있다. 이에 따라 국가경제 전체적으로 금융재무 전공자에 대한 수요가 급증하는 추세입니다. 금융재무학과는 이러한 사회적·경제적 요구에 부응하여 금융전문가 및 재무관리자 육성 교육에 초점을 맞추고 있습니다."
  },
  {
    "input": "금융재무학과 교육목표는?",
    "output": "금융재무학과는 현대 경제의 핵심인 금융, 기업 경영의 중추인 재무 분야에 진출할 전문인재 양성을 목표로 합니다. 이를 위해 은행/보험, 증권/투자, 기업재무, 글로벌금융, 금융재무학 등 세부 교육트랙을 운영하고 있으며 각 트랙별 이수 교과목을 설정하여 체계적인 진로지도를 수행하고 있습니다. 또한 빠르게 고령사회로 진입함에 따라 종합재무설계의 수요가 급증하고 있어 고령친화 금융서비스 교육을 강화하였습니다. 금융재무 분야의 특화교육을 위해 경영대학 내 독립된 학과로 운영되고 있는 금융재무학과는 글로벌 전문인재 양성을 선도하는 학과로 도약하고자 합니다."
  },
  {
    "input": "금융재무학과 전망이 어때?",
    "output": "- 기업성장에 기여할 재무관리자 수요 증가  \n- 세계에서 가장 빠르게 고령화하는 우리 사회에 대처할 재무설계사 수요 증가 \n- 자산운용을 담당할 금융투자 전문가 수요 증가 \n- 통합적이고 전문적인 금융 지식에 대한 수요 증가"
  },
  {
    "input": "금융재무학과 진로는?",
    "output": "금융재무학과 졸업생들은 은행, 보험, 금융투자회사 등의 금융기관과 기업의 핵심 분야인 자금 및 재무 부서에 진출하여 경력을 쌓을 수 있습니다. 또한 관공서의 재무 및 경제부서에 진출하여 전문가로 성장할 수 있다. 한편 금융재무 분야의 전문자격증 취득은 졸업 후의 취업 및 경력관리에 큰 도움이 될 것입니다."
  },
  {
    "input": "스타트업비즈니스(융합)전공 소개는?",
    "output": "스타트업비즈니스(융합)전공은 우리 대학에 분산되어 있던 창업관련 과목들을 보다 체계쩍으로 통합하여 교육과정으로 학생들에게 제공하기 위하여 2022년 신설된 복수전공 대상 프로그램으로 3개 대학, 7명의 교수님들이 참여하여 운영하는 융합전공 프로그램입니다. 이수하는 학생은 융합학사가 수여됩니다. 지금 세계는 격변하는 기술과 상황 속에서 끊임없이 새로운 일자리가 창출되고 있으며 실무능력을 갖춘 창업기업가가 능력을 발휘할 기회를 부여하고 있습니다. 본 융합전공은 \"열린 경영학, 창의적인 진로개척\"이라는 비전을 가지고 복수전공을 고민하는 미래 창업가를 맞이합니다."
  },
  {
    "input": "스타트업비즈니스(융합)전공 교육목표가 뭐야",
    "output": "스타트업비즈니스(융합)전공 교육목표는 1. 새로운 일자리를 개척하려는 진취적인 학생들에게 다가가기 쉬운 경영 교육 제공 2. 열린 융합 경영 프로그램을 통해 실무능력의 함양과 급변하는 환경에 대응능력을 강화하고 창업기업 진입장벽을 낮춤 3. 복수전공 대상 프로그램으로 학생들의 진로 탐색 가능성을 극대화하며 교육의 유연성을 통해 시대와 보조를 맞추는 교육 4. 동문 창업자와 긴밀한 네트워크 구성으로 동문의 경험을 활용하여 실용 역량을 갖춘 인재 육성입니다."
  },
  {
    "input": "스타트업비즈니스(융합)전공  인재상좀 알려줘",
    "output": "스타트업비지니스(융합)전공의 인재상은 비지니스 모델을 제시하고 실천할 수 있는 능동적, 창의적, 실용적 인재를 양성하는 것입니다."
  },
  {
    "input": "스타트업비즈니스(융합)전공 졸업 후 진로가 어떻게 돼?",
    "output": "본 융합전공에서는 다양한 진로에 대해 열려있습니다. 경영(경영지도사, 마케팅전문가, 여론조사 전문가, 공연기획자), 교육(창업금융 세무 회계 상담, 이러닝 교육설계자/개발자), 개인서비스(세무사, 웹프로듀서, 저작권 에이전트 등) 있습니다."
  },
  {
    "input": "자연과학대학에는 어떤 전공들이 소속되어 있어?",
    "output": "자연과학대학 소속 전공에는 화학과, 생명과학과, 바이오메디컬학과, 식품영양학과, 환경생명공학과, 언어청각학부, 체육학과, 바이오헬스케어융합전공이 있습니다."
  },
  {
    "input": "화학과에 대해서 알려줘",
    "output": "화학은 물질의 구조 및 성질을 규명하고 물질의 변환을 통해 새로운 물질을 창조하고 그 과정을 연구하는 학문으로서 의약품, 신소재, 에너지 등 인류의 복지에 이바지하는 다양한 분야를 이해하고 발전시키는 토대가 되는 학문입니다. 화학의 건실한 기초를 가진 연구 인력은 거의 모든 응용과학기술 분야에 요구되고 있습니다.\n최근 들어 활발하여진 전자산업, 생명공학 및 의약학 등의 기초 연구 분야는 물론 공해와 관련된 환경과학 분야에도 훈련된 화학도의 기여가 크게 증가하고 있습니다. 화학과에서는 학생들이 화학에만 국한되지 않고 화학의 기초를 가진 과학도로서 넓은 관련분야에 진출하여 공헌할 수 있는 자질을 배양하기 위하여 노력하고 있습니다."
  },
  {
    "input": "화학과 교육 목적을 말해줘",
    "output": "한림대 화학과는 중심학문인 화학을 원리부터 이해하기 쉽게 교육하고 학생들이 화학을 다양한 분야에 응용할 수 있도록 교육시키는 것을 최우선 목표로 하고 있습니다. 한림대 화학과의 교육목표는 “공부하면 할수록 즐거워지는 화학”을 모토로 학생 한명 한명을 화학을 이해하는 훌륭한 화학자로 길러 내는 것입니다.\n\n한림대 화학과에서는  화학의 기본원리와 응용체계를 이론과 실험을 통해서 이해할 수 있도록 체계적인 교육과정을 준비하여 화학을 이해하고 응용가능한 인재를 길러내고 있습니다. 특히, 기초지식 교육 후 여러 분야의 문제를 함께 해결함으로써 다양한 분야에 응용이 가능한 핵심지식을 소유한 화학자로 교육하고 있습니다."
  },
  {
    "input": "화학과는 전망이 어때? 화학과 진로에 대해서도 알려줘",
    "output": "졸업생 중 상당수가 외국에서의 연구 활동 후 국내외에서 교수나 연구 요원으로 활동하고 있습니다. 그 밖에 보건원, 병원, 종합상사, 화학공장, 제약부문, 중화학 부문 등 다방면으로 진출하고 있습니다."
  },
  {
    "input": "생명과학과를 소개해",
    "output": "생명과학은 인류의 향상된 삶을 영위하는 데 있어 근간이 되는 건강·의료 및 식량 수급의 문제 해결에 가장 직접적으로 기여하는 종합 학문입니다. 생명과학을 근간으로 하는 생명건강 산업은 초고령 사회 진입에 따른 헬스케어·의료 서비스 수요를 충족하는 미래 신성장동력사업의 핵심을 이룹니다.\n생명과학과는 급속히 축적되고 있는 생물학적 지식을 기반으로 학문 간의 장벽을 허물고 새로운 가치를 창조하는 융·복합 학문으로 발전시킴으로써, 의료·바이오 분야의 맞춤형 고급인재 양성을 위해 노력하고 있습니다.\n본 학과에서는 ‘개인 맞춤형 건강관리 서비스’ 산업 분야의 경쟁력 있는 인재를 양성하기 위해 생명과학 기초 교과목과 선택적 전공 교과목과 생명정보학 및 시스템분석 기술과 관련된 강의 내용을 교육하고 있습니다. 특히, 본 학과의 NGS (대용량 염기서열 분석장비) 기반 유전체 분석 활용 분야를 특성화하고 있으며, 이를 기반으로 대학원의 양적·질적 확대, 학과 운영 개편, 다양한 형태의 산학 공동연구의 활성화와 더불어 연구과정에서 도출되는 실험실습 과목 및 집중 육성분야 위주의 교과과정을 운영 중에 있습니다."
  },
  {
    "input": "생명과학과는 전망과 진로가 어떻게 돼?",
    "output": "생명과학과의 전공이 세분화 되어 생명현상의 새로운 사실이 계속 밝혀질 것이고 이에 생물과학 기술이 축적되어 생명체를 이용한 생물공학의 발전 또한 계속될 것입니다. 이것은 생물산업으로 이어져 인류 삶의 질을 향상시킬수 있을 것이다. 따라서 생물학과 연관 대학원 및 연구소 연구원(의학, 농학, 환경학, 식품학), 관련분야 공무원, 중고등학교 교사 등의 진출이 활발하게 이루어지고 있는 실정입니다."
  },
  {
    "input": "바이오메디컬학과 소개해",
    "output": "최근 눈부시게 발전하고 있는 생명과학 및 생명공학은 21세기에 도약하기 위한 우리 대한민국의 주요 전략산업이며, 세계적으로도 각광을 받는 가공할만한 잠재력을 가진 첨단 핵심 학문입니다. 우리나라는 본격적인 고령화시대에 접어들면서 치매, 당뇨, 암 등 환자 수가 급격히 증가하고 있으며, 이에 따라 질병 진단 및 치료시장이 성장하고 정밀의학이 발전하고 있습니다. 바이오메디컬 분야의 바이오의약품, 유전체분석 및 정밀의료, 분자진단과 줄기세포 등 획기적인 연구 성과는 의생명과학, 제약·바이오 및 헬스케어 분야에 대한 관심의 집중과 획기적인 발전을 불러오고 있으며, 전망이 매우 밝습니다.\n\n바이오메디컬학과는 이와 같이 빠른 속도로 발전하는 생명과학, 생명공학, 의생명과학 등 여러 분야를 아우르는 전문 지식을 갖춘 우수한 인재를 양성하는 목표를 갖고 있습니다, 생명과학과 의·과학분야의 융합에 기반한 교과과정을 운영하고, 심도 있는 실험 및 산학공동연구 등을 통하여 단백질 정제, 유전자 재조합 및 분석, 유전체 분석, 고등생물 세포 및 미생물 배양, 형질전환, 인체 질환 진단 및 치료 방법 개발 기술 등을 연마케 하고 있습니다. 이와 같이 이론과 실험을 균형 있게 교육함으로써 창의성 있는 연구역량을 보유한 의생명과학 전문인력을 양성하고 있습니다.\n\n바이오메디컬학과는 학과 단위로 정부로부터 대형연구과제를 수주하여 학과 교수와 학생이 공동으로 참여·협력하는 연구를 수행함으로써, 의생명과학 기초연구와 진단 및 치료기술 개발연구 성과물의 실용화를 추구하고 있으며, 이를 통하여 전세계적으로 각광을 받고 있는 고부가가치 바이오산업(BT)분야에서 경쟁력을 갖춘 전문인력을 양성·배출하고 있습니다."
  },
  {
    "input": "바이오메디컬학과 전망과 진로 알려줘",
    "output": "바이오메디컬학과는 우리 대한민국의 주요 전략산업이자 급속도로 발전하고 있는 첨단학문인 바이오테크(BT) 분야의 전문지식과 창의성 있는 연구역량을 갖춘 우수한 인재를 배출하여 학교, 연구소, BT 기업체, 의료계 등에 진출시키고 있습니다. 관련분야인 제약·바이오산업은 세계적으로 고령화가 급속히 확장되는 추세이므로 미래가치가 매우 높습니다.\n\n바이오메디컬학과는 유전자조작 및 세포배양기술을 적용한 재조합 단백질의약품, 유전자치료제, (줄기)세포치료제 등의 첨단 바이오의약품과 유전제분석, 체외진단 등의 각종 첨단 바이오기술과 밀접히 관련된 학과이므로 전망이 밝습니다. 제약·바이오산업의 국내 취업 유발효과는 2018년 기준 국내 최대 규모 주력사업인 시스템반도체사업과 대등한 수준이었습니다.\n\n졸업 후의 진로는 전공분야의 심도 있는 연구를 위하여 바이오메디컬 및 생명과학 전 분야 전공의 국내·외 대학원에 진학하거나, 의·치의학교육 및 약학교육 입문검사(DEET/MEET/PEET)에 응시하여  의·치의학전문대학원 및 약학대학으로의 진학이 가능합니다. 그리고 바이오메디컬 및 생명과학 분야의 다양한 기관, 연구소, 회사 등으로 취업이 가능합니다."
  },
  {
    "input": "식품영양학과 소개좀",
    "output": "식품영양학은 식품원료의 이화학적 특성을 파악하여 적절하게 저장, 가공, 제조하는 방법을 다루는 식품과학과 식품의 섭취 후 인체 내에서의 소화, 흡수이용에 대한 영양 생리적 작용을 다루는 영양학을 서로 연계시켜 탐구하는 실용적인 학문입니다.\n\n그러므로 식품영양학 전공은 인간의 생존과 건강유지에 가장 기본이 되는 식품과 영양에 대한 이론과 실험을 익혀 우리의 식생활에 직접 응용할 수 있는 능력을 배양시키고, 과학적인 영양소의 섭취를 통하여 개인, 가정, 사회, 국가차원에서 국민체위 향상과 건강생활 유지에 기여함, 사회와 국가가 요청하는 단체급식과 식품위생\n\n관리 및 식품의 제조와 가공에 대한 전문인을 양성하는데 목적이 있습니다."
  },
  {
    "input": "식품영양학과 교육 목표좀",
    "output": "식품영양학과의 교육 목표는 식품과 영양에 대한 학문적 이론과 실험을 익혀 국민보건향상과 건강생활 유지에 기여하며, 사회와 국가가 요청하는 영양사(단체급식), 영양교사, 식품위생 관리 및 식품의 제조와 가공에 대한 전문인을 양성하는 것입니다."
  },
  {
    "input": "식품영양학과 전망과 진로 알려줘",
    "output": "우선 영양사로서의 취업입니다. 기업체, 병원, 학교 등 단체급식을 실시하고 있는 곳에서는 의무적으로 영양사를 채용하여 단체급식 관리 책임을 맡도록 하고 있습니다. 한편 1994년에 전문직 공무원 채용분야에 5급, 7급, 9급의 식품 위생직이 신설되어 식품위생직 준비에 유리하도록 시험과목들을 교과과정 편성에 반영하고 있습니다. 그리고 졸업 후 학문을 계속하거나 정부출연 연구소 또는 식품관련 기업체의 연구소에 진출할 학생들에게는 대학원 진학의 길이 있습니다. 학생들을 재학 시 교수들의 연구 활동에 참여시켜 연구경험과 실험능력을 갖추도록 하여 국내외의 어느 대학원 생활도 잘 적응할 수 있도록 지도합니다."
  },
  {
    "input": "환경생명공학과 설명좀",
    "output": "산업화사회의 급속한 발전으로 인하여 오늘날 환경문제가 우리에게 중대 관심사가 되고 있으며, 이러한 문제의 해결을 위한 과학기술도 매우 빠른 속도로 발전하여 환경과학은 이제 인간 생활에 매우 중요한 학문의 한 분야가 되고 있습니다. 환경생명공학 전공은 이러한 환경과학의 여러 분야들을 연구, 발전시켜 지구 환경의 개선, 즉 대기, 수질, 토양, 해양, 소음, 진동 및 폐기물 관리에 응용함으로써 환경의 질적인 향상을 도모하는데 목적이 있습니다. 환경생명공학과의 교과과정은 수학, 물리학, 화학, 생물학 등의 기초과학을 터득하게 한 후, 환경과학에 필요한 보다 수준 높은 교과목을 학습하도록 구성되어 있으며 환경과학에 대한 폭넓은 강의와 연구실험, 현장실습 등을 통하여 학생들이 졸업 후 사회참여에 기여할 수 있는 인재를 양성하는 데에 그 목적을 두고 있습니다."
  },
  {
    "input": "환경생명공학과 교육목표는?",
    "output": "환경생명공학과 교과 과정은 환경과학에 필요하고, 보다 고등한 교과목으로 구성되어 있으며 환경과학의 폭넓은 강의와 연구 실험 등을 통하여 본 학과 학생들의 졸업 후 사회 참여에 기여할 수 있도록 인재를 양성하는데 그 교육목적이 있습니다."
  },
  {
    "input": "환경생명공학과 전망과 진로는?",
    "output": "전 세계적으로 “환경”은 다양한 분야의 의사결정에 있어서 매우 중요하게 고려되는 요소입니다. 최근에는 기후변화에 대한 관심이 높아져 환경생명공학 전공자들이 해야 할 일은 계속 증가추세에 있습니다.\n이처럼 앞으로의 환경은 보다 전문적인 분야로서 탄탄한 기초 및 전공 지식을 갖춘 고급 인력을 필요로 하며, 세계의 흐름에 먼저 다가설 수 있는 인재를 원하고 있습니다.  주요 진로로는 공무원 (중앙부처, 지방자치제, 환경기술고시 및 환경행정직), 공기업체 (환경관리공단, 수자원공사, 한국환경자원공사 등), 학교 (환경교사, 과학교사 등), 환경오염방지시설 운영 전문기업, 환경설비 및 장치 전문 제조업체 등이 있습니다."
  },
  {
    "input": "언어병리학과 소개좀",
    "output": "언어병리학은 영아기부터 노인기까지 사람들이 전 생애에 걸쳐 다양하게 경험할 수 있는 말·언어장애를 연구하는 학문으로, 아동기 언어발달장애를 비롯하여, 뇌졸중 또는 치매와 같이 두뇌기능 장애로 인한 신경 말·언어장애, 발음에 문제를 보이는 조음음운장애, 성대와 같은 음성산출기관의 문제로 인해 목소리를 정상적으로 산출하지 못하는 음성장애, 말더듬과 같은 유창성장애, 지적장애 또는 자폐 범주성 장애 등과 같은 발달장애를 동반한 의사소통장애, 청각장애를 동반한 말·언어장애와 같은 의사소통장애의 원인과 특성을 연구하고 이를 평가 및 재활하는 과정을 공부합니다."
  },
  {
    "input": "언어치료사 자격증 취득 어떻게 해?",
    "output": "언어병리학과에서 소정의 과정을 이수하고 한국언어치료협회 주관의 언어치료사 2급 자격증 시험에 합격하는 경우 2급 언어치료사 자격증을 취득할 수 있습니다. 또한 대학원에 진학하여 과정을 이수하고, 언어치료사 1급 자격증 시험과 소정의 수련과정을 마치면 언어치료사 1급 자격증을 취득하게 됩니다."
  },
  {
    "input": "언어병리학과 진로 알려줘봐",
    "output": "언어치료사가 되면 아래와 같이 다양한 교육 및 임상현장에서 여러 전문가들과 함께 일할 수 있는 기회를 갖게 됩니다.\n- 종합병원 및 개인병원 (신경과, 정신과, 소아정신과, 소아과, 이비인후과, 재활학과 등)\n- 어린이집 및 학교 (일반학교 및 특수학교)\n- 복지관 (장애인복지관, 노인복지관, 종합복지관)\n- 각종 발달연구소\n- 사설언어치료실 개원"
  },
  {
    "input": "언어병리학과 전망 좋아?",
    "output": "언어치료사는 영아기부터 노인기까지 사람들이 경험할 수 있는 모든 유형의 언어 및 의사소통장애를 대상으로 합니다. 현재 국내외 각종 기관에서는 언어장애인을 전 인구의 5-10% (최소 200만 명 이상)로 추정하고 있으며, 이들에 대한 언어치료사의 많은 수요가 요구되고 있습니다.\n- 영아기(0-3세) 발달장애 지원 정책으로 인한 언어치료사 수요 증가\n- 학령전기 및 학령기 발달장애(지적장애, 자폐범주성장애, 뇌병변장애, 청각장애 등)의 약 70% 이상이 언어 및 의사소통장애를 동반, 언어치료서비스 최우선 요구\n- 급증하는 성인 및 노인성장애(뇌졸중, 치매 등) 중 약 50% 내외가 말-언어장애를 동반\n\n또한 최근 국제결혼의 급격한 증가로 인해 발생한 다문화가정 자녀들의 경우 최소 50% 이상이 언어문제를 동반하고 있으며, 이 아동들에 대한 언어치료사의 지원 필요성까지 고려할 때 언어치료사의 수요는 계속 증가할 전망입니다."
  },
  {
    "input": "언어청각학부 소속 전공에는 뭐가 있어?",
    "output": "언어청각학부는 언어병리학 전공, 청각학 전공으로 구성되어 있습니다."
  },
  {
    "input": "청각학과 소개해봐",
    "output": "청각학은 현재 우리나라의 장애 출현율 3위인 청각장애와 관련된 융복합적 성격의 학문입니다. 청각학은 청각장애인을 위한 청능평가, 청능훈련, 청능재활에 대하여 이론과 실습을 병행하여 공부하게 됩니다. 전공 분야는 기초청각학, 임상청각학, 재활청각학, 보청기, 인공와우 및 대체공학, 아동청각학, 노인청각학, 산업청각학, 교육청각학입니다. 이러한 분야의 전문적 지식을 바탕으로 청각학은 정상적 청력을 보존하고, 비정상적 청력상태를 예방, 평가, 재활 할 수 있는 전문가 교육을 이수하게 됩니다. 국내외적으로 청각장애인이 증가하는 추세이며 그에 따른 사회적 관심과 요구가 급격히 확대되어가고 있습니다. 신생아 청각선별검사의 대중화 및 인구의 고령화에 따라 선천성 및 노인성 난청 인구도 증가할 뿐 아니라 산업화와 개인용 음향기기사용의 확대로 소음성 난청도 증가하고 있어 우리나라 총인구의 약 10%인 500만명 정도가 청각장애인 것으로 추정합니다. 따라서 진로의 영역이 다양하고 미래가 밝은 학문입니다."
  },
  {
    "input": "청각학과 교육목표는?",
    "output": "현장적합형 실무능력을 갖추고 창의적 지식과 기술을 융ㆍ복합할 수 있는 국제적 수준의 교과과정과 실습교육으로 문제해결 능력을 갖추고 창의적 지식과 기술을 구현하는 글로벌한 청각학분야의 전문인을 양성하는 것에 교육목표를 두고 있습니다."
  },
  {
    "input": "청능사 자격증 취득 방법좀",
    "output": "청각학 전공 이수 후 자격시험에 합격하면 2007년에는 미래의 유망직종 2위에 선정(2007년 1월 3일자 중앙일보)된 청능사 자격증을 취득할 수 있습니다. 청능사 자격증 시험은 제 3학년을 수료한 후에 가능하며 청각학 학사 학위이상의 학력과 경력을 요구합니다. 전공이수과정에서 자격증 시험을 위한 검정과목인 기초, 임상, 재활청각학과 실습과목을 준비하게 됩니다."
  },
  {
    "input": "청각학과 진로는?",
    "output": "청능사 자격 취득 후 수련교육과정을 마치면 종합병원 및 개인병원, 보청기 및 인공와우 와 관련된 외국기업, 청각장애 평가 관련 기기 회사, 연구소, 난청재활센터, 청각장애학교와 관련된 복지관, 소음관리사업장 등 다양한 관련 분야에 취업할 수 있고 개인 난청센터를 개업할 수도 있습니다. 현재 졸업생의 90% 이상이 수도권 및 전국지역의 전공과 관련된 분야에 취업하고 있습니다."
  },
  {
    "input": "체육학과 소개해봐",
    "output": "체육학과는 100세 건강사회를 대비하여 전공 융합형 특성화 교육과정을 개발하고, 지역사회와의 동반 성장을 선도하는 통합시스템을 구축하며, 산학협력 취업을 위한 융복합체제를 구축하여 창조적 지성과 풍부한 인성을 겸비한 스포츠헬스케어 전문인력을 양성하는데 교육목표를 두고 있습니다.\n\n체육학과는 1984년도에 학과를 설립하여 사회에서 요구하는 전문인력을 양성하기 위하여 그 동안 많은 노력을 해왔으며, 그 결과 대학교수, 연구원, 중등교사, 공무원, 체육관련 단체의 행정직 및 관리자, 스포츠센터지도자, 스포츠산업체 종사자 등 다양한 분야에서 전문가로서 활동을 하고 있습니다. 우리 체육학과는 한림대학교 내에서도 올바른 인성 함양과 함께 취업률이 높은 학과로도 유명합니다.\n\n체육학과는 훌륭한 교수진과 교육과정, 최신의 실험실습기자재, 최고의 스포츠시설 및 장비를 구비하고 있으며, 학생들의 취업기회를 촉진하기 위하여 체계적이고 차별화된 현장맞춤형 교육프로그램을 운영하고 있습니다."
  },
  {
    "input": "체육학과 교육목표는 뭐야?",
    "output": "체육학과의 비전은 100세 건강시대를 대비한 스포츠헬스케어 현장 맞춤형 인재양성이며, 인재상은 창조적 지성과 풍부한 인성을 갖춘 현장 맞춤형 운동과학 인재입니다. 이러한 스포츠헬스케어 인재양성을 위하여 트랙별 교육특성화 프로그램을 개발하고 있습니다. 즉, 트랙별 다양한 교과과정을 개발하고 있으며 스포츠지도사, 건강운동관리사, 스포츠경영관리사, 운동과학 전문가 등을 양성하기 위한 교과과정을 개발하고 있습니다.\n\n체육학과의 구체적인 교육목표는 다음과 같습니다.\n첫째, 우리나라 체육을 선도할 지도자가 되기 위하여 전반적인 체육학이론과 실기를 배양한 다.\n둘째, 신체활동 증진을 위한 첨단 지식과 기술을 적용할 수 있는 전문능력을 배양한다.\n셋째, 창조적 지성과 풍부한 인성을 겸비한 체육인을 양성한다."
  },
  {
    "input": "체육학과 전망과 진로는 뭐가 있어?",
    "output": "21세기 사회는 개인의 행복한 삶을 추구하는 복지사회로서 개인과 사회의 건강유지가 주된 이슈입니다. 이에 따라 본 학과의 비전은 100세 건강사회를 대비한 스포츠헬스케어 현장 맞춤형 인재양성이며, 이를 위하여 트랙별 교육특성화 프로그램을 운영하고 있습니다. 체육학과의 구체적인 진로는 임상 및 건강분야의 건강운동관리사, 전문체육 및 생활체육 스포츠지도사, 특수체육 지도자, 선수트레이너, 교수 및 교사, 공무원, 체육관련 단체 행정직 및 관리자, 스포츠경영컨설팅회사, 연구소, 스포츠센터 관리자, 스포츠산업체, 리조트 등으로 다양한 분야에 진출하여 그 역량을 발휘하고 있습니다."
  },
  {
    "input": "바이오헬스케어융합전공은 뭐야?",
    "output": "본 전공은 화학과, 생명과학과, 바이오메디컬학과, 식품영양학과, 환경생명공학과, 언어청각학부, 체육학과가 연합하여 개설한 전공으로서 산업체 참여 위주의 교육과정을 운영하고 바이오헬스케어 분야의 융합인재를 육성하여 취업으로 연계하고자 개설되었습니다.\n헬스케어 산업에 대한 전반적 이해와 융복합적 교육을 통해 연계분야의 현황과 문제점을 정확히 파악할 수 있도록 하고, 구성원으로서 자신이 담당할 실무 능력을 갖추어  문제를 해결하며 조직 내 협력할 수 있는 유연한 사고와 소통능력을 갖춘 인재를 양성하고자 합니다."
  },
  {
    "input": "바이오헬스케어융합전공 전망이랑 진로 알려줘",
    "output": "바이오헬스케어분야 기업으로의 취업, 생명과학 및 제약 연구개발, 바이오데이터의 수집 및 분석서비스 분야(바이오제품의 개발 및 생산, 유전체 및 단백체 분석을 통한 맞춤형 치료제 개발 등)로의 진출이 가능합니다."
  },
  {
    "input": "의과대학 소개해줘",
    "output": "의예과 2년은 의료인 또는 의과학자에게 필요한 원만한 인격과 소양을 갖출 수 있는 일반교양 교육, 의학전공에 기반이 되는 자연과학의 기초 학문 습득, 급변하는 세계화에 동참하여 인류의 건강에 공헌할 수 있도록 국제 감각과 외국어 능력을 향상시키는 교육을 지향하고 있습니다. 따라서 건전한 윤리적 판단을 구비한 의사, 새로운 지식과 의술을 끊임없이 체득하고 향상시키는 의사, 국제화와 리더십을 통하여 인류 건강에 기여함으로서 자아실현을 추구하는 의사를 배출하는데 목적을 두고 있습니다.\n의학과 4년 과정은 의학 지식, 기술 습득은 물론 환자를 위한 합리적이고 최선의 의학적 결정을 할 수 있는 전문능력을 갖출 수 있도록 문제해결식 교육과 환자 중심의 교육을 지향하며 자기주도적인 자율 및 팀별 학습이 이루어지도록 하고 있습니다. 또한 미국 Columbia-Cornell University를 비롯한, 미국, 유럽, 일본 등의 의과대학 부속병원에 매년 교환학생을 파견하여 국제적 감각을 익히고 시야를 넓히도록 배움의 기회를 제공하고 있습니다. 의학 지식 외에도 의사로서의 도덕적, 인성적 자질을 향상시키고 사회의 한 구성원으로 책임을 다하고 봉사하며 지속적인 자기개발을 통하여 전인적 의료인과 리더로서의 역할을 할 수 있도록 교육과정을 편성하고 있습니다.\n이러한 교육과정을 모두 마치고 졸업을 하면 의사국가시험에 응시할 자격이 생기며, 시험에 합격하여 의사면허를 교부받아야 비로소 의사로서 살아갈 자격을 부여받습니다."
  },
  {
    "input": "의과대학은 뭐배워?",
    "output": "교육 목적을 달성하기 위하여 다음과 같은 의사를 양성함을 목표로 한다.\n1. 환자중심의 진료 및 소통능력을 갖춘 의사\n2. 올바른 윤리관을 갖추고 전문가적 능력을 지속 개발하는 의사\n3. 사회, 국가, 인류에 공헌하는 의사\n4. 국제사회 활동 능력을 갖춘 의사"
  },
  {
    "input": "의과대학 진로좀 말해줘",
    "output": "의사면허를 취득하면, 다양한 진로가 가능한데. 임상의사, 기초의학자, 그리고 의사로서 역할을 필요로 하는 다양한 분야에서 활동하며 살아가게 됩니다. 임상의사가 되는 길은 병원에서 인턴, 레지던트 과정을 이수하고 전문의 시험을 거쳐 해당 분야의 전문의가 되는데, 한림대학교의 경우 교육 가능한 수련병원이 한림대학교 의료원 산하 병원 등(춘천, 강남, 한강, 한림, 동탄, 강동성심병원) 많기 때문에 원하는 임상과를 보다 자유로이 선택하여 수련을 받을 수 있다는 장점이 있습니다. 전문의가 된 뒤에도 개업의사로서 환자를 돌보는 일을 하면서 지역사회에 이바지 할 수도 있고, 의과대학 병원에 남아 임상의사이면서 동시에 연구활동과 학생교육을 담당하는 대학교수가 될 수도 있습니다.\n \n기초의학에 관심이 있는 경우 의과대학 기초의학교실에서 학위과정을 거쳐 기초의학의 발전에 동참할 수 있습니다. 기초의학 전공자들은 현재 한림대학교 의과대학 및 국내외 유수한 의과대학에서 교수로 활동하고 있습니다. 그 외 보건복지부를 비롯한 정부 산하기관, 다양한 의학연구소, 세계보건기구, 언론기관 등에서 전문인으로서 활약하는 졸업생들도 있습니다. 한림대학교 의과대학은 의과대학생들 모두 급변하는 의료, 보건환경에 대처할 수 있는 전문지식과 소양을 갖춘 의료인으로 성장하도록 다양한 교육환경을 제공하고 있습니다."
  },
  {
    "input": "간호학과좀 소개해봐",
    "output": "간호학과는 인간의 건강 상태를 최적의 수준으로 유지, 증진하고 질병 예방과 회복 및 재활을 돕기 위한 임상 간호사, 간호 연구자, 간호 교육자 및 간호 행정가 육성을 교육 목적으로 합니다. 본교를 졸업한 간호학생은 변화하는 의료 환경에 민감하게 대처하고, 국제적 경쟁력을 제고 하며 인간의 건강 향상에 적극 기여할 수 있는 풍부한 인간성과 창조적 지성을 지니어 국내‧외에서 간호지도자의 역할을 감당할 것으로 기대합니다."
  },
  {
    "input": "간호학과 전망이랑 진로 말해봐",
    "output": "임상 간호사 및 조산사, 마취 간호사, 초ㆍ중ㆍ고등학교 보건교사 등으로서의 취업이 가능하며 일반은행, 항공사 등의 간호사, 국민보건을 담당하는 보건진료원 등으로도 진출할 수 있습니다. 또한 정책ㆍ행정에 참여하는 국가행정공무원으로도 활동할 수 있으며, 모든 건강 관련 대학원에 진학이 가능합니다. 졸업 후의 전문 간호과정 교육을 통해 가정간호, 노인 간호, 호스피스, 산업 간호, 감염관리, 마취간호, 보건간호, 보험심사간호, 응급간호, 중환자간호, 정신간호 등 분야의 전문 간호사로서도 역할을 할 수 있습니다."
  },
  {
    "input": "글로벌융합대학에는 어떤 전공들이 있어?",
    "output": "글로벌융합대학 소속 전공으로는 글로벌학부, 융합과학수사학과, 융합인재학부, 기후변화융합전공이 있습니다."
  },
  {
    "input": "글로벌학부가 뭐야?",
    "output": "한림대학교 글로벌학부는 학생들을 교육함에 있어서 각자의 전공분야에서 국제적으로 유능한 인재가 될 수 있도록 양성시키는 데 그 목표를 두고 있습니다. 글로벌학부 학생들은 체계적인 외국어 교육과 함께 전공 수업을 이수하면서 글로벌 인재로서의 역량을 키워가게 됩니다. 이후 4학년이 되었을 때 이러한 경험을 바탕으로 하여 필요한 전문지식과 실력을 갖추고 졸업하여 자신의 목표에 한 발짝 더 다가가게 됩니다. 무엇보다 중요한 점은 글로벌학부 학생들은 4년간의 교육과정을 통해 글로벌 인재로서 감성과 지성을 키울 수 있다는 점입니다. 그리고 사회 어디에서든 기여할 수 있는 중요한 인재로 거듭날 것입니다. 글로벌학부의 모든 교수진과 스텝들은, 학생들이 도전정신을 키우고 교육과정 속에서 겪을 수 있는 어려움을 해결하는 데 아낌없이 지원할 것입니다. 이러한 차별화된 글로벌학부만의 노력이야말로 학생에 대한 특별한 관심과 지도로 학생 개개인의 능력을 극대화시키는 밑거름이 될 것입니다."
  },
  {
    "input": "글로벌학부 SAP가 뭐야?",
    "output": "해외 현지 학업프로그램(SAP)은 글로벌학부 학생들의 졸업필수조건으로, 이에 참여하는 학생들은 해외에서 얻게되는 국제적인 경험과 지식을 통해 졸업 이후 글로벌 비즈니스 환경에 적합한 인재로 성장할 것입니다."
  },
  {
    "input": "글로벌학부 소속 전공에 대해 설명해",
    "output": "글로벌학부 소속 전공으로는 글로벌비지니스 전공, 정보법과학(LIFS)전공, 문화산업전공, 한국학 전공이 있습니다."
  },
  {
    "input": "글로벌비즈니스 전공 소개해주세요",
    "output": "글로벌비지니스 전공은 국제통상의 이론과 실무능력을 겸비하고 외국어 구사능력이 뛰어난 전문 인력양성 교육을 강화함으로써, 국제화 시대에 부응하는 전문지식과 국제적 감각을 겸비한 국제통상 전문인을 양성하는 것을 목표로 합니다."
  },
  {
    "input": "글로벌비즈니스 전공 교육목표 알려주세요",
    "output": "글로벌비즈니스 전공은 국제통상의 이론과 실무능력을 겸비하고 외국어 구사능력이 뛰어난 전문 인력양성 교육을 강화함으로써, 국제화 시대에 부응하는 전문지식과 국제적 감각을 겸비한 국제통상 전문인을 양성하는 것을 목표로 합니다. 본 전공의 교육과정은 다음과 같이 구성되어 있습니다.\n\n- 국제화에 대한 종합적 이해와 과학적 판단력 배양을 위한 교양교육\n- 국제비즈니스 환경에 적응하기 위한 영어 및 제2외국어(일어, 중국어, 스페인어) 학습\n- 경제, 경영, 재무금융 등 핵심 분야 학습을 통한 국제 통상·경영의 포괄적인 전공 지식 습득\n- 국제통상 관련 자격증 취득지원 및 장려"
  },
  {
    "input": "글로벌비즈니스 전공 전망과 진로 알려줘",
    "output": "글로벌비즈니스전공 학생은 고도의 전문지식을 추구함으로써 졸업 후 이들이 나아갈 취업과 진로는 매우 다양합니다. 무역 관련 기업, 외국계 기업, 국제통상 관련 정책기관, 국제금융 관련기관, 공공기관 및 일반 기업체 등으로 진출할 수 있으며, 연구 및 교육부문(국책연구소, 기업연구소, 대학, 국제기구 등), 언론 및 사회부문의 경제전문가 등으로도 활약할 수 있습니다.\n또한 국제통상 관련 사업이나 벤처창업 등도 가능합니다. 이와 같이 글로벌비즈니스 전공자들은 글로벌비즈니스가 포괄하는 범위만큼이나 다양하고 광범위한 분야로 진출할 수 있습니다. 글로벌비즈니스와 관련된 자격증으로는 국제무역사, 외환관리사, 관세사, 물류관리사 등이 있으며, 이러한 자격증의 취득은 졸업 후 진로 및 취업에 많은 도움이 될 것입니다."
  },
  {
    "input": "정보법과학 (LIFS) 전공 소개해주세요",
    "output": "정보법과학(LIFS: Legal Informatics & Forensic Science) 전공은 인문사회 중심의 교과과정에서 과감하게 탈피, 외국어 능력과 실무능력을 강화하고 배양할 수 있는 교육과정을 통하여 ICT 사회가 필요로 하는 정보보안 전문가 양성을 목표로 하고 있습니다. 본 전공은 실무능력을 육성하기 위하여 강원지방경찰청, 경찰수사연수원, 금융경제, 정보보안 기관 등과의 업무협약을 통해 학부생의 실습 참여 기회를 제공하고 있으며, 국제적인 감각을 갖춘 인재 육성을 위해 정보보안 및 Digital Forensic Science 전공분야의 해외 유수대학과 교류협약을 체결하고 있습니다. 정보법과학 전공은 정보보안, 범죄수사, 과학수사 및 범죄심리 등에 관심 있는 학생의 교육욕구를 충족시킬 수 있는 다양한 교양 교과목 개설 및 심화과정의 개설을 통한 교과목의 다양성과 학생선택의 폭을 넓힐 수 있도록 구성되어 있습니다."
  },
  {
    "input": "정보법과학 (LIFS) 교육 목표가 뭐임",
    "output": "정보법과학 전공(LIFS : Legal Informatics & Forensic Science) 은 인문사회 중심의 교과과정에서 과감하게 탈피, 외국어 능력과 실무능력을 강화하고 배양할 수 있는 교육과정을 통하여 ICT 사회가 필요로 하는 정보보안 전문가 양성을 목표로 하고 있습니다.\n\n- 기초지식 과목(법과 직업윤리, 형사법, 영미법, 현대사회와 정보기술 등)\n- 과학수사에 대한 지식습득(법과학, 범죄수사, 범죄심리학, 디지털포렌식 등)\n- 정보보안분야 과목(정보보호법개론, 정보통신법규와 정책 등)\n- 기타 전공 및 실무가 연계된 가변형 교과과정\n- 향후 학부과정의 커리큘럼을 석사 교과과정의 연계함으로써 학생 교육역량 강화"
  },
  {
    "input": "정보법과학 (LIFS) 교육 전공 전망과 진로가 뭐야?",
    "output": "최근 개인정보유출에 따른 사이버보안문제에 대한 사회적 인식이 급격히 높아져 있어, 법과학 분야의 전문가 특히 Digital Forensic에 대한 사회적 수요가 급격히 증가하고 있습니다. 정보법과학의 경우 영어를 기반으로 정보보안 관련 전문적 교육을 통해 정부기관, 보안기업, 기업 감사, 법무법인 등 다양한 분야에서 전문가로 활동할 기회를 갖게 될 것입니다."
  },
  {
    "input": "문화산업전공(Cultural Industry) 소개좀 해주세요",
    "output": "오늘날 전 세계적으로 문화산업에 대한 관심과 중요성이 증대되고 있습니다. 한림대학교 문화산업전공은 ‘문화’라는 영역에 포함되는 경영학적인 부분들을 특화한 전공으로 엔터테인먼트, 공연, 컨벤션, 전시, 축제, 이벤트, 관광 등의 다양한 문화 콘텐츠 분야의 기획 및 운영에 대한 내용을 학습합니다.\n문화에 대한 경영학적 접근뿐만 아니라 학기 중에 다양한 레저, 서비스 분야에 대한 차별적인 교육을 제공하고 있습니다. 예를 들어 승마, 골프, 스키 등과 바리스타, 와인처럼 독특한 분야에 대한 실제 실습을 제공하고 있습니다. 이 전공은 외국인 대상이나 복수전공으로 신청 가능합니다."
  },
  {
    "input": "문화산업전공 교육목표가 어떻게 돼?",
    "output": "문화상품 기획전문가로 양성하는 것을 목표로 하고 있습니다."
  },
  {
    "input": "문화산업전공 전망과 진로좀 설명해봐",
    "output": "문화 관련 서비스 상품 기획력에 대한 학습을 통해 창의성이 요구되는 분야로의 취업 전망이 기대됩니다. \n\n전망 : 문화산업과 관련된 서비스 산업 분야로의 취업 가능\n- 공연 및 행사 기획사, 여행사, 호텔, 문화 및 관광산업 관련 기관\n\n진로 :\n- 문화관련 산업 혹은 국가 기관 취업\n- 관련 기업체 취업(각종 기획사 등)\n- 지속 학습: 석·박사과정을 통해 전문가로 성장\n- 중국 CCPIT(China Council For the Promotion of International Trade)와의 연계를 통한 자격증 획득 및 방학 중 인턴십 제공"
  },
  {
    "input": "한국학 전공 소개해",
    "output": "한림대학교 한국학 프로그램은 글로벌 시대의 구성원들로 하여금 세계시민의식과 함께 한국에 대한 다양한 분야의 콘텐츠를 체계적인 교육과정을 통해 제공하는 프로그램입니다. 한국의 언어, 역사, 문화유산을 비롯하여 경제, 사회, 정치 등을 총 망라한 주제를 100% 영어강의를 통해 재학생의 복수전공은 물론 외국인 유학생의 복수학위 과정으로 제공합니다. 이는 K-Pop, K-Drama 등 한류 열풍에 힘입어 한국을 더욱 알고자 하는 세계인의 적극적인 관심 증대는 물론이고, 이러한 한국에 대한 지식을 전파하는 데에 우리 학생들이 그 역할을 할 수 있는 학습의 기회와 능력을 겸비하여 말 그대로 ‘글로벌 리더’로서 성장할 수 있도록 하기 위함입니다. 이 전공은 외국인 대상이나 복수전공으로 신청 가능합니다."
  },
  {
    "input": "한국학 전공 교육목표가 뭔가요?",
    "output": "한국학 프로그램은 본교 재학생과 외국인 학생(교환학생 포함)들로 하여금 한국어를 비롯하여 한국의 역사, 문화유산, 정치, 경제, 사회 및 대중문화와 예술 등의 폭넓은 내용을 학습하도록 하여 한국에 대한 체계적인 지식 습득의 기회로 삼고, 나아가 이들이 글로벌 사회에서 ‘한국 전문가’로서의 역량을 발휘할 수 있도록 하는 데에 그 교육의 목표를 두고 있습니다."
  },
  {
    "input": "융합과학수사학과 설명해주세요",
    "output": "한림대학교 정보법과학전공이 융합과학수사학과로 확대 개편되었습니다.\n2015년 국내 유일의 영어강의 기반 디지털포렌식 전문가 양성과정으로서 정보법과학전공을 창설하고 디지털포렌식 분야 졸업생을 배출해온 글로벌융합대학은 전 세계적인 범죄심리, CSI, 디지털포렌식 등 경찰 과학수사 분야에 대한 수요 증가에 따라 2021년부터 융합전공으로서 경찰과학수사융합전공을 운영하였으며, 2023년 두 첨단 전공을 통하여 축적된 성공사례를 기반으로 융합과학수사학과를 신설하였습니다.\n한림대학교 융합과학수사학과는 2024학년도부터 40명을 선발하여 AI로 급변하는 사회에서 발생하는 융복합적 문제에 대처하고, AI 데이터 활용을 통하여 첨단과학수사 발전을 선도할 수 있는 인재를 양성합니다.\n챗GPT로 대표되는 4차산업혁명, 빅데이터의 시대는 수동적이고 정형화된 지식을 가진 인재보다 AI를 보다 잘 활용하는 인재를 필요로 합니다.\n융합과학수사학과는 교수진이 가진 경찰 및 수사 관련 교과목과 더불어 AI 응용기술, 정보보안 및 보안 정책 등 첨단 과학수사에 요구되는 교육과정을 제공함으로써 학생들을 IT 관련 범죄나 사이버수사 경찰뿐만 아니라 기업감사, 디지털 플랫폼 기업, 정보보안, 개인정보보호 등의 민간분야로도 진출할 수 있는 기술력을 보유한 융합인재로 양성합니다."
  },
  {
    "input": "융합인재학부가 뭐에요?",
    "output": "융합인재학부는 의과학융합 분야의 인재를 양성하는 데 교육의 목표를 두고 최우등 학생 교육(Honors Program)을 실시하는 특성화 학부입니다. 신산업 분야가 요구하는 창의적 문제해결형 인재 양성을 위하여 유망 학문 분야를 유기적으로 융합하여 제공하고, 국내 최고 수준의 장학제도로 학생들을 지원합니다.\n\n기술의 혁신은 우리 사회의 전 분야에서 나타나는 현상인데, 이 가운데 보건의료는 특히 신기술의 개발과 도입이 가장 빠르게 일어나는 분야 가운데 하나입니다. 한림대학교는 설립 이래로 의학, 간호학, 보건학 및 이와 관련된 인문, 사회, 자연과학 등의 분야를 집중적으로 육성하여, 이제는 명실상부한 국내 최고 수준을 자랑하고 있습니다. 보건의료 분야의 성장과 확장으로 더 많은 전문인력, 특히 기술과 다방면의 지식이 결합된 창의적인 역량을 갖춘 인재에 대한 수요는 나날이 커지고 있습니다. 이에 융합인재학부 의과학융합전공은 미래 사회의 보건의료를 선도할 유능한 인재를 양성하는 데에 그 목표를 두고 부단의 노력을 다하고 있습니다."
  },
  {
    "input": "융합인재학부 교육과정 알려줘",
    "output": "데이터분석과 컴퓨터프로그래밍, 사회과학, 경영학, 기술과학분야를 단계별로 기초부터 심화과정까지 제공함은 물론, 졸업시까지 8개 이상의 집중영어교과와 제2외국어, 독립연구를 통한 연구결과발표, 그리고 최상위권 어학취득을 필수로 하고 있습니다. 더불어 ‘AI융합전공’과 연계하여 지자체・공공기관 취업모델을 만들어가는 「강원도 지역선도대학 육성사업」 ‘디지털헬스케어전공’ 교육과정을 주도하고 있습니다."
  },
  {
    "input": "융합인재학부 Honors(아너스) 가 뭐야?",
    "output": "융합인재학부에는 Honors(아너스) 장학제도가 있습니다. 융합인재학부에 입학하는 학생 대부분은 Honors 장학제도로 국내 최고 수준의 장학금 혜택을 지원받습니다. 입학성적에 따라 입학금을 포함한 8학기 등록금 전액, 독립연구를 위한 연구활동비와 기숙사비를 매 학기 지원합니다."
  },
  {
    "input": "융합인재학부는 졸업하고 뭐해?",
    "output": "융합인재학부 의과학융합전공은 취·창업과 연구활동, 대학원 진학에 필수 요소인 데이터분석, 프로그래밍, 기술창업, 산업정책 등 의과학 연계 교과와, 실무역량을 극대화하는 프로젝트 비교과 활동을 통해 데이터분석사, 사회조사분석사, 의료정보관리사, 정보처리기사 등 전문자격증 취득을 지원합니다.\n\n소규모 프로젝트 중심의 전공 교육과정과 수준 높은 Honors Program을 성공적으로 이수한 학생들은 향후 글로벌 기업 및 의료 기관, 국제기구 및 국책연구소 취업, 국내외 Top 스쿨 대학원 진학 등 융합분야 전문가로의 활약을 기대할 수 있습니다. 또한 본인의 융복합 연구 결과물을 직접 사업화하는 방안으로 개인 창업을 적극 지원합니다."
  },
  {
    "input": "기후변화융합전공 교육목표가 뭔데?",
    "output": "한림대 <기후변화융합전공>은 학부과정으로는 국내 최초로 기후변화 정책(Climate Change Policy)과 온실가스 관리(GHG Management) 분야에서 미래의 기후변화 산업과 연구를 선도할 전문인력 양성을 목표로 합니다.\n현재 그리고 미래의 공공부문 및 민간부문의 기후변화 전문인력 수요를 고려하여, ‘기후변화 대응 정책전문가’와 ‘온실가스 관리 실무전문가’ 양성을 위한 2개의 특성화된 전공 Track으로 운영하며, 융합전공 각 Track 전공과정을 통해 관련 산업과 연구 분야의 지역/국가/국제(local/national/international) 기관에 진출하기 위한 충분한 이론적 지식과 현장 적용성이 높은 실무적 역량을 함양합니다."
  },
  {
    "input": "미디어스쿨 소속 전공은 어떤게 있는데?",
    "output": "미디어스쿨 소속 전공으로는 언론방송융합미디어 전공, 디지털미디어콘텐츠 전공이 있습니다."
  },
  {
    "input": "언론방송융합미디어 전공 교육목표 알려줘봐",
    "output": "언론 / 방송인에 필요한 글쓰기, 말하기, 영상으로 이야기 하는 능력을 기름으로써 매체 종사자는 물론 다양한 직종에도 적응할 수 있는 기능을 익힙니다."
  },
  {
    "input": "언론방송융합미디어 전공 교과과정 설명좀",
    "output": "신문저널리즘\n- 교내외 매체 게재를 전제로 한 실무 교육\n- 현장실습 인턴 의무화로 현장 감각유지\n- 글, 사진, 편집능력 입증할 포트폴리오 축적\n방송저널리즘\n- 교내 교외매체의 방송 프로그램 납품 전제한 실무교육\n- 인턴 의무화\n- 보도, 촬영, 편집 포트폴리오 준비\n방송콘텐츠 제작\n- 영상 공모전 응모 전제한 수업과 동아리 활동 전개\n- 인턴 의무화\n- 영상 스토리텔링 능력 포트폴리오화\n복수전공추천트랙 : 홍보\n- 정부, 기업, 홍보 대행사 진출 위해 홍보학과 복수전공\n- 인턴 의무화\n- 본 전공의 기사 작성 능력 바탕에 홍보학지식 접목\n대학원 진학\n- 연구소, 조사회사, 학자양성 위해 사회조가 실무 등 교수진 연구 참여 활동\n- 해외 대학 교환프로그램 참여"
  },
  {
    "input": "언론방송융합미디어 전공 진로 알려줘",
    "output": "신문방송 등 각종 매체 기자\n기업의 홍보부서나 홍보대행사 직원\n다큐멘터리제작 관련 프로듀서\n뉴스캐스터 등 아나운서\n말하기, 글쓰기를 요구하는 다양한 직장과 직종"
  },
  {
    "input": "디지털미디어콘텐츠 전공 교육 목표 말해봐",
    "output": "디지털미디어콘텐츠 전공 교육목표는 디자인, 스토리텔링, 이용자와의 인터액션에 관련된 지식과 실무 능력을 배양함으로써 디지털 미디어 콘텐츠 산업 분야에 다양하게 진출할 인재를 양성하는 것입니다."
  },
  {
    "input": "디지털미티어콘텐츠 졸업하면 뭐해?",
    "output": "게임기획개발\n디지털영상편집, 디지털특수영상 제작\n콘텐츠 홍보와 마케팅\n디지털 북 기획 개발\n포털 등 웹 콘텐츠 기획, 제작\n기획, 홍보, 제작 관련 다양한 직장과 직종"
  },
  {
    "input": "반도체·디스플레이스쿨 소속 전공 말해봐",
    "output": "반도체·디스플레이스쿨 소속 전공에는 반도체 전공, 디스플레이 전공이 있습니다."
  },
  {
    "input": "반도체·디스플레이스쿨 정보좀",
    "output": "반도체 ‧ 디스플레이는 대한민국이 세계를 선도하는 국가 핵심 산업으로, 높은 성장률에 따라 대규모의 신규 산업기술인력을 필요로 하는 분야입니다. 반도체 ‧ 디스플레이스쿨은 21세기의 최첨단 나노기술을 끌어갈 핵심분야인 반도체전공 및 디스플레이전공으로 구성되어 있으며, 나노기술을 기반으로 반도체 ‧ 디스플레이분야를 융합하여 21세기를 선도할 융합형 전문 인력을 양성하고자 합니다."
  },
  {
    "input": "반도체·디스플레이스쿨 진로 알려줘",
    "output": "1. 국가 주력 산업 분야 진출 : 반도체 / 디스플레이 분야는 정부에서 지정한 12대 주력산업으로, 전체 산업기술인력의 67%를 차지하고 있습니다. 해당 산업의 성장에 따라 향후 기술인력에 대한 수요는 급격히 높아질 것으로 예상됩니다. 따라서 졸업 후 국가 핵심산업인 반도체 산업, 디스플레이산업, 공정/ 장비 산업, 소재/부품 산업 등 매우 다양한 분야에 진출할 수 있습니다.\n2. 국공립 연구소 진출 및 대학원 진학 : 대학원 진학에 있어 필요한 역량을 학생들이 키울 수 있도록 지도교수가 맞춤형 개인지도를 진행합니다. 그 외에 자격증 취득 지원 등을 통해 국공립 연구소 및 산업 관련 정부기관에서도 일할 수 있도록 지도하고 있습니다.\n3. 창업 : 창업과 관련된 전문가와의 만남 및 창업 정보를 풍부히 제공함으로써 창업을 통한 기업가로의 꿈을 실현할 수 있습니다."
  },
  {
    "input": "정보과학대학 소속 전공 설명좀",
    "output": "정보과학대학 소속 전공으로는 소프트웨어학부, 인공지능 융합학부, 데이터사이언스 학부가 있습니다."
  },
  {
    "input": "소프트웨어학부 교과과정 설명해줘",
    "output": "산업체 수요에 대응하는 문제해결형 SW 인재 양성을 교육 목표로 4차 산업혁명 기술변화 흐름에 맞는 실전적 SW 교육과정으로 전면 개편하였습니다. 교육과정은 SW전공역량에 필수적인 기초코딩과목과 SW전공과목, 프로젝트기반 산학협동과목을 운영하고, 4차 산업혁명을 선도할 핵심전공인 빅데이터전공, 콘텐츠IT전공, 스마트IoT전공을 특화시켜 운영합니다.\n교육과정은 SW전공역량에 필수적인 기초코딩과목과 SW전공과목, 프로젝트기반 산학협동과목을 운영하고, 4차 산업혁명을 선도할 핵심전공인 빅데이터전공, 콘텐츠IT전공, 스마트IoT전공을 특화시켜 운영합니다."
  },
  {
    "input": "소프트웨어학부 소속 전공 나열해봐",
    "output": "소프트웨어학부 소속 전공으로는 빅데이터 전공, 콘텐츠IT 전공, 스마트IoT 전공이 있습니다."
  },
  {
    "input": "소프트웨어학부 졸업 후 진로 알려줘",
    "output": "소프트웨어융합대학의 세부 핵심전공을 졸업하고, IT, 의료, 자동차, 금융, 방송 문화 등의 4차 산업혁명관련 주요분야에서 인공지능 소프트웨어 개발 및 관리자, IoT 시스템 개발 및 관리자, 데이터 분석가, 콘텐츠 개발자등으로서 활발하게 진출하고 있습니다. 최근, 모든 산업분야가 IT/소프트웨어 기술과 융합되면서 거의 모든 산업분야로의 진출로 확대되고 있습니다. 졸업생들은 삼성전자, LG전자 등의 주요 대기업, 메디슨, 다음 카카오 등 중견기업, Intel, Google과 같은 외국계 기업에 진출하여 능력을 발휘하고 있습니다. 또한, 정부기관 및 정부출연 기관에서도 다양하게 활약하고 있습니다.\n학·석사 연계과정이나 졸업 후 대학원에 진학하여 학위 취득 후 연구 및 교육 분야로 진출할 수도 있으며, 현재 국내 및 해외 대학교에서 교수로서 활동하기도 합니다."
  },
  {
    "input": "빅데이터 전공이 뭐야?",
    "output": "빅데이터 전공에서는 빅데이터/클라우드, 인공지능 분야에 대한 특화된 교육과정을 제공함으로써, 급변하는 컴퓨터 기술의 발전에 선도적으로 참여하고 빅데이터 활용 능력을 통해 미래를 만들어 갈 수 있는 인력을 배양하는데 그 목표를 두고 있습니다. 이를 위하여 기술 변화와 시장의 수요에 맞게 교과 과정을 운영하고 있으며, 정보화 시대인 21세기를 선도하는 이론과 기술을 익힌 정보인력을 양성하고 있습니다. 졸업 후 산업체 및 연구소 등에서 수준 높은 소프트웨어 개발자로 사회에 기여할 수 있도록 지원하고 있습니다."
  },
  {
    "input": "빅데이터 전공 교과과정이 어떻게 돼?",
    "output": "빅데이터 전공 교육과정은 대용량의 데이터를 효과적으로 저장 관리하는 방법 및 고성능으로 처리하기 위해 요구되는 교과목을 제공합니다. 또한 인공지능/기계학습을 적용한 데이터 학습과 추론을 통해 데이터를 분석하는 소프트웨어 알고리즘 관련 교과목들을 학습합니다.\n 주요 교과목으로는 데이터시각화, 데이터마이닝, 딥러닝이론및응용, 시스템보안, 텍스트정보처리, 머신러닝, 클라우드컴퓨팅, 빅데이터개론, 네트워크보안, 소프트웨어캡스톤디자인이 있습니다."
  },
  {
    "input": "빅데이터 전공 졸업 하고는 뭐가 될 수 있어?",
    "output": "프로그래밍과 정보처리 기술에 특화된 교육을 통해 IT 기업의 SW개발 전 분야에 전문 프로그래머로 취업이 가능합니다.\n최신 가장 큰 이슈인 빅데이터와 인공지능은 새로운 창업 및 취업시장을 지속적으로 만들어 내고 있으며, 이에 따라 취업 및 창업의 기회가 커지고 있습니다. 많은 대학의 대학원에서 빅데이터와 인공지능에 대한 연구가 활발하게 진행되고 있으며, 대학원으로의 진학해서 더 깊이 있는 학습을 하고 전문 연구원으로의 활동도 가능합니다.\n\nIT분야: IT 연구원, IT 컨설팅, 빅데이터 구축관리 전문가, 비즈니스분석 컨설턴트\n금융기관: 은행, 카드회사, 보험회사, 증권회사\n정부기관: 공무원(기획재정부, 통계청), 정부출연 연구기관\n의료기관: 병원의무기록사, 의료정보 분석 전문가\n마케팅 및 리서치분야: 시장트렌드 및 소비자분석전문가, 고객정보분석 및 여론리서치\n온라인게임업체: 게임데이터분석 전문가\n기타: 국내외 대학원진학, 빅데이터 관련 창업"
  },
  {
    "input": "콘텐츠IT 전공 소개해봐",
    "output": "첨단 IT분야 중 하나인 콘텐츠IT 분야에 특화된 전문 인력 양성을 목표로, 실무 중심의 교육을 통하여 콘텐츠 관련 산업계에서 요구하는 전문적인 소프트웨어 개발 능력을 배양합니다. 최근 콘텐츠IT 분야의 전문 인력 수요를 고려하여, 저희 전공에서는 가상현실/증강현실(VR/AR) 트랙과 게임 개발자 트랙으로 구분하여 진로를 계획적으로 설정하고 학업을 체계적으로 이수할 수 있도록 도움을 주고 있습니다."
  },
  {
    "input": "콘텐츠IT 전공 교과과정 알려줘",
    "output": "콘텐츠IT 전공에서는 3차원 그래픽스 및 게임엔진 이론을 바탕으로 가상현실/증강현실/게임 콘텐츠의 설계 및 구현에 필요한 원리 및 다양한 기술을 배웁니다. 학생들은 심화전공 교과목들에서 3차원 가상공간을 이해하기 위한 그래픽스 기술, 3차원 가상공간에 현실감을 부여하기 위한 물리엔진 기술, 가상현실과 증강현실의 동작원리를 적용하는 혼합현실 기술, 그래픽스와 물리엔진 기술을 혼합한 게임프로그래밍 기술 및 PC와 모바일 플랫폼의 성능 향상을 위한 GPU 프로그래밍 기술을 학습합니다. 주요 교과목에는 VR/AR/게임제작기초, 컴퓨터그래픽스, 소프트웨어캡스톤디자인, 멀티미디어개론, 가상현실과증강현실, 영상처리와딥러닝, 게임프로그래밍, HCI가 있습니다."
  },
  {
    "input": "콘텐츠IT 전공 졸업 후 진로 알려줘",
    "output": "게임기업: 게임개발자, 게임기획자, 게임OA 전문가, UI/UX 개발자 (넥슨, 엔씨소프트 등)\n영화분야: 영화 특수효과/애니메이션 제작사 (FXGear 등)\n정부기관: 공무원, 정부출연 연구기관 (전자통신연구원, KIST 등)\n교육기관: VR/AR 기반 교육 SW 콘텐츠 개발자\n의료기관: VR/AR 기반 의료재활치료 SW 개발자\n스마트디바이스 앱 개발자\n기타: 국내외 대학원진학, 멀티미디어 콘텐츠 관련 창업\n기타사항"
  },
  {
    "input": "스마트IoT 전공 소개 말해봐",
    "output": "스마트IoT 전공에서는 사물인터넷 (IoT: Internet of Things) 구축과 IoT 기반 서비스 설계 및 구현에 필요한 원리와 다양한 기술을 배웁니다. IoT에서 다양한 정보를 수집하기 위한 센서 기술과 신호 처리 기술, 정보를 전달하기 위한 IoT 통신망, 수집된 데이터를 분석・평가하고 결정을 내리기 위한 데이터 분석 기법 및 인공지능 관련 기술, 그리고 정보보호를 위한 기술 등을 배웁니다. 이러한 교육을 통해 IoT 환경에 필요한 기본적인 공학적 소양을 갖추고 이를 산업에 적용할 수 있는 IoT 전문 인력을 양성합니다."
  },
  {
    "input": "스마트IoT 전공 교과과정 설명좀 해줘",
    "output": "스마트IoT 전공 교육과정은 다양한 센서 정보를 신뢰성있게 처리하고, 무선네트워크를 통해 효과적으로 전송하는데 있어서 필요한 다양한 교과목들을 제공합니다. 또한 센싱 정보를 사용자가 가장 쉽게 이용할 수 있도록 해주는 인공지능/기계학습 기반 IoT플랫폼 기술과 IoT 시스템 해킹으로 인한 시스템 오작동, 정보유출사고 등에 대응할 수 있도록 해주는 정보보호 관련 기술을 학습합니다. 스마트IoT 전공 주요 교과목으로는 디지털신호처리, IoT네트워크, 네트워크보안, 블록체인이 있습니다."
  },
  {
    "input": "스마트IoT 전공 졸업 후 진로 궁금해",
    "output": "ICT분야: 소프트웨어 및 이동통신 관련 글로벌 기업 (삼성전자, LG전자, SK텔레콤 등)\n금융기관: 핀테크 관련 시스템 설계 (은행, 증권회사 전산 관련 부서)\n정부기관: 공무원 (과학기술부, 산업통상자원부), 정부출연 연구기관 (ETRI 등)\n의료기관: ICT 의료시스템 연구개발 및 관리자\n정보보호 관련기업: 은행, 증권회사, SK텔레콤, KT, 안랩 등\n스마트 홈/팩토리 관련 기업\n기타: 국내외 대학원진학, IoT 관련 창업"
  },
  {
    "input": "인공지능융합학부 정보가 궁금해",
    "output": "한림대학교는 첨단기술분야의 인재양성을 위해, 2021년 “인공지능융합학부(입학정원 60명)”를 신설하였고, 2022년 입학정원을 90명으로 확대하여 대학 전체의 인공지능 융합(AI+X)을 선도하는 대표적인 학문단위로 육성하고자 합니다.\n한림대학교의료원(5개 병원)과 연계한 “AI의료융합전공”을 비롯하여, “AI로봇융합전공”, “AI기술경영융합전공” 개설하여 의료인공지능분야를 특성화하고, 기계, 전산, 전자공학 분야의 다학제적 융합과 첨단기술을 활용한 비즈니스 모델 개발 분야에도 주력합니다.\n한림대학교 인공지능융합학부는 공학뿐만 아니라, 인문사회를 포함하는 모든 영역에서 인공지능 융합을 선도하여, 금융 분야의 핀테크(Fin-Tech), 디지털인문예술, 기후 및 지리정보(Climate information and GIS) 등 그 영역이 확장될 것으로 예측되는 첨단 분야 융합을 촉진해 나갈 것입니다."
  },
  {
    "input": "인공지능융합학부 교육과정은 뭐가 있어?",
    "output": "인공지능융합학부는 스마트 헬스&라이프케어 산업 고도화, 미래형 보건·복지 시스템 구축을 주요 의제로 하고 의료+인공지능에 대한 융합교육 전면화를 통하여 의료인공지능 융합 인재 양성하는 ‘AI의료융합전공’, 로봇 기술 분야 간의 융복합에 의한 신산업의 등장 및 지능형 혁신 기술의 산업적 수요 증대에 대응 가능한 실무형, 창의 융합 인재 양성을 목표로 하는 ‘AI로봇융합전공’ ,인공지능과 빅데이터 활용에 관한 축적된 전문지식과 경제·경영·정책 등 사회과학 분야에 대한 이해를 통해 첨단기술을 활용한 비즈니스 모델의 개발, 과학기술정책 컨설팅 및 평가 관련 전문인력을 양성하는 ‘AI기술경영융합전공’을 핵심 전공으로 운영합니다."
  },
  {
    "input": "인공지능융합학부 교육과정 졸업하면 어디 취업해?",
    "output": "AI·데이터 분야는 세계경제포럼(WEF)이 전망한 ‘가장 유망할 것’으로 예상되는 7대 직업군 중 하나이며, 연간 일자리 성장률이 가장 큰 직업군입니다.\n한림대학교 인공지능융합학부는 AI분야의 핵심이 되는 머신러닝과 딥러닝을 포함하여, 데이터 관리/분석력을 배양하는 응용통계학 분야를 핵심기초로 하고, 의료 및 헬스케어, 로봇, 기술경영 및 범죄수사 등 다양한 분야에서 인공지능기술의 응용 및 활용능력을 갖출 수 있도록 하는 교육콘텐츠를 제공함으로써 졸업 후 진로를 지원합니다."
  },
  {
    "input": "인공지능융합학부 관련 자격증은 뭐가 있어?",
    "output": "인공지능융합학부 관련 국내 자격증은 COS, 코딩지도사자격증, 빅데이터분석기사, 사회조사분석기사, 의료정보분석사, 경영빅데이터분석사 등이 있습니다. 인공지능융합학부 관련 해외 자격증은 CAP, CPADS, MMDSGC, CCA 등이 있습니다."
  },
  {
    "input": "데이터사이언스학부가 뭐야",
    "output": "4차 산업혁명은 인공지능의 시대라고 사람들은 이야기 합니다. 인공지능의 핵심이 바로 데이터입니다. 여러분들이 많이 사용하는 SNS나 구글 등은 플랫폼을 이용해 데이터를 수집하고, 데이터를 활용하는 사업모델을 가지고 있습니다. 결국 미래사회의 핵심이 바로 ‘데이터’입니다. 데이터사이언스학부는 \"Hi FIVE\" 교육모델과 교수방법을 통하여 4차 산업혁명시대로의 변화 동력인 \"빅데이터\"를 잘 이해하고 다룰 수 있는 미래사회 선도 역량을 갖춘 인재 육성에 힘쓰고 있습니다. 학생의 신사회진출 기회의 수월성을 높이는 기업수요 맞춤 \"데이터 신 산업교육 선도모델\" 을 운영하는 측면에서 차별성을 갖습니다."
  },
  {
    "input": "데이터사이언스학부 교육목표좀",
    "output": "한림대학교 데이터사이언스학부에서는 데이터에 대한 문해력(Data Literacy)과 데이터분석력(Data Analysis Competency)을 갖춘 데이터 사이언티스트(Data Scientist) 양성을 목표로 하고 있습니다."
  },
  {
    "input": "데이터사이언스학부 교육과정이 궁금해",
    "output": "전공프로그램의 1+1 복수전공 선택을 통하여 전공심화 효율성 확보와 융합적 콘텐츠 기획력을 배양하는 '초연결 데이터 전문가' 교육과정\n초연결 사회(Hyper-connected Society)에서 생성되는 다차원 데이터를 분석하여 선도적 정보가치를 창출, 창의적 기획/전략을 통하여 신성장 산업에 필요한 경쟁력 있는 '콘텐츠 기획역량 전문가' 교육과정\n산 ‧ 학 ‧ 병 ‧ 관 실무전문가의 공동연구 프로젝트(학생참여)를 통하여 학생의 실무역량 현장실습 확대와 어드벤쳐 디자인 및 캡스톤 디자인(Adventure Design & Capstone Design)등의 역량훈련과정 도입; PBL&FL(Project Based Learning & Flipped Learning) 공동연구 세미나 교과과정 산 ‧ 학 ‧ 병 ‧ 관 표준인증\n데이터 분석 전문가에 필요한 5C(Critical Thinking, Creativity, Colaboration, Catalyst Maker)역량 훈련 선도 교육모델; \"Hi FIVE 교육모델\" 운영"
  },
  {
    "input": "데이터사이언스학부 전망과 진로가 알고싶다",
    "output": "[데이터테크 + 임상의학통계] 복수전공자의 사회 진출\n임상의학 분야는 사람의 데이터를 다루는 곳이기 떄문에, 데이터의 생산과 처리에 다양한 제약이 있다. 또한 최근에는 유전체 정보가 임상에 활용됨에 따라 빅데이터 분석 전문가 수요가 급증하고 있다. 따라서, 이러한 임상 빅데이터의 분석과 해석 전문가 훈련을 통해 병원, 제약회사, 생체정보를 다루는 전자회사, 임상연구소, 생명과학 관련 기업, 임상시험 전문기업 등에서 분석 전문가로 활동할 수 있다.\n\n[데이터테크 + 디지털금융정보] 복수전공자의 사회 진출\n금융업은 많은 사람들의 거래 데이터를 기반으로 하고 있다. 데이터를 바탕으로 보험상품을 개발하고, 금리를 예측하거나 증시, 환율 등의 동향을 파악하는 업무 전문가로서의 활동을 할 수 있다. 보험계리사 자격증을 준비항 보험분야에서의 경쟁력을 높일 수 있고, 은행, 증권사, 카드사 등의 업계에서 해당 분야의 데이터에 익숙한 분석 전문가로 활동 할 수 있다"
  },
  {
    "input": "미래융합스쿨 소속 전공은 뭘 배워?",
    "output": "미래융합스쿨 소속 전공에는 디지털인문예술전공, 글로벌협력전공, 융합관광경영전공, 의약신소재전공, 융합신소재공학전공, 마케팅커뮤니케이션테크놀로지융합전공, 공공인재융합전공이 있습니다."
  },
  {
    "input": "디지털인문예술전공좀 설명해봐",
    "output": "디지털 기술의 비약적인 발전은 인문학의 근본적인 변화를 요구하는 동시에 인문학의 가능성을 획기적으로 증대시키고 있습니다. 또한 오랜 세월 동안 쌓인 인문학의 지적 자산과 미적 감각, 창의력을 절실하게 필요로 하고 있습니다. 여기에 디지털 기술과 인문학이 화학적으로 결합할 필요성이 있습니다. 디지털인문예술 융합전공은 디지털 문명과 기술에 대한 인문학적 성찰과 문화와 예술에 대한 감각을 바탕으로 시대가 요구하는 새로운 것을 창조하는 능력을 함양시킬 것입니다."
  },
  {
    "input": "디지털인문예술전공 졸업인증 기준이 어떻게 돼?",
    "output": "본 전공이 주전공일 경우 오디세이세미나3 필수 수강\n 복수전공 학점 33학점 이수.\n 영상문예창작협동전공 소속 학생이 이미 수강한 영상문예창작협동전공 교과목은 모두 디지털인문예술전공 교과 이수 학점으로 인정함."
  },
  {
    "input": "디지털인문예술전공 졸업 후 진로 알려줘",
    "output": "빅데이터 분석가(언어 빅데이터 분석 및 시각화)\n데이터 과학자(인문 정보자료 통계 처리 시각화)\n 웹, 앱, 그래픽, 멀티미디어 디자이너\n 디지털 예술, 이미지, 영상 창작자\n 전자 출판 전문가\n 문화 기획, 창작, 운영 전문가\n 디지털 미디어 스토리텔링 전문가\n 박물관, 도서관, 미술관 디지털 큐레이터\n 디지털 리터러시 교육자\n 디지털 마케팅, 홍보, 기획 전문가"
  },
  {
    "input": "글로벌협력전공 소개해봐",
    "output": "글로벌 협력 전문가와 사회혁신 창업가를 육성하기 위한 전공으로서, 학생들을 지속가능 개발목표(SDGs) 달성을 위한 글로벌 협력과 사회혁신 역량을 갖춘 전문인력으로 성장시켜 국내는 물론 글로벌 환경에서 활동하도록 하는 것을 목표로 합니다."
  },
  {
    "input": "글로벌협력전공 졸업 후 진로가 뭐야",
    "output": "소셜 벤처/ 비영리 스타텁/ 사회적 기업 창업\n소셜벤처/ 지역혁신/ NGO의 사회혁신 기획자 및 운영자\n서울 및 지역단위 사회적 경제 지원센터, 사회혁신센터 코디네이터 \n중앙정부, 지방자치단체 사회적 경제 및 사회혁신 분야 코디네이터 \n기업 ESG 및 사회공헌 담당 매니저/ 해외 사업 매니저\n글로벌개발협력 분야 공공기관 취업 (e.g. KOICA, 코웍스 등)\n시민사회의 국제개발협력 NGO 전문가 (월드비전, 굿네이버스, 초록우산어린이재단, 월드투게더 등)  \n개발협력 전문 컨설팅 기관 취업 (KODAC, 한국개발전략연구소 등) \n한국국제협력단(KOICA), 한국수출입은행, 한국국제교류재단, 세종학당, 한국문화원 등 개발원조 및 문화협력 파견 전문가"
  },
  {
    "input": "융합관광경영 전공 교육목표 알려줘",
    "output": "관광학과 경영학, 인문학, 법학, 레저스포츠학, 식음료학, IT기술, 의학 등 학제간 융합지식을 학습하는 융합형 관광인재를 양성합니다.\n신아이디어와 IT산업 환경에 익숙한 미래 관광산업 맞춤형 관광인재를 양성합니다.\n중앙정부와 강원도의 전략산업 중 하나인 관광산업에 필요한 인력을 제공하는 신성장전략산업 맞춤형 인재를 양성합니다."
  },
  {
    "input": "융합관광경영 전공 졸업 후 진로 말해줘",
    "output": "항공상, 여행사, 리조트, 테마파크, 카지노, 크루즈사, 호텔업, 외식업, 컨벤션전시센터, 컨벤션뷰로(CVB), 컨벤션기획회사, 전시기획회사, 이벤트사, 레저스포츠산업, 의료관광산업(의료관광코디네이터, 의료관광마케터), 문화관광분야(문화관광해설사), 공문원 및 공사(중앙/지자체공무원-관광분야, 한국관광공사, 지자체관광개발공사), 국립/도립/시립/군립공원, 프리랜서(해외여행통역가이드, 여행작가, 여행전문기자, PCO-컨벤션기획사, PEO-전시기획사, MP-미팅플래너, PP-파티플래너, WP-웨딩플래너, SNS여행컨텐츠프로그래머 등)"
  },
  {
    "input": "의약신소재 전공 소개해줘",
    "output": "- 미래 의약 및 바이오산업 분야에서 중추적 역할을 할 실무형 전문지식을 갖춘 글로벌 융합인재를 양성합니다.\n- 전세계적 고령화와 의료기술의 발달로 꾸준히 성장하고 있는 의약 산업은 미래의 핵심 전략 산업으로 각광받고 있는 반면, 국내 의약 산업을 선도해 나갈 인력을 배출하는 전문 교육은 글로벌 의약품 시장을 선도해 나가기에 턱없이 부족한 실정이기에 의약신소재 융합전공은 R&D 개발 전문 인력뿐만 아니라, 비임상 및 임상 실험, 의약품 생산, 시설, 및 품질 관리, 의약품 인허가, 마케팅 등의 현장 실무적 교육을 통해 미래 선도 산업을 이끌어갈 의약 산업 전문 인력 양성을 목표로 합니다."
  },
  {
    "input": "의약신소재 전공 졸업 후 진로 알려줘봐",
    "output": "1. 의약학 및 생명과학 관련 대학원 진학, 유학 등\n2. 제약회사, 바이오기업, 화장품회사, 식품 회사 등 연구소\n3. 국공립연구소, 대학 및 병원 연구소\n4. 벤처 창업 등"
  },
  {
    "input": "융합신소재공학과 소개좀",
    "output": "융합신소재공학은 삶의 질적 향상을 추구하는 21세기 산업분야의 핵심소재를 연구/개발하는 독립분야일 뿐 아니라, 바이오/의료 분야와 학문적으로 연계하여 그 과학과 기술을 융합하고 구체적으로 실현하는 학문입니다. 특히 차세대 의료기기 분야의 새로운 인재 양성을 위한 교육 및 연구 특성화 프로그램을 운영하고 있습니다.\n학부과정에서는 소재 전반에 관한 기초 이론을 심도있게 학습하고, 이에 관한 실험을 통하여 다양한 형태의 바이오메디컬 엔지니어링, 소재 및 소자, 생산 공정 등을 기획/설계하고 이를 응용분야에 활용할 수 있는 능력을 배양하여 기초 과학과 공학, 첨단 지식을 기반으로 BT(Biotechnology) 및 4차산업혁명 분야 등 미래 핵심 산업분야에서 중추적 역할을 할 글로벌 인재를 양성하는데 그 목적이 있습니다."
  },
  {
    "input": "융합신소재공학과 졸업 후 진로는?",
    "output": "졸업 후 차세대 유망산업인 바이오 산업, 의료서비스 산업 뿐만 아니라 국가 핵심 소재 산업인 반도체 산업, 디스플레이 산업, 공정 / 장비 산업, 소재 / 부품 산업, 자동차 산업 등 매우 다양한 분야에 진출할 수 있습니다. 또한 국공립 연구소 및 정부기관에서도 근무할 수 있으며, 학문의 특성상 벤처 창업을 통한 기업가로의 꿈을 실현할 수도 있습니다.\n \n자동차\n의료기기\n공정 / 장비\n반도체\n정부출연 연구소\n바이오\n디스플레이\n소재"
  },
  {
    "input": "마케팅커뮤니케이션테크놀로지융합전공 졸업하면 뭐 먹고 살아?",
    "output": "디지털마케팅 및 마케팅커뮤니케이션 분야에서 아래와 같은 전문가로 활동할 수 있습니다.\n \n디지털마케팅 솔루션/플랫폼 기획자 (Digital Marketing Solution/Platform Planner)\n마케팅 및 광고 데이터 분석전문가 (MarTech/AdTech Data Analyst)\n디지털마케팅 기획자(Account Planner & Executive)\n디지털마케팅캠페인 (Digital Campaign Director)\n디지털매체 기획자 (Digital Media planner)"
  },
  {
    "input": "공공인재융합 전공 소개해볼래?",
    "output": "공공인재융합전공은 강원권 5개 대학이 참여하는 지역선도대학 육성사업의 일환으로 공공기관 취업 맞춤형 인재양성을 위해 2020년 1학기에 새롭게 출범하였습니다. 공공인재융합전공은 공공기관 수요에 따라 문제해결능력을 가진 실무적 인재, 창의적 사고를 함양한 초연결성 혁신 인재를 육성하는 목표를 가지고 있습니다. 본 전공을 통해 학생들은 혁신도시 및 전국에 소재하는 다양한 공공기관에 취업 기회를 얻을 수 있습니다.\n 본 융합 전공은 공공기관 수요가 높은 교과목들로 구성되어 있으며, 기존 유관 학과들에 개설된 교과목들과의 폭넓은 코드쉐어로 전공에 대한 접근성이 높습니다. 하계 및 동계에는 혁신도시 소재 공공기관 소속 교수가 직접 강의하는 캠프를 운영하며, 캠프에 참여하는 학생들에게는 소정의 장학금도 지급하고 있습니다."
  },
  {
    "input": "공공인재융합 전공 인재상이 궁금하네",
    "output": "1. 사회혁신에 기여할 공공분야 핵심 인재\n2. 수요 맞춤형 융합 전문 인재\n3. 문제해결능력을 함양한 실무 인재\n4. 창의적 사고를 함양한 초연결성 혁신 인재"
  },
  {
    "input": "공공인재융합 전공 졸업 후 진로 말해봐",
    "output": "혁신도시 소재 공공기관 16개 기관 맞춤형 취업\n전국 단위 공공기관(340개) 및 지방자치단체 공공기관(893개) 취업\n지역 핵심산업 (헬스케어, 관광, 문화, 남북 교류협력) 관련 기업 취업\n일반직 공무원 등"
  }
"""
#2 교수정보
prof_info = """
{
    "input": "김명준 교수님 연구실 어디야?",
    "output": "김명준 교수님은 인문학부 교수님으로 분야는 한국고전시가입니다. 교수님의 연구실은 대학본부·인문 1관 2609호입니다. 연락처는 033-248-1513이며 이메일 주소는 oldpoetry@hallym.ac.kr입니다."
  },
  {
    "input": "김명준 교수님 전화번호 알려줘",
    "output": "김명준 교수님은 인문학부 교수님으로 분야는 한국고전시가입니다. 교수님의 연락처는 033-248-1513입니다. 연구실은 대학본부·인문 1관 2609호이며 이메일 주소는 oldpoetry@hallym.ac.kr입니다."
  },
  {
    "input": "김명준 교수님 이메일 알려줘",
    "output": "김명준 교수님은 인문학부 교수님으로 분야는 한국고전시가입니다. 교수님의 이메일 주소는 oldpoetry@hallym.ac.kr입니다. 연락처는 033-248-1513이며 연구실은 대학본부·인문 1관 2609호입니다."
  },
  {
    "input": "이병기 교수님 연구실 어디야?",
    "output": "이병기 교수님은 인문학부 교수님으로 분야는 한국어문법사입니다. 교수님의 연구실은 대학본부·인문 1관 2607호입니다. 연락처는 033-248-1512이며 이메일 주소는 prefalla@hallym.ac.kr입니다."
  },
  {
    "input": "이병기 교수님 전화번호 알려줘",
    "output": "이병기 교수님은 인문학부 교수님으로 분야는 한국어문법사입니다. 교수님의 연락처는 033-248-1512입니다. 연구실은 대학본부·인문 1관 2607호이며 이메일 주소는 prefalla@hallym.ac.kr입니다."
  },
  {
    "input": "이병기 교수님 이메일 알려줘",
    "output": "이병기 교수님은 인문학부 교수님으로 분야는 한국어문법사입니다. 교수님의 이메일 주소는 prefalla@hallym.ac.kr입니다. 연락처는 033-248-1512이며 연구실은 대학본부·인문 1관 2607호입니다."
  },
  {
    "input": "신서인 교수님 연구실 어디야?",
    "output": "신서인 교수님은 인문학부 교수님으로 분야는 현대국어문법입니다. 교수님의 연구실은 대학본부·인문 1관 2605호입니다. 연락처는 033-248-1517이며 이메일 주소는 dryice@hallym.ac.kr입니다."
  },
  {
    "input": "신서인 교수님 전화번호 알려줘",
    "output": "신서인 교수님은 인문학부 교수님으로 분야는 현대국어문법입니다. 교수님의 연락처는 033-248-1517입니다. 연구실은 대학본부·인문 1관 2605호이며 이메일 주소는 dryice@hallym.ac.kr입니다."
  },
  {
    "input": "신서인 교수님 이메일 알려줘",
    "output": "신서인 교수님은 인문학부 교수님으로 분야는 현대국어문법입니다. 교수님의 이메일 주소는 dryice@hallym.ac.kr입니다. 연락처는 033-248-1517이며 연구실은 대학본부·인문 1관 2605호입니다."
  },
  {
    "input": "서은혜 교수님 연구실 어디야?",
    "output": "서은혜 교수님은 인문학부 조교수님으로 분야는 한국고전소설입니다. 교수님의 연구실은 대학본부·인문 1관 2610호입니다. 연락처는 033-248-1514이며 이메일 주소는 seh@hallym.ac.kr입니다."
  },
  {
    "input": "서은혜 교수님 전화번호 알려줘",
    "output": "서은혜 교수님은 인문학부 조교수님으로 분야는 한국고전소설입니다. 교수님의 연락처는 033-248-1514입니다. 연구실은 대학본부·인문 1관 2610호이며 이메일 주소는 seh@hallym.ac.kr입니다."
  },
  {
    "input": "서은혜 교수님 이메일 알려줘",
    "output": "서은혜 교수님은 인문학부 조교수님으로 분야는 한국고전소설입니다. 교수님의 이메일 주소는 seh@hallym.ac.kr입니다. 연락처는 033-248-1514이며 연구실은 대학본부·인문 1관 2610호입니다."
  },
  {
    "input": "유인선 교수님 연구실 어디야?",
    "output": "유인선 교수님은 인문학부 조교수님으로 분야는 한국고전소설입니다. 교수님의 연구실은 대학본부·인문 1관 2406호입니다. 연락처는 033-248-1516이며 이메일 주소는 insun710@hallym.ac.kr입니다."
  },
  {
    "input": "유인선 교수님 전화번호 알려줘",
    "output": "유인선 교수님은 인문학부 조교수님으로 분야는 한국고전소설입니다. 교수님의 연락처는 033-248-1516입니다. 연구실은 대학본부·인문 1관 2406호이며 이메일 주소는 insun710@hallym.ac.kr입니다."
  },
  {
    "input": "유인선 교수님 이메일 알려줘",
    "output": "유인선 교수님은 인문학부 조교수님으로 분야는 한국고전소설입니다. 교수님의 이메일 주소는 seh@hallym.ac.kr입니다. 연락처는 033-248-1514이며 연구실은 대학본부·인문 1관 2610호입니다."
  },
  {
    "input": "오춘택 교수님 연구실 어디야?",
    "output": "오춘택 교수님은 인문학부 명예교수님으로 분야는 한국고전산문입니다. 교수님의 연구실은 대학본부별관 5201호(명예교수실)입니다. 연락처는 033-248-2936이며 이메일 주소는 ctoh@hallym.ac.kr입니다."
  },
  {
    "input": "오춘택 교수님 전화번호 알려줘",
    "output": "오춘택 교수님은 인문학부 명예교수님으로 분야는 한국고전산문입니다. 교수님의 연락처는 033-248-2936입니다. 연구실은 대학본부별관 5201호(명예교수실)호이며 이메일 주소는 ctoh@hallym.ac.kr입니다."
  },
  {
    "input": "오춘택 교수님 이메일 알려줘",
    "output": "오춘택 교수님은 인문학부 명예교수님으로 분야는 한국고전산문입니다. 교수님의 이메일 주소는 ctoh@hallym.ac.kr입니다. 연락처는 033-248-2936이며 연구실은 대학본부별관 5201호(명예교수실)입니다."
  },
  {
    "input": "염정섭 교수님 연구실 어디야?",
    "output": "염정섭 교수님은 인문학부 교수님으로 분야는 조선시대사, 한국농업사, 한국사회경제사입니다. 교수님의 연구실은 대학본부·인문 1관 2533-2호입니다. 연락처는 033-248-1579이며 이메일 주소는 yeobul@hallym.ac.kr입니다."
  },
  {
    "input": "염정섭 교수님 전화번호 알려줘",
    "output": "염정섭 교수님은 인문학부 교수님으로 분야는 조선시대사, 한국농업사, 한국사회경제사입니다. 교수님의 연락처는 033-248-1579입니다. 연구실은 대학본부·인문 1관 2533-2호이며 이메일 주소는 yeobul@hallym.ac.kr입니다."
  },
  {
    "input": "염정섭 교수님 이메일 알려줘",
    "output": "염정섭 교수님은 인문학부 교수님으로 분야는 조선시대사, 한국농업사, 한국사회경제사입니다. 교수님의 이메일 주소는 yeobul@hallym.ac.kr입니다. 연락처는 033-248-1579이며 연구실은 대학본부·인문 1관 2533-2호입니다."
  },
  {
    "input": "박은재 교수님 연구실 어디야?",
    "output": "박은재 교수님은 인문학부 조교수님으로 분야는 서양 근현대사, 영국사입니다. 교수님의 연구실은 대학본부·인문 1관 2504호입니다. 연락처는 033-248-1577이며 이메일 주소는 eunjae.park@hallym.ac.kr입니다."
  },
  {
    "input": "박은재 교수님 전화번호 알려줘",
    "output": "박은재 교수님은 인문학부 조교수님으로 분야는 서양 근현대사, 영국사입니다. 교수님의 연락처는 033-248-1577입니다. 연구실은 대학본부·인문 1관 2504호이며 이메일 주소는 eunjae.park@hallym.ac.kr입니다."
  },
  {
    "input": "박은재 교수님 이메일 알려줘",
    "output": "박은재 교수님은 인문학부 조교수님으로 분야는 서양 근현대사, 영국사입니다. 교수님의 이메일 주소는 eunjae.park@hallym.ac.kr입니다. 연락처는 033-248-1577이며 연구실은 대학본부·인문 1관 2504호입니다."
  },
  {
    "input": "김아람 교수님 연구실 어디야?",
    "output": "김아람 교수님은 인문학부 조교수님으로 분야는 한국근현대사, 한국사회문학사, 지역사, 구술사입니다. 교수님의 연구실은 대학본부·인문 1관 2503호입니다. 연락처는 033-248-1575이며 이메일 주소는 aramkim@hallym.ac.kr입니다."
  },
  {
    "input": "김아람 교수님 전화번호 알려줘",
    "output": "김아람 교수님은 인문학부 조교수님으로 분야는 한국근현대사, 한국사회문학사, 지역사, 구술사입니다. 교수님의 연락처는 033-248-1575입니다. 연구실은 대학본부·인문 1관 2503호이며 이메일 주소는 aramkim@hallym.ac.kr입니다."
  },
  {
    "input": "김아람 교수님 이메일 알려줘",
    "output": "김아람 교수님은 인문학부 조교수님으로 분야는 한국근현대사, 한국사회문학사, 지역사, 구술사입니다. 교수님의 이메일 주소는 aramkim@hallym.ac.kr입니다. 연락처는 033-248-1575이며 연구실은 대학본부·인문 1관 2503호입니다."
  },
  {
    "input": "조병식 교수님 연구실 어디야?",
    "output": "조병식 교수님은 인문학부 조교수님으로 분야는 중국 근현대사, 근대 동아시아 법제사입니다. 교수님의 연구실은 대학본부·인문 1관 2506호입니다. 연락처는 033-248-1578이며 이메일 주소는 CHOBS@hallym.ac.kr입니다."
  },
  {
    "input": "조병식 교수님 전화번호 알려줘",
    "output": "조병식 교수님은 인문학부 조교수님으로 분야는 중국 근현대사, 근대 동아시아 법제사입니다. 교수님의 연락처는 033-248-1578입니다. 연구실은 대학본부·인문 1관 2506호이며 이메일 주소는 CHOBS@hallym.ac.kr입니다."
  },
  {
    "input": "조병식 교수님 이메일 알려줘",
    "output": "조병식 교수님은 인문학부 조교수님으로 분야는 중국 근현대사, 근대 동아시아 법제사입니다. 교수님의 이메일 주소는 CHOBS@hallym.ac.kr입니다. 연락처는 033-248-1578이며 연구실은 대학본부·인문 1관 2506호입니다."
  },
  {
    "input": "노혁진 교수님 연구실 어디야?",
    "output": "노혁진 교수님은 인문학부 명예교수님으로 분야는 한국선사고고학, 강원고고학입니다. 교수님의 연구실은 대학본부·인문 1관 2505호입니다. 연락처는 033-248-1571이며 이메일 주소는 hjroh@hallym.ac.kr입니다."
  },
  {
    "input": "노혁진 교수님 전화번호 알려줘",
    "output": "노혁진 교수님은 인문학부 명예교수님으로 분야는 한국선사고고학, 강원고고학입니다. 교수님의 연락처는 033-248-1571입니다. 연구실은 대학본부·인문 1관 2505호이며 이메일 주소는 hjroh@hallym.ac.kr입니다."
  },
  {
    "input": "노혁진 교수님 이메일 알려줘",
    "output": "노혁진 교수님은 인문학부 명예교수님으로 분야는 한국선사고고학, 강원고고학입니다. 교수님의 이메일 주소는 hjroh@hallym.ac.kr입니다. 연락처는 033-248-1571이며 연구실은 대학본부·인문 1관 2505호입니다."
  },
  {
    "input": "김기순 교수님 연구실 어디야?",
    "output": "김기순 교수님은 인문학부 명예교수님으로 분야는 영국사, 서양사상사입니다. 교수님의 연구실은 대학본부·인문 1관 2504호입니다. 연락처는 033-248-1577이며 이메일 주소는 kskim1@hallym.ac.kr입니다."
  },
  {
    "input": "김기순 교수님 전화번호 알려줘",
    "output": "김기순 교수님은 인문학부 명예교수님으로 분야는 영국사, 서양사상사입니다. 교수님의 연락처는 033-248-1577입니다. 연구실은 대학본부·인문 1관 2504호이며 이메일 주소는 kskim1@hallym.ac.kr입니다."
  },
  {
    "input": "김기순 교수님 이메일 알려줘",
    "output": "김기순 교수님은 인문학부 명예교수님으로 분야는 영국사, 서양사상사입니다. 교수님의 이메일 주소는 kskim1@hallym.ac.kr입니다. 연락처는 033-248-1577이며 연구실은 대학본부·인문 1관 2504호입니다."
  },
  {
    "input": "오진탁 교수님 연구실 어디야?",
    "output": "오진탁 교수님은 인문학부 교수님으로 분야는 불교철학, 노장철학, 생사학입니다. 교수님의 연구실은 대학본부·인문 1관 2512호입니다. 연락처는 033-248-1554이며 이메일 주소는 jtoh@hallym.ac.kr입니다."
  },
  {
    "input": "오진탁 교수님 전화번호 알려줘",
    "output": "오진탁 교수님은 인문학부 교수님으로 분야는 불교철학, 노장철학, 생사학입니다. 교수님의 연락처는 033-248-1554입니다. 연구실은 대학본부·인문 1관 2512호이며 이메일 주소는 jtoh@hallym.ac.kr입니다."
  },
  {
    "input": "오진탁 교수님 이메일 알려줘",
    "output": "오진탁 교수님은 인문학부 교수님으로 분야는 불교철학, 노장철학, 생사학입니다. 교수님의 이메일 주소는 jtoh@hallym.ac.kr입니다. 연락처는 033-248-1554이며 연구실은 대학본부·인문 1관 2512호입니다."
  },
  {
    "input": "주동률 교수님 연구실 어디야?",
    "output": "주동률 교수님은 인문학부 교수님으로 분야는 윤리학 및 문화철학입니다. 교수님의 연구실은 대학본부·인문 1관 2507호입니다. 연락처는 033-248-1556이며 이메일 주소는 drchoo@hallym.ac.kr입니다."
  },
  {
    "input": "주동률 교수님 전화번호 알려줘",
    "output": "주동률 교수님은 인문학부 교수님으로 분야는 윤리학 및 문화철학입니다. 교수님의 연락처는 033-248-1556입니다. 연구실은 대학본부·인문 1관 2507호이며 이메일 주소는 drchoo@hallym.ac.kr입니다."
  },
  {
    "input": "주동률 교수님 이메일 알려줘",
    "output": "주동률 교수님은 인문학부 교수님으로 분야는 윤리학 및 문화철학입니다. 교수님의 이메일 주소는 drchoo@hallym.ac.kr입니다. 연락처는 033-248-1556이며 연구실은 대학본부·인문 1관 2507호입니다."
  },
  {
    "input": "성기현 교수님 연구실 어디야?",
    "output": "성기현 교수님은 인문학부 조교수님으로 분야는 현대유럽철학, 사회철학, 예술철학입니다. 교수님의 연구실은 대학본부·인문 1관 2513호입니다. 연락처는 033-248-1553이며 이메일 주소는 ghseong@hallym.ac.kr입니다."
  },
  {
    "input": "성기현 교수님 전화번호 알려줘",
    "output": "성기현 교수님은 인문학부 조교수님으로 분야는 현대유럽철학, 사회철학, 예술철학입니다. 교수님의 연락처는 033-248-1553입니다. 연구실은 대학본부·인문 1관 2513호이며 이메일 주소는 ghseong@hallym.ac.kr입니다."
  },
  {
    "input": "성기현 교수님 이메일 알려줘",
    "output": "성기현 교수님은 인문학부 조교수님으로 분야는 현대유럽철학, 사회철학, 예술철학입니다. 교수님의 이메일 주소는 ghseong@hallym.ac.kr입니다. 연락처는 033-248-1553이며 연구실은 대학본부·인문 1관 2513호입니다."
  },
  {
    "input": "손병홍 교수님 연구실 어디야?",
    "output": "손병홍 교수님은 인문학부 명예교수님으로 분야는 논리학, 분석철학, 언어철학입니다. 교수님의 연구실은 대학본부별관 5201호(명예교수실)입니다. 손병홍 교수님의 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 bhson@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "손병홍 교수님 전화번호 알려줘",
    "output": "손병홍 교수님은 인문학부 명예교수님으로 분야는 현대유럽철학, 사회철학, 예술철학입니다. 손병홍 교수님의 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 bhson@hallym.ac.kr로 문의하실 수 있습니다. 교수님의 연구실은 대학본부별관 5201호(명예교수실)입니다."
  },
  {
    "input": "손병홍 교수님 이메일 알려줘",
    "output": "손병홍 교수님은 인문학부 명예교수님으로 분야는 현대유럽철학, 사회철학, 예술철학입니다. 교수님의 이메일 주소는 bhson@hallym.ac.kr입니다. 교수님의 연구실은 대학본부별관 5201호(명예교수실)입니다. 손병홍 교수님의 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소로 문의하실 수 있습니다."
  },
  {
    "input": "김성진 교수님 연구실 어디야?",
    "output": "김성진 교수님은 인문학부 명예교수님으로 분야는 서양고대철학, 환경철학, 서양중세철학입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 sjkphil@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "김성진 교수님 전화번호 알려줘",
    "output": "김성진 교수님은 인문학부 명예교수님으로 분야는 서양고대철학, 환경철학, 서양중세철학입니다. 교수님의 연락처와 연구실 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 sjkphil@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "김성진 교수님 이메일 알려줘",
    "output": "김성진 교수님은 인문학부 명예교수님으로 분야는 서양고대철학, 환경철학, 서양중세철학입니다. 교수님의 이메일 주소는 sjkphil@hallym.ac.kr입니다. 교수님의 연구실과 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소로 문의하실 수 있습니다."
  },
  {
    "input": "최승락 교수님 연구실 어디야?",
    "output": "최승락 교수님은 인문학부 조교수님으로 분야는 영미분석철학, 논리학입니다. 교수님의 연구실은 대학본부·인문 1관 2512호입니다. 연락처는 033-248-1554이며 이메일 주소는 ChoiSR@hallym.ac.kr입니다."
  },
  {
    "input": "최승락 교수님 전화번호 알려줘",
    "output": "최승락 교수님은 인문학부 조교수님으로 분야는 영미분석철학, 논리학입니다. 교수님의 연락처는 033-248-1554입니다. 연구실은 대학본부·인문 1관 2512호이며 이메일 주소는 ChoiSR@hallym.ac.kr입니다."
  },
  {
    "input": "최승락 교수님 이메일 알려줘",
    "output": "최승락 교수님은 인문학부 조교수님으로 분야는 영미분석철학, 논리학입니다. 교수님의 이메일 주소는 ChoiSR@hallym.ac.kr입니다. 연락처는 033-248-1554이며 연구실은 대학본부·인문 1관 2512호입니다."
  },
  {
    "input": "김번 교수님 연구실 어디야?",
    "output": "김번 교수님은 영어영문학과 교수님으로 분야는 18세기영국문학, 영미대중소설, 정신분석입니다. 교수님의 연구실은 대학본부·인문 1관 2625호입니다. 연락처는 033-248-1536이며 이메일 주소는 bkim@hallym.ac.kr입니다."
  },
  {
    "input": "김번 교수님 전화번호 알려줘",
    "output": "김번 교수님은 영어영문학과 교수님으로 분야는 18세기영국문학, 영미대중소설, 정신분석입니다. 교수님의 연락처는 033-248-1536입니다. 연구실은 대학본부·인문 1관 2625호이며 이메일 주소는 bkim@hallym.ac.kr입니다."
  },
  {
    "input": "김번 교수님 이메일 알려줘",
    "output": "김번 교수님은 영어영문학과 교수님으로 분야는 18세기영국문학, 영미대중소설, 정신분석입니다. 교수님의 이메일 주소는 bkim@hallym.ac.kr입니다. 연락처는 033-248-1536이며 연구실은 대학본부·인문 1관 2625호입니다."
  },
  {
    "input": "김용수 교수님 연구실 어디야?",
    "output": "김용수 교수님은 영어영문학과 교수님으로 분야는 미국소설, 영화, 라캉 정신분석이론, 디지털인문학입니다. 교수님의 연구실은 대학본부·인문 1관 2613호입니다. 연락처는 033-248-1538이며 이메일 주소는 yongsookim@hallym.ac.kr입니다."
  },
  {
    "input": "김용수 교수님 전화번호 알려줘",
    "output": "김용수 교수님은 영어영문학과 교수님으로 분야는 미국소설, 영화, 라캉 정신분석이론, 디지털인문학입니다. 교수님의 연락처는 033-248-1538입니다. 연구실은 대학본부·인문 1관 2613호이며 이메일 주소는 yongsookim@hallym.ac.kr입니다."
  },
  {
    "input": "김용수 교수님 이메일 알려줘",
    "output": "김용수 교수님은 영어영문학과 교수님으로 분야는 미국소설, 영화, 라캉 정신분석이론, 디지털인문학입니다. 교수님의 이메일 주소는 yongsookim@hallym.ac.kr입니다. 연락처는 033-248-1538이며 연구실은 대학본부·인문 1관 2613호입니다."
  },
  {
    "input": "카나다스 아이반 교수님 연구실 어디야?",
    "output": "카나다스 아이반 교수님은 영어영문학과 교수님으로 분야는  Drama: Shakespeare to Modern입니다. 교수님의 연구실은 대학본부·인문 1관 2623호입니다. 연락처는 033-248-1537이며 이메일 주소는 i_canadas@hallym.ac.kr입니다."
  },
  {
    "input": "카나다스 아이반 교수님 전화번호 알려줘",
    "output": "카나다스 아이반 교수님은 영어영문학과 교수님으로 분야는  Drama: Shakespeare to Modern입니다. 교수님의 연락처는 033-248-1538입니다. 연구실은 대학본부·인문 1관 2623호이며 이메일 주소는 i_canadas@hallym.ac.kr입니다."
  },
  {
    "input": "카나다스 아이반 교수님 이메일 알려줘",
    "output": "카나다스 아이반 교수님은 영어영문학과 교수님으로 분야는  Drama: Shakespeare to Modern입니다. 교수님의 이메일 주소는 i_canadas@hallym.ac.kr입니다. 연락처는 033-248-1538이며 연구실은 대학본부·인문 1관 2623호입니다."
  },
  {
    "input": "한경민 교수님 연구실 어디야?",
    "output": "한경민 교수님은 영어영문학과 교수님으로 분야는 19세기 영문학에서의 공감론 낭만주의 언어관입니다. 교수님의 연구실은 대학본부·인문 1관 2628호입니다. 연락처는 033-248-1534이며 이메일 주소는 kminhan@hallym.ac.kr입니다."
  },
  {
    "input": "한경민 교수님 전화번호 알려줘",
    "output": "한경민 교수님은 영어영문학과 교수님으로 분야는 19세기 영문학에서의 공감론 낭만주의 언어관입니다. 교수님의 연락처는 033-248-1534입니다. 연구실은 대학본부·인문 1관 2628호이며 이메일 주소는 kminhan@hallym.ac.kr입니다."
  },
  {
    "input": "한경민 교수님 이메일 알려줘",
    "output": "한경민 교수님은 영어영문학과 교수님으로 분야는 19세기 영문학에서의 공감론 낭만주의 언어관입니다. 교수님의 이메일 주소는 kminhan@hallym.ac.kr입니다. 연락처는 033-248-1534이며 연구실은 대학본부·인문 1관 2628호입니다."
  },
  {
    "input": "육청민 교수님 연구실 어디야?",
    "output": "육청민 교수님은 영어영문학과 부교수님으로 분야는 영어교육입니다. 교수님의 연구실은 대학본부·인문 1관 2626호입니다. 연락처는 033-248-1533이며 이메일 주소는 cmyook@hallym.ac.kr입니다."
  },
  {
    "input": "육청민 교수님 전화번호 알려줘",
    "output": "육청민 교수님은 영어영문학과 부교수님으로 분야는 영어교육입니다. 교수님의 연락처는 033-248-1533입니다. 연구실은 대학본부·인문 1관 2626호이며 이메일 주소는 cmyook@hallym.ac.kr입니다."
  },
  {
    "input": "육청민 교수님 이메일 알려줘",
    "output": "육청민 교수님은 영어영문학과 부교수님으로 분야는 영어교육입니다. 교수님의 이메일 주소는 cmyook@hallym.ac.kr입니다. 연락처는 033-248-1533이며 연구실은 대학본부·인문 1관 2626호입니다."
  },
  {
    "input": "한수미 교수님 연구실 어디야?",
    "output": "한수미 교수님은 영어영문학과 조교수님으로 분야는 응용언어학(코퍼스언어학), 언어평가, 빅데이터분석입니다. 교수님의 연구실은 대학본부·인문 1관 2624호입니다. 연락처는 033-248-1532이며 이메일 주소는 sumihan@hallym.ac.kr입니다."
  },
  {
    "input": "한수미 교수님 전화번호 알려줘",
    "output": "한수미 교수님은 영어영문학과 조교수님으로 분야는 응용언어학(코퍼스언어학), 언어평가, 빅데이터분석입니다. 교수님의 연락처는 033-248-1532입니다. 연구실은 대학본부·인문 1관 2624호이며 이메일 주소는 sumihan@hallym.ac.kr입니다."
  },
  {
    "input": "한수미 교수님 이메일 알려줘",
    "output": "한수미 교수님은 영어영문학과 조교수님으로 분야는 응용언어학(코퍼스언어학), 언어평가, 빅데이터분석입니다. 교수님의 이메일 주소는 sumihan@hallym.ac.kr입니다. 연락처는 033-248-1532이며 연구실은 대학본부·인문 1관 2624호입니다."
  },
  {
    "input": "김효원 교수님 연구실 어디야?",
    "output": "김효원 교수님은 영어영문학과 명예교수님으로 분야는 영미소설, 비교문학입니다. 교수님의 연구실은 대학본부별관 5201호(명예교수실)입니다. 연락처는 033-248-1530이며 이메일 주소는 hwkim@hallym.ac.kr입니다."
  },
  {
    "input": "김효원 교수님 전화번호 알려줘",
    "output": "김효원 교수님은 영어영문학과 명예교수님으로 분야는 영미소설, 비교문학입니다. 교수님의 연락처는 033-248-1530입니다. 연구실은 대학본부별관 5201호(명예교수실)이며 이메일 주소는 hwkim@hallym.ac.kr입니다."
  },
  {
    "input": "김효원 교수님 이메일 알려줘",
    "output": "김효원 교수님은 영어영문학과 명예교수님으로 분야는 영미소설, 비교문학입니다. 교수님의 이메일 주소는 hwkim@hallym.ac.kr입니다. 연락처는 033-248-1530이며 연구실은 대학본부별관 5201호(명예교수실)입니다."
  },
  {
    "input": "김영화 교수님 연구실 어디야?",
    "output": "김영화 교수님은 영어영문학과 명예교수님으로 분야는 통사론, 형태통사론, 언어습득, 언어교육입니다. 교수님의 연구실은 대학본부별관 5201호(명예교수실)입니다. 김영화 교수님의 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 youwkim@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "김영화 교수님 전화번호 알려줘",
    "output": "김영화 교수님은 영어영문학과 명예교수님으로 분야는 통사론, 형태통사론, 언어습득, 언어교육입니다. 김영화 교수님의 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 youwkim@hallym.ac.kr로 문의하실 수 있습니다. 연구실은 대학본부별관 5201호(명예교수실)입니다."
  },
  {
    "input": "김영화 교수님 이메일 알려줘",
    "output": "김영화 교수님은 영어영문학과 명예교수님으로 분야는 통사론, 형태통사론, 언어습득, 언어교육입니다. 교수님의 이메일 주소는 youwkim@hallym.ac.kr입니다. 연구실은 대학본부별관 5201호(명예교수실)입니다. 김영화 교수님의 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소로 문의하실 수 있습니다."
  },
  {
    "input": "임혜순 교수님 연구실 어디야?",
    "output": "임혜순 교수님은 영어영문학과 명예교수님으로 분야는 사회언어학, 영어사, 중세영어입니다. 교수님의 연구실은 대학본부별관 5201호(명예교수실)입니다. 연락처는 033-248-1530이며 이메일 주소는 hslim@hallym.ac.kr입니다."
  },
  {
    "input": "임혜순 교수님 전화번호 알려줘",
    "output": "임혜순 교수님은 영어영문학과 명예교수님으로 분야는 사회언어학, 영어사, 중세영어입니다. 교수님의 연락처는 033-248-1530입니다. 연구실은 대학본부별관 5201호(명예교수실)이며 이메일 주소는 hslim@hallym.ac.kr입니다."
  },
  {
    "input": "임혜순 교수님 이메일 알려줘",
    "output": "임혜순 교수님은 영어영문학과 명예교수님으로 분야는 사회언어학, 영어사, 중세영어입니다. 교수님의 이메일 주소는 hslim@hallym.ac.kr입니다. 연락처는 033-248-1530이며 연구실은 대학본부별관 5201호(명예교수실)입니다."
  },
  {
    "input": "고민희 교수님 연구실 어디야?",
    "output": "고민희 교수님은 중국학과 명예교수님으로 분야는 중국소설입니다. 교수님의 연구실은 대학본부·인문 1관 2527호입니다. 연락처는 033-248-1591이며 이메일 주소는 miniko@hallym.ac.kr입니다."
  },
  {
    "input": "고민희 교수님 전화번호 알려줘",
    "output": "고민희 교수님은 중국학과 명예교수님으로 분야는 중국소설입니다. 교수님의 연락처는 033-248-1591입니다. 연구실은 대학본부·인문 1관 2527호이며 이메일 주소는 miniko@hallym.ac.kr입니다."
  },
  {
    "input": "고민희 교수님 이메일 알려줘",
    "output": "고민희 교수님은 중국학과 명예교수님으로 분야는 중국소설입니다. 교수님의 이메일 주소는 miniko@hallym.ac.kr입니다. 연락처는 033-248-1591이며 연구실은 대학본부·인문 1관 2527호입니다."
  },
  {
    "input": "김민호 교수님 연구실 어디야?",
    "output": "김민호 교수님은 중국학과 교수님으로 분야는 중국고전서사입니다. 교수님의 연구실은 대학본부·인문 1관 2532호입니다. 연락처는 033-248-1595이며 이메일 주소는 kmh@hallym.ac.kr입니다."
  },
  {
    "input": "김민호 교수님 전화번호 알려줘",
    "output": "김민호 교수님은 중국학과 교수님으로 분야는 중국고전서사입니다. 교수님의 연락처는 033-248-1595입니다. 연구실은 대학본부·인문 1관 2532호이며 이메일 주소는 kmh@hallym.ac.kr입니다."
  },
  {
    "input": "김민호 교수님 이메일 알려줘",
    "output": "김민호 교수님은 중국학과 교수님으로 분야는 중국고전서사입니다. 교수님의 이메일 주소는 kmh@hallym.ac.kr입니다. 연락처는 033-248-1595이며 연구실은 대학본부·인문 1관 2532호입니다."
  },
  {
    "input": "성시일 교수님 연구실 어디야?",
    "output": "성시일 교수님은 중국학과 교수님으로 분야는 중국경제·통상입니다. 교수님의 연구실은 대학본부·인문 1관 2526호입니다. 연락처는 033-248-1593이며 이메일 주소는 siil@hallym.ac.kr입니다."
  },
  {
    "input": "성시일 교수님 전화번호 알려줘",
    "output": "성시일 교수님은 중국학과 교수님으로 분야는 중국경제·통상입니다. 교수님의 연락처는 033-248-1593입니다. 연구실은 대학본부·인문 1관 2526호이며 이메일 주소는 siil@hallym.ac.kr입니다."
  },
  {
    "input": "양태근 교수님 연구실 어디야?",
    "output": "양태근 교수님은 중국학과 교수님으로 분야는 중국근현대정치사상, 대만사입니다. 교수님의 연구실은 대학본부·인문 1관 2531호입니다. 연락처는 033-248-1596이며 이메일 주소는 ytk@hallym.ac.kr입니다."
  },
  {
    "input": "양태근 교수님 전화번호 알려줘",
    "output": "양태근 교수님은 중국학과 교수님으로 분야는 중국근현대정치사상, 대만사입니다. 교수님의 연락처는 033-248-1596입니다. 연구실은 대학본부·인문 1관 2531호이며 이메일 주소는 ytk@hallym.ac.kr입니다."
  },
  {
    "input": "양태근 교수님 이메일 알려줘",
    "output": "양태근 교수님은 중국학과 교수님으로 분야는 중국근현대정치사상, 대만사입니다. 교수님의 이메일 주소는 ytk@hallym.ac.kr입니다. 연락처는 033-248-1596이며 연구실은 대학본부·인문 1관 2531호입니다."
  },
  {
    "input": "조정래 교수님 연구실 어디야?",
    "output": "조정래 교수님은 중국학과 교수님으로 분야는 동양미학·예술사입니다. 교수님의 연구실은 대학본부·인문 1관 2529호입니다. 연락처는 033-248-1594이며 이메일 주소는 jjr@hallym.ac.kr입니다."
  },
  {
    "input": "조정래 교수님 전화번호 알려줘",
    "output": "조정래 교수님은 중국학과 교수님으로 분야는 동양미학·예술사입니다. 교수님의 연락처는 033-248-1594입니다. 연구실은 대학본부·인문 1관 2529호이며 이메일 주소는 jjr@hallym.ac.kr입니다."
  },
  {
    "input": "조정래 교수님 이메일 알려줘",
    "output": "조정래 교수님은 중국학과 교수님으로 분야는 동양미학·예술사입니다. 교수님의 이메일 주소는 jjr@hallym.ac.kr입니다. 연락처는 033-248-1594이며 연구실은 대학본부·인문 1관 2529호입니다."
  },
  {
    "input": "이지현 교수님 연구실 어디야?",
    "output": "이지현 교수님은 중국학과 교수님으로 분야는 현대중국어 문법 및 교수법입니다. 교수님의 연구실은 대학본부·인문 1관 2611호입니다. 연락처는 033-248-1592이며 이메일 주소는 jlee06@hallym.ac.kr입니다."
  },
  {
    "input": "이지현 교수님 전화번호 알려줘",
    "output": "이지현 교수님은 중국학과 교수님으로 분야는 현대중국어 문법 및 교수법입니다. 교수님의 연락처는 033-248-1592입니다. 연구실은 대학본부·인문 1관 2611호이며 이메일 주소는 jlee06@hallym.ac.kr입니다."
  },
  {
    "input": "이지현 교수님 이메일 알려줘",
    "output": "이지현 교수님은 중국학과 교수님으로 분야는 현대중국어 문법 및 교수법입니다. 교수님의 이메일 주소는 jlee06@hallym.ac.kr입니다. 연락처는 033-248-1592이며 연구실은 대학본부·인문 1관 2611호입니다."
  },
  {
    "input": "염철 교수님 연구실 어디야?",
    "output": "염철 교수님은 중국학과 부교수님으로 분야는 언어학 및 한중통번역입니다. 교수님의 연구실은 대학본부·인문 1관 2528호입니다. 연락처는 033-248-1597이며 이메일 주소는 lianzhe@hallym.ac.kr입니다."
  },
  {
    "input": "염철 교수님 전화번호 알려줘",
    "output": "염철 교수님은 중국학과 부교수님으로 분야는 언어학 및 한중통번역입니다. 교수님의 연락처는 033-248-1597입니다. 연구실은 대학본부·인문 1관 2528호이며 이메일 주소는 lianzhe@hallym.ac.kr입니다."
  },
  {
    "input": "염철 교수님 이메일 알려줘",
    "output": "염철 교수님은 중국학과 부교수님으로 분야는 언어학 및 한중통번역입니다. 교수님의 이메일 주소는 lianzhe@hallym.ac.kr입니다. 연락처는 033-248-1597이며 연구실은 대학본부·인문 1관 2528호입니다."
  },
  {
    "input": "사이토 아케미 교수님 연구실 어디야?",
    "output": "사이토 아케미 교수님은 일본학과 명예교수님으로 분야는 일본어학, 일본어교육입니다. 교수님의 연구실은 대학본부별관 명예교수실입니다. 사이토 아케미 교수님의 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 saito@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "사이토 아케미 교수님 전화번호 알려줘",
    "output": "사이토 아케미 교수님은 일본학과 명예교수님으로 분야는 일본어학, 일본어교육입니다. 사이토 아케미 교수님의 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 saito@hallym.ac.kr로 문의하실 수 있습니다. 연구실은 대학본부별관 명예교수실입니다."
  },
  {
    "input": "사이토 아케미 교수님 이메일 알려줘",
    "output": "사이토 아케미 교수님은 일본학과 명예교수님으로 분야는 일본어학, 일본어교육입니다. 교수님의 이메일 주소는 lianzhe@hallym.ac.kr입니다. 연구실은 대학본부별관 명예교수실입니다. 사이토 아케미 교수님의 연락처는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소로 문의하실 수 있습니다."
  },
  {
    "input": "서정완 교수님 연구실 어디야?",
    "output": "서정완 교수님은 일본학과 교수님으로 분야는 能樂史, 일본 예능사입니다. 교수님의 연구실은 대학본부·인문 1관 2404호입니다. 연락처는 033-248-1611이며 이메일 주소는 jsuh@hallym.ac.kr입니다."
  },
  {
    "input": "서정완 교수님 전화번호 알려줘",
    "output": "서정완 교수님은 일본학과 교수님으로 분야는 能樂史, 일본 예능사입니다. 교수님의 연락처는 033-248-1611입니다. 연구실은 대학본부·인문 1관 2404호이며 이메일 주소는 jsuh@hallym.ac.kr입니다."
  },
  {
    "input": "서정완 교수님 이메일 알려줘",
    "output": "서정완 교수님은 일본학과 교수님으로 분야는 能樂史, 일본 예능사입니다. 교수님의 이메일 주소는 jsuh@hallym.ac.kr입니다. 연락처는 033-248-1611이며 연구실은 대학본부·인문 1관 2404호입니다."
  },
  {
    "input": "남기학 교수님 연구실 어디야?",
    "output": "남기학 교수님은 일본학과 교수님으로 분야는 일본사, 중세입니다. 교수님의 연구실은 대학본부·인문 1관 2405호입니다. 연락처는 033-248-1613이며 이메일 주소는 khnam@hallym.ac.kr입니다."
  },
  {
    "input": "남기학 교수님 전화번호 알려줘",
    "output": "남기학 교수님은 일본학과 교수님으로 분야는 일본사, 중세입니다. 교수님의 연락처는 033-248-1613입니다. 연구실은 대학본부·인문 1관 2405호이며 이메일 주소는 khnam@hallym.ac.kr입니다."
  },
  {
    "input": "남기학 교수님 이메일 알려줘",
    "output": "남기학 교수님은 일본학과 교수님으로 분야는 일본사, 중세입니다. 교수님의 이메일 주소는 khnam@hallym.ac.kr입니다. 연락처는 033-248-1613이며 연구실은 대학본부·인문 1관 2405호입니다."
  },
  {
    "input": "이지원 교수님 연구실 어디야?",
    "output": "이지원 교수님은 일본학과 교수님으로 분야는 현대일본사회문화입니다. 교수님의 연구실은 대학본부·인문 1관 2403호입니다. 연락처는 033-248-1612이며 이메일 주소는 cw2403@hallym.ac.kr입니다."
  },
  {
    "input": "이지원 교수님 전화번호 알려줘",
    "output": "이지원 교수님은 일본학과 교수님으로 분야는 현대일본사회문화입니다. 교수님의 연락처는 033-248-1612입니다. 연구실은 대학본부·인문 1관 2403호이며 이메일 주소는 cw2403@hallym.ac.kr입니다."
  },
  {
    "input": "이지원 교수님 이메일 알려줘",
    "output": "이지원 교수님은 일본학과 교수님으로 분야는 현대일본사회문화입니다. 교수님의 이메일 주소는 cw2403@hallym.ac.kr입니다. 연락처는 033-248-1612이며 연구실은 대학본부·인문 1관 2403호입니다."
  },
  {
    "input": "고시로 아끼코 교수님 연구실 어디야?",
    "output": "고시로 아끼코 교수님은 일본학과 부교수님으로 분야는 일본어학, 일본어음성학입니다. 교수님의 연구실은 대학본부·인문 1관 2401호입니다. 연락처는 033-248-1615이며 이메일 주소는 koshiro@hallym.ac.kr입니다."
  },
  {
    "input": "고시로 아끼코 교수님 전화번호 알려줘",
    "output": "고시로 아끼코 교수님은 일본학과 부교수님으로 분야는 일본어학, 일본어음성학입니다. 교수님의 연락처는 033-248-1615입니다. 연구실은 대학본부·인문 1관 2401호이며 이메일 주소는 koshiro@hallym.ac.kr입니다."
  },
  {
    "input": "고시로 아끼코 교수님 이메일 알려줘",
    "output": "고시로 아끼코 교수님은 일본학과 부교수님으로 분야는 일본어학, 일본어음성학입니다. 교수님의 이메일 주소는 koshiro@hallym.ac.kr입니다. 연락처는 033-248-1615이며 연구실은 대학본부·인문 1관 2401호입니다."
  },
  {
    "input": "황경자 교수님 연구실 어디야?",
    "output": "황경자 교수님은 일본학과 부교수님으로 분야는 일본어학전공, 일한번역연구입니다. 교수님의 연구실은 대학본부·인문 1관 2402호입니다. 연락처는 033-248-1617이며 이메일 주소는 gjhwang@hallym.ac.kr입니다."
  },
  {
    "input": "황경자 교수님 전화번호 알려줘",
    "output": "황경자 교수님은 일본학과 부교수님으로 분야는 일본어학전공, 일한번역연구입니다. 교수님의 연락처는 033-248-1617입니다. 연구실은 대학본부·인문 1관 2402호이며 이메일 주소는 gjhwang@hallym.ac.kr입니다."
  },
  {
    "input": "황경자 교수님 이메일 알려줘",
    "output": "황경자 교수님은 일본학과 부교수님으로 분야는 일본어학전공, 일한번역연구입니다. 교수님의 이메일 주소는 gjhwang@hallym.ac.kr입니다. 연락처는 033-248-1617이며 연구실은 대학본부·인문 1관 2402호입니다."
  },
  {
    "input": "오현열 교수님 연구실 어디야?",
    "output": "오현열 교수님은 일본학과 겸임교수님으로 분야는 일본문학입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 oh2576548@hanmail.net으로 문의하실 수 있습니다."
  },
  {
    "input": "오현열 교수님 전화번호 알려줘",
    "output": "오현열 교수님은 일본학과 겸임교수님으로 분야는 일본문학입니다. 교수님의 연락처와 연구실 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 oh2576548@hanmail.net으로 문의하실 수 있습니다."
  },
  {
    "input": "오현열 교수님 이메일 알려줘",
    "output": "오현열 교수님은 일본학과 겸임교수님으로 분야는 일본문학입니다. 교수님의 이메일 주소는 gjhwang@hallym.ac.kr입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소로 문의하실 수 있습니다."
  },
  {
    "input": "박혜경 교수님 연구실 어디야?",
    "output": "박혜경 교수님은 러시아학과 교수님으로 분야는 러시아문학입니다. 교수님의 연구실은 대학본부·인문 1관 2409호입니다. 연락처는 033-248-1633이며 이메일 주소는 hkpark1@hallym.ac.kr입니다."
  },
  {
    "input": "박혜경 교수님 전화번호 알려줘",
    "output": "박혜경 교수님은 러시아학과 교수님으로 분야는 러시아문학입니다. 교수님의 연락처는 033-248-1633입니다. 연구실은 대학본부·인문 1관 2409호이며 이메일 주소는 hkpark1@hallym.ac.kr입니다."
  },
  {
    "input": "박혜경 교수님 이메일 알려줘",
    "output": "박혜경 교수님은 러시아학과 교수님으로 분야는 러시아문학입니다. 교수님의 이메일 주소는 hkpark1@hallym.ac.kr입니다. 연락처는 033-248-1633이며 연구실은 대학본부·인문 1관 2409호입니다."
  },
  {
    "input": "홍택규 교수님 연구실 어디야?",
    "output": "홍택규 교수님은 러시아학과 교수님으로 분야는 러시아어학입니다. 교수님의 연구실은 대학본부·인문 1관 2410호입니다. 연락처는 033-248-1635이며 이메일 주소는 jacob@hallym.ac.kr입니다."
  },
  {
    "input": "홍택규 교수님 전화번호 알려줘",
    "output": "홍택규 교수님은 러시아학과 교수님으로 분야는 러시아어학입니다. 교수님의 연락처는 033-248-1635입니다. 연구실은 대학본부·인문 1관 2410호이며 이메일 주소는 jacob@hallym.ac.kr입니다."
  },
  {
    "input": "홍택규 교수님 이메일 알려줘",
    "output": "홍택규 교수님은 러시아학과 교수님으로 분야는 러시아어학입니다. 교수님의 이메일 주소는 jacob@hallym.ac.kr입니다. 연락처는 033-248-1635이며 연구실은 대학본부·인문 1관 2410호입니다."
  },
  {
    "input": "고영웅 교수님 연구실 어디야?",
    "output": "고영웅 교수님은 소프트웨어학부 교수님으로 분야는 운영체제입니다. 교수님의 연구실은 공학관(1관) A1207호입니다. 연락처는 033-248-2329이며 이메일 주소는 yuko@hallym.ac.kr입니다."
  },
  {
    "input": "고영웅 교수님 전화번호 알려줘",
    "output": "고영웅 교수님은 소프트웨어학부 교수님으로 분야는 운영체제입니다. 교수님의 연락처는 033-248-2329입니다. 연구실은 공학관(1관) A1207호이며 이메일 주소는 yuko@hallym.ac.kr입니다."
  },
  {
    "input": "고영웅 교수님 이메일 알려줘",
    "output": "고영웅 교수님은 소프트웨어학부 교수님으로 분야는 운영체제입니다. 교수님의 이메일 주소는 yuko@hallym.ac.kr입니다. 연락처는 033-248-2329이며 연구실은 공학관(1관) A1207호입니다."
  },
  {
    "input": "김동회 교수님 연구실 어디야?",
    "output": "김동회 교수님은 소프트웨어학부 부교수님으로 분야는 생물정보학, 데이터마이닝입니다. 교수님의 연구실은 공학관(1관) A1203호입니다. 연락처는 033-248-2332이며 이메일 주소는 kdh@hallym.ac.kr입니다."
  },
  {
    "input": "김동회 교수님 전화번호 알려줘",
    "output": "김동회 교수님은 소프트웨어학부 부교수님으로 분야는 생물정보학, 데이터마이닝입니다. 교수님의 연락처는 033-248-2332입니다. 연구실은 공학관(1관) A1203호이며 이메일 주소는 kdh@hallym.ac.kr입니다."
  },
  {
    "input": "김동회 교수님 이메일 알려줘",
    "output": "김동회 교수님은 소프트웨어학부 부교수님으로 분야는 생물정보학, 데이터마이닝입니다. 교수님의 이메일 주소는 kdh@hallym.ac.kr입니다. 연락처는 033-248-2332이며 연구실은 공학관(1관) A1203호입니다."
  },
  {
    "input": "김백섭 교수님 연구실 어디야?",
    "output": "김백섭 교수님은 소프트웨어학부 명예교수님으로 분야는 패턴인식입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 bskim@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "김백섭 교수님 전화번호 알려줘",
    "output": "김백섭 교수님은 소프트웨어학부 명예교수님으로 분야는 패턴인식입니다. 교수님의 연락처와 연구실 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 bskim@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "김백섭 교수님 이메일 알려줘",
    "output": "김백섭 교수님은 소프트웨어학부 명예교수님으로 분야는 패턴인식입니다. 교수님의 이메일 주소는 kdh@hallym.ac.kr입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소로 문의하실 수 있습니다."
  },
  {
    "input": "김병정 교수님 연구실 어디야?",
    "output": "김병정 교수님은 소프트웨어학부 조교수님으로 분야는 전자공학입니다. 교수님의 연구실은 공학관(1관) A1203호입니다. 연락처는 033-248-2356이며 이메일 주소는 kevin@hallym.ac.kr입니다."
  },
  {
    "input": "김병정 교수님 전화번호 알려줘",
    "output": "김병정 교수님은 소프트웨어학부 조교수님으로 분야는 전자공학입니다. 교수님의 연락처는 033-248-2356입니다. 연구실은 공학관(1관) A1203호이며 이메일 주소는 kevin@hallym.ac.kr입니다."
  },
  {
    "input": "김병정 교수님 이메일 알려줘",
    "output": "김병정 교수님은 소프트웨어학부 조교수님으로 분야는 전자공학입니다. 교수님의 이메일 주소는 kevin@hallym.ac.kr입니다. 연락처는 033-248-2356이며 연구실은 공학관(1관) A1203호입니다."
  },
  {
    "input": "김선정 교수님 연구실 어디야?",
    "output": "김선정 교수님은 소프트웨어학부 교수님으로 분야는 컴퓨터그래픽스입니다. 교수님의 연구실은 공학관(1관) 1215호입니다. 연락처는 033-248-2323이며 이메일 주소는 sunkim@hallym.ac.kr입니다."
  },
  {
    "input": "김선정 교수님 전화번호 알려줘",
    "output": "김선정 교수님은 소프트웨어학부 교수님으로 분야는 컴퓨터그래픽스입니다. 교수님의 연락처는 033-248-2323입니다. 연구실은 공학관(1관) 1215호이며 이메일 주소는 sunkim@hallym.ac.kr입니다."
  },
  {
    "input": "김선정 교수님 이메일 알려줘",
    "output": "김선정 교수님은 소프트웨어학부 교수님으로 분야는 컴퓨터그래픽스입니다. 교수님의 이메일 주소는 sunkim@hallym.ac.kr입니다. 연락처는 033-248-2323이며 연구실은 공학관(1관) 1215호입니다."
  },
  {
    "input": "김유섭 교수님 연구실 어디야?",
    "output": "김유섭 교수님은 소프트웨어학부 교수님으로 분야는 자연어처리입니다. 교수님의 연구실은 공학관(1관) A1306호입니다. 연락처는 033-248-2324이며 이메일 주소는 yskim01@hallym.ac.kr입니다."
  },
  {
    "input": "김유섭 교수님 전화번호 알려줘",
    "output": "김유섭 교수님은 소프트웨어학부 교수님으로 분야는 자연어처리입니다. 교수님의 연락처는 033-248-2324입니다. 연구실은 공학관(1관) A1306호이며 이메일 주소는 yskim01@hallym.ac.kr입니다."
  },
  {
    "input": "김유섭 교수님 이메일 알려줘",
    "output": "김유섭 교수님은 소프트웨어학부 교수님으로 분야는 자연어처리입니다. 교수님의 이메일 주소는 yskim01@hallym.ac.kr입니다. 연락처는 033-248-2324이며 연구실은 공학관(1관) A1306호입니다."
  },
  {
    "input": "김은주 교수님 연구실 어디야?",
    "output": "김은주 교수님은 소프트웨어학부 부교수님으로 분야는 인터넷 정보처리입니다. 교수님의 연구실은 공학관(1관) A1303호입니다. 연락처는 033-248-2325이며 이메일 주소는 ejkim628@hallym.ac.kr입니다."
  },
  {
    "input": "김은주 교수님 전화번호 알려줘",
    "output": "김은주 교수님은 소프트웨어학부 부교수님으로 분야는 인터넷 정보처리입니다. 교수님의 연락처는 033-248-2325입니다. 연구실은 공학관(1관) A1303호이며 이메일 주소는 ejkim628@hallym.ac.kr입니다."
  },
  {
    "input": "김은주 교수님 이메일 알려줘",
    "output": "김은주 교수님은 소프트웨어학부 부교수님으로 분야는 인터넷 정보처리입니다. 교수님의 이메일 주소는 ejkim628@hallym.ac.kr입니다. 연락처는 033-248-2325이며 연구실은 공학관(1관) A1303호입니다."
  },
  {
    "input": "김의직 교수님 연구실 어디야?",
    "output": "김의직 교수님은 소프트웨어학부 부교수님으로 분야는 무선통신·네트워크, 사물인터넷(IoT)입니다. 교수님의 연구실은 공학관(1관) 1203호입니다. 연락처는 033-248-2333이며 이메일 주소는 ejkim32@hallym.ac.kr입니다."
  },
  {
    "input": "김의직 교수님 전화번호 알려줘",
    "output": "김의직 교수님은 소프트웨어학부 부교수님으로 분야는 무선통신·네트워크, 사물인터넷(IoT)입니다. 교수님의 연락처는 033-248-2333입니다. 연구실은 공학관(1관) 1203호이며 이메일 주소는 ejkim32@hallym.ac.kr입니다."
  },
  {
    "input": "김의직 교수님 이메일 알려줘",
    "output": "김의직 교수님은 소프트웨어학부 부교수님으로 분야는 무선통신·네트워크, 사물인터넷(IoT)입니다. 교수님의 이메일 주소는 ejkim32@hallym.ac.kr입니다. 연락처는 033-248-2333이며 연구실은 공학관(1관) 1203호입니다."
  },
  {
    "input": "김점근 교수님 연구실 어디야?",
    "output": "김점근 교수님은 소프트웨어학부 명예교수님으로 분야는 모터제어응용입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하실 경우 교수님의 이메일 주소 jkim@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "김점근 교수님 전화번호 알려줘",
    "output": "김점근 교수님은 소프트웨어학부 명예교수님으로 분야는 모터제어응용입니다. 교수님의 연락처와 연구실 정보는 제공되어 있지 않습니다. 문의가 필요하실 경우 교수님의 이메일 주소 jkim@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "김점근 교수님 이메일 알려줘",
    "output": "김점근 교수님은 소프트웨어학부 명예교수님으로 분야는 모터제어응용입니다. 교수님의 이메일 주소는 ejkim32@hallym.ac.kr입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하실 경우 교수님의 이메일 주소로 문의하실 수 있습니다."
  },
  {
    "input": "김종대 교수님 연구실 어디야?",
    "output": "김종대 교수님은 소프트웨어학부 교수님으로 분야는 가상현실, 컴퓨터 그래픽스입니다. 교수님의 연구실은 공학관(1관) 1207호입니다. 연락처는 033-248-2320이며 이메일 주소는 kimjd@hallym.ac.kr입니다."
  },
  {
    "input": "김종대 교수님 전화번호 알려줘",
    "output": "김종대 교수님은 소프트웨어학부 교수님으로 분야는 가상현실, 컴퓨터 그래픽스입니다. 교수님의 연락처는 033-248-2320입니다. 연구실은 공학관(1관) 1207호이며 이메일 주소는 kimjd@hallym.ac.kr입니다."
  },
  {
    "input": "김종대 교수님 이메일 알려줘",
    "output": "김종대 교수님은 소프트웨어학부 교수님으로 분야는 가상현실, 컴퓨터 그래픽스입니다. 교수님의 이메일 주소는 kimjd@hallym.ac.kr입니다. 연락처는 033-248-2320이며 연구실은 공학관(1관) 1207호입니다."
  },
  {
    "input": "김진 교수님 연구실 어디야?",
    "output": "김진 교수님은 소프트웨어학부 교수님으로 분야는 생물정보학입니다. 교수님의 연구실은 자연과학관(7관) 7618호입니다. 연락처는 033-248-2318이며 이메일 주소는 jinkim@hallym.ac.kr입니다."
  },
  {
    "input": "김진 교수님 전화번호 알려줘",
    "output": "김진 교수님은 소프트웨어학부 교수님으로 분야는 생물정보학입니다. 교수님의 연락처는 033-248-2318입니다. 연구실은 자연과학관(7관) 7618호이며 이메일 주소는 jinkim@hallym.ac.kr입니다."
  },
  {
    "input": "김진 교수님 이메일 알려줘",
    "output": "김진 교수님은 소프트웨어학부 교수님으로 분야는 생물정보학입니다. 교수님의 이메일 주소는 jinkim@hallym.ac.kr입니다. 연락처는 033-248-2318이며 연구실은 자연과학관(7관) 7618호입니다."
  },
  {
    "input": "문규 교수님 연구실 어디야?",
    "output": "문규 교수님은 소프트웨어학부 명예교수님으로 분야는 초고속 반도체설계입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 gmoon@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "문규 교수님 전화번호 알려줘",
    "output": "문규 교수님은 소프트웨어학부 명예교수님으로 분야는 초고속 반도체설계입니다.교수님의 연락처와 연구실 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 gmoon@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "문규 교수님 이메일 알려줘",
    "output": "문규 교수님은 소프트웨어학부 명예교수님으로 분야는 초고속 반도체설계입니다. 교수님의 이메일 주소는 gmoon@hallym.ac.kr입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소로 문의하실 수 있습니다."
  },
  {
    "input": "박섭형 교수님 연구실 어디야?",
    "output": "박섭형 교수님은 소프트웨어학부 교수님으로 분야는 머신러닝입니다. 교수님의 연구실은 공학관(1관) 1343호입니다. 연락처는 033-248-2342이며 이메일 주소는 spark@hallym.ac.kr입니다."
  },
  {
    "input": "박섭형 교수님 전화번호 알려줘",
    "output": "박섭형 교수님은 소프트웨어학부 교수님으로 분야는 머신러닝입니다. 교수님의 연락처는 033-248-2342입니다. 연구실은 공학관(1관) 1343호이며 이메일 주소는 spark@hallym.ac.kr입니다."
  },
  {
    "input": "박섭형 교수님 이메일 알려줘",
    "output": "박섭형 교수님은 소프트웨어학부 교수님으로 분야는 머신러닝입니다. 교수님의 이메일 주소는 spark@hallym.ac.kr입니다. 연락처는 033-248-2342이며 연구실은 공학관(1관) 1343호입니다."
  },
  {
    "input": "박찬영 교수님 연구실 어디야?",
    "output": "박찬영 교수님은 소프트웨어학부 교수님으로 분야는 통신 네트워크입니다. 교수님의 연구실은 공학관(1관) 1239호입니다. 연락처는 033-248-2321이며 이메일 주소는 cypark@hallym.ac.kr입니다."
  },
  {
    "input": "박찬영 교수님 전화번호 알려줘",
    "output": "박찬영 교수님은 소프트웨어학부 교수님으로 분야는 통신 네트워크입니다. 교수님의 연락처는 033-248-2321입니다. 연구실은 공학관(1관) 1239호이며 이메일 주소는 cypark@hallym.ac.kr입니다."
  },
  {
    "input": "박찬영 교수님 이메일 알려줘",
    "output": "박찬영 교수님은 소프트웨어학부 교수님으로 분야는 통신 네트워크입니다. 교수님의 이메일 주소는 cypark@hallym.ac.kr입니다. 연락처는 033-248-2321이며 연구실은 공학관(1관) 1239호입니다."
  },
  {
    "input": "방성근 교수님 연구실 어디야?",
    "output": "방성근 교수님은 소프트웨어학부 교수님으로 분야는 초고주파공학입니다. 교수님의 연구실은 공학관(1관) A1206호입니다. 연락처는 033-248-2349이며 이메일 주소는 baang@hallym.ac.kr입니다."
  },
  {
    "input": "방성근 교수님 전화번호 알려줘",
    "output": "방성근 교수님은 소프트웨어학부 교수님으로 분야는 초고주파공학입니다. 교수님의 연락처는 033-248-2349입니다. 연구실은 공학관(1관) A1206호이며 이메일 주소는 baang@hallym.ac.kr입니다."
  },
  {
    "input": "방성근 교수님 이메일 알려줘",
    "output": "방성근 교수님은 소프트웨어학부 교수님으로 분야는 초고주파공학입니다. 교수님의 이메일 주소는 baang@hallym.ac.kr입니다. 연락처는 033-248-2349이며 연구실은 공학관(1관) A1206호입니다."
  },
  {
    "input": "배무호 교수님 연구실 어디야?",
    "output": "배무호 교수님은 소프트웨어학부 교수님으로 분야는 신호처리, 회로시스템입니다. 교수님의 연구실은 공학관(1관) 1339호입니다. 연락처는 033-248-2350이며 이메일 주소는 mhbae@hallym.ac.kr입니다."
  },
  {
    "input": "배무호 교수님 전화번호 알려줘",
    "output": "배무호 교수님은 소프트웨어학부 교수님으로 분야는 신호처리, 회로시스템입니다. 교수님의 연락처는 033-248-2350입니다. 연구실은 공학관(1관) 1339호이며 이메일 주소는 mhbae@hallym.ac.kr입니다."
  },
  {
    "input": "배무호 교수님 이메일 알려줘",
    "output": "배무호 교수님은 소프트웨어학부 교수님으로 분야는 신호처리, 회로시스템입니다. 교수님의 이메일 주소는 mhbae@hallym.ac.kr입니다. 연락처는 033-248-2350이며 연구실은 공학관(1관) 1339호입니다."
  },
  {
    "input": "송성호 교수님 연구실 어디야?",
    "output": "송성호 교수님은 소프트웨어학부 교수님으로 분야는 자동제어 및 응용입니다. 교수님의 연구실은 공학관(1관) A1302호입니다. 연락처는 033-248-2346이며 이메일 주소는 ssh@hallym.ac.kr입니다."
  },
  {
    "input": "송성호 교수님 전화번호 알려줘",
    "output": "송성호 교수님은 소프트웨어학부 교수님으로 분야는 자동제어 및 응용입니다. 교수님의 연락처는 033-248-2346입니다. 연구실은 공학관(1관) 1339호이며 이메일 주소는 ssh@hallym.ac.kr입니다."
  },
  {
    "input": "송성호 교수님 이메일 알려줘",
    "output": "송성호 교수님은 소프트웨어학부 교수님으로 분야는 자동제어 및 응용입니다. 교수님의 이메일 주소는 ssh@hallym.ac.kr입니다. 연락처는 033-248-2346이며 연구실은 공학관(1관) 1339호입니다."
  },
  {
    "input": "송창근 교수님 연구실 어디야?",
    "output": "송창근 교수님은 소프트웨어학부 교수님으로 분야는 가상현실, 컴퓨터 그래픽스입니다. 교수님의 연구실은 공학관(1관) 1235호입니다. 연락처는 033-248-2317이며 이메일 주소는 cgsong@hallym.ac.kr입니다."
  },
  {
    "input": "송창근 교수님 전화번호 알려줘",
    "output": "송창근 교수님은 소프트웨어학부 교수님으로 분야는 가상현실, 컴퓨터 그래픽스입니다. 교수님의 연락처는 033-248-2317입니다. 연구실은 공학관(1관) 1235호이며 이메일 주소는 cgsong@hallym.ac.kr입니다."
  },
  {
    "input": "송창근 교수님 이메일 알려줘",
    "output": "송창근 교수님은 소프트웨어학부 부교수님으로 분야는 가상현실, 컴퓨터 그래픽스입니다. 교수님의 이메일 주소는 cgsong@hallym.ac.kr입니다. 연락처는 033-248-2317이며 연구실은 공학관(1관) 1235호입니다."
  },
  {
    "input": "신미영 교수님 연구실 어디야?",
    "output": "신미영 교수님은 소프트웨어학부 부교수님으로 분야는 컴퓨터구조, 멀티미디어입니다. 교수님의 연구실은 공학관(1관) A1303호입니다. 연락처는 033-248-2328이며 이메일 주소는 ireong@hallym.ac.kr입니다."
  },
  {
    "input": "신미영 교수님 전화번호 알려줘",
    "output": "신미영 교수님은 소프트웨어학부 부교수님으로 분야는 컴퓨터구조, 멀티미디어입니다. 교수님의 연락처는 033-248-2328입니다. 연구실은 공학관(1관) A1303호이며 이메일 주소는 ireong@hallym.ac.kr입니다."
  },
  {
    "input": "신미영 교수님 이메일 알려줘",
    "output": "신미영 교수님은 소프트웨어학부 부교수님으로 분야는 컴퓨터구조, 멀티미디어입니다. 교수님의 이메일 주소는 ireong@hallym.ac.kr입니다. 연락처는 033-248-2328이며 연구실은 공학관(1관) A1303호입니다."
  },
  {
    "input": "안재목 교수님 연구실 어디야?",
    "output": "안재목 교수님은 소프트웨어학부 교수님으로 분야는 생체계측 및 헬스케어시스템입니다. 교수님의 연구실은 공학관(1관) 1315호입니다. 연락처는 033-248-2347이며 이메일 주소는 ajm@hallym.ac.kr입니다."
  },
  {
    "input": "안재목 교수님 전화번호 알려줘",
    "output": "안재목 교수님은 소프트웨어학부 교수님으로 분야는 생체계측 및 헬스케어시스템입니다. 교수님의 연락처는 033-248-2347입니다. 연구실은 공학관(1관) 1315호이며 이메일 주소는 ajm@hallym.ac.kr입니다."
  },
  {
    "input": "안재목 교수님 이메일 알려줘",
    "output": "안재목 교수님은 소프트웨어학부 교수님으로 분야는 생체계측 및 헬스케어시스템입니다. 교수님의 이메일 주소는 ajm@hallym.ac.kr입니다. 연락처는 033-248-2347이며 연구실은 공학관(1관) 1315호입니다."
  },
  {
    "input": "양은샘 교수님 연구실 어디야?",
    "output": "양은샘 교수님은 소프트웨어학부 부교수님으로 분야는 정보통신입니다. 교수님의 연구실은 공학관(1관) 1204호입니다. 연락처는 033-248-2326이며 이메일 주소는 yanges@hallym.ac.kr입니다."
  },
  {
    "input": "양은샘 교수님 전화번호 알려줘",
    "output": "양은샘 교수님은 소프트웨어학부 부교수님으로 분야는 정보통신입니다. 교수님의 연락처는 033-248-2326입니다. 연구실은 공학관(1관) 1204호이며 이메일 주소는 yanges@hallym.ac.kr입니다."
  },
  {
    "input": "양은샘 교수님 이메일 알려줘",
    "output": "양은샘 교수님은 소프트웨어학부 부교수님으로 분야는 정보통신입니다. 교수님의 이메일 주소는 yanges@hallym.ac.kr입니다. 연락처는 033-248-2326이며 연구실은 공학관(1관) 1204호입니다."
  },
  {
    "input": "윤지희 교수님 연구실 어디야?",
    "output": "윤지희 교수님은 소프트웨어학부 교수님으로 분야는 데이터베이스입니다. 교수님의 연구실은 공학관(1관) A1307호입니다. 연락처는 033-248-2316이며 이메일 주소는 jhyoon@hallym.ac.kr입니다."
  },
  {
    "input": "윤지희 교수님 전화번호 알려줘",
    "output": "윤지희 교수님은 소프트웨어학부 교수님으로 분야는 데이터베이스입니다. 교수님의 연락처는 033-248-2316입니다. 연구실은 공학관(1관) A1307호이며 이메일 주소는 jhyoon@hallym.ac.kr입니다."
  },
  {
    "input": "윤지희 교수님 이메일 알려줘",
    "output": "윤지희 교수님은 소프트웨어학부 교수님으로 분야는 데이터베이스입니다. 교수님의 이메일 주소는 jhyoon@hallym.ac.kr입니다. 연락처는 033-248-2316이며 연구실은 공학관(1관) A1307호입니다."
  },
  {
    "input": "이선우 교수님 연구실 어디야?",
    "output": "이선우 교수님은 소프트웨어학부 교수님으로 분야는 임베디드시스템, HCI입니다. 교수님의 연구실은 공학관(1관) 1341호입니다. 연락처는 033-248-2348이며 이메일 주소는 senu@hallym.ac.kr입니다."
  },
  {
    "input": "이선우 교수님 전화번호 알려줘",
    "output": "이선우 교수님은 소프트웨어학부 교수님으로 분야는 임베디드시스템, HCI입니다. 교수님의 연락처는 033-248-2348입니다. 연구실은 공학관(1관) 1341호이며 이메일 주소는 senu@hallym.ac.kr입니다."
  },
  {
    "input": "이선우 교수님 이메일 알려줘",
    "output": "이선우 교수님은 소프트웨어학부 교수님으로 분야는 임베디드시스템, HCI입니다. 교수님의 이메일 주소는 senu@hallym.ac.kr입니다. 연락처는 033-248-2348이며 연구실은 공학관(1관) 1341호입니다."
  },
  {
    "input": "이용업 교수님 연구실 어디야?",
    "output": "이용업 교수님은 소프트웨어학부 교수님으로 분야는 유무선 통신시스템입니다. 교수님의 연구실은 공학관(1관) 1237호입니다. 연락처는 033-248-2344이며 이메일 주소는 yulee@hallym.ac.kr입니다."
  },
  {
    "input": "이용업 교수님 전화번호 알려줘",
    "output": "이용업 교수님은 소프트웨어학부 교수님으로 분야는 유무선 통신시스템입니다. 교수님의 연락처는 033-248-2344입니다. 연구실은 공학관(1관) 1237호이며 이메일 주소는 yulee@hallym.ac.kr입니다."
  },
  {
    "input": "이용업 교수님 이메일 알려줘",
    "output": "이용업 교수님은 소프트웨어학부 교수님으로 분야는 유무선 통신시스템입니다. 교수님의 이메일 주소는 yulee@hallym.ac.kr입니다. 연락처는 033-248-2344이며 연구실은 공학관(1관) 1237호입니다."
  },
  {
    "input": "이은주 교수님 연구실 어디야?",
    "output": "이은주 교수님은 소프트웨어학부 부교수님으로 분야는 물리전자, 의공학입니다. 교수님의 연구실은 공학관(1관) 1307호입니다. 연락처는 033-248-2354이며 이메일 주소는 ejlee@hallym.ac.kr입니다."
  },
  {
    "input": "이은주 교수님 전화번호 알려줘",
    "output": "이은주 교수님은 소프트웨어학부 부교수님으로 분야는 물리전자, 의공학입니다. 교수님의 연락처는 033-248-2354입니다. 연구실은 공학관(1관) 1307호이며 이메일 주소는 ejlee@hallym.ac.kr입니다."
  },
  {
    "input": "이은주 교수님 이메일 알려줘",
    "output": "이은주 교수님은 소프트웨어학부 부교수님으로 분야는 물리전자, 의공학입니다. 교수님의 이메일 주소는 ejlee@hallym.ac.kr입니다. 연락처는 033-248-2354이며 연구실은 공학관(1관) 1307호입니다."
  },
  {
    "input": "이재영 교수님 연구실 어디야?",
    "output": "이재영 교수님은 소프트웨어학부 명예교수님으로 분야는 컴퓨터 네트워크입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 jylee@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "이재영 교수님 전화번호 알려줘",
    "output": "이재영 교수님은 소프트웨어학부 명예교수님으로 분야는 컴퓨터 네트워크입니다. 교수님의 연락처와 연구실 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소 jylee@hallym.ac.kr로 문의하실 수 있습니다."
  },
  {
    "input": "이재영 교수님 이메일 알려줘",
    "output": "이재영 교수님은 소프트웨어학부 명예교수님으로 분야는 컴퓨터 네트워크입니다. 교수님의 이메일 주소는 jylee@hallym.ac.kr입니다. 교수님의 연구실과 연락처 정보는 제공되어 있지 않습니다. 문의가 필요하신 경우 교수님의 이메일 주소로 문의하실 수 있습니다."
  },
  {
    "input": "이정근 교수님 연구실 어디야?",
    "output": "이정근 교수님은 소프트웨어학부 명예교수님으로 분야는 Asynchronous Circuit & System Design입니다. 교수님의 연구실은 공학관(1관) A1201호입니다. 연락처는 033-248-2312이며 이메일 주소는 jeonggun.lee@hallym.ac.kr입니다."
  },
  {
    "input": "이정근 교수님 전화번호 알려줘",
    "output": "이정근 교수님은 소프트웨어학부 명예교수님으로 분야는 Asynchronous Circuit & System Design입니다. 교수님의 연락처는 033-248-2312입니다. 연구실은 공학관(1관) A1201호이며 이메일 주소는 jeonggun.lee@hallym.ac.kr입니다."
  },
  {
    "input": "이정근 교수님 이메일 알려줘",
    "output": "이정근 교수님은 소프트웨어학부 명예교수님으로 분야는 Asynchronous Circuit & System Design입니다. 교수님의 이메일 주소는 jeonggun.lee@hallym.ac.kr입니다. 연락처는 033-248-2312이며 연구실은 공학관(1관) A1201호입니다."
  },
  {
    "input": "주한규 교수님 연구실 어디야?",
    "output": "주한규 교수님은 소프트웨어학부 교수님으로 분야는 정보보호입니다. 교수님의 연구실은 공학관(1관) 1233호입니다. 연락처는 033-248-2322이며 이메일 주소는 hkjoo@hallym.ac.kr입니다."
  },
  {
    "input": "주한규 교수님 전화번호 알려줘",
    "output": "주한규 교수님은 소프트웨어학부 교수님으로 분야는 정보보호입니다. 교수님의 연락처는 033-248-2322입니다. 연구실은 공학관(1관) 1233호이며 이메일 주소는 hkjoo@hallym.ac.kr입니다."
  },
  {
    "input": "주한규 교수님 이메일 알려줘",
    "output": "주한규 교수님은 소프트웨어학부 교수님으로 분야는 정보보호입니다. 교수님의 이메일 주소는 hkjoo@hallym.ac.kr입니다. 연락처는 033-248-2322이며 연구실은 공학관(1관) 1233호입니다."
  },
  {
    "input": "노원종 교수님 연구실 어디야?",
    "output": "노원종 교수님은 소프트웨어학부 부교수님으로 분야는 이동통신 및 IoT입니다. 교수님의 연구실은 공학관(1관) 1346호입니다. 연락처는 033-248-2337이며 이메일 주소는 wonjong.noh@hallym.ac.kr입니다."
  },
  {
    "input": "노원종 교수님 전화번호 알려줘",
    "output": "노원종 교수님은 소프트웨어학부 부교수님으로 분야는 이동통신 및 IoT입니다. 교수님의 연락처는 033-248-2337입니다. 연구실은 공학관(1관) 1346호이며 이메일 주소는 wonjong.noh@hallym.ac.kr입니다."
  },
  {
    "input": "노원종 교수님 이메일 알려줘",
    "output": "노원종 교수님은 소프트웨어학부 부교수님으로 분야는 이동통신 및 IoT입니다. 교수님의 이메일 주소는 wonjong.noh@hallym.ac.kr입니다. 연락처는 033-248-2337이며 연구실은 공학관(1관) 1346호입니다."
  },
  {
    "input": "김진국 교수님 연구실 어디야?",
    "output": "김진국 교수님은 소프트웨어학부 부교수님으로 분야는 전자공학입니다. 교수님의 연구실은 공학관(1관) A1408호입니다. 연락처는 033-248-3352이며 이메일 주소는 smjkkim@hallym.ac.kr입니다."
  },
  {
    "input": "김진국 교수님 전화번호 알려줘",
    "output": "김진국 교수님은 소프트웨어학부 부교수님으로 분야는 전자공학입니다. 교수님의 연락처는 033-248-3352입니다. 연구실은 공학관(1관) A1408호이며 이메일 주소는 smjkkim@hallym.ac.kr입니다."
  },
  {
    "input": "김진국 교수님 이메일 알려줘",
    "output": "김진국 교수님은 소프트웨어학부 부교수님으로 분야는 전자공학입니다. 교수님의 이메일 주소는 smjkkim@hallym.ac.kr입니다. 연락처는 033-248-3352이며 연구실은 공학관(1관) A1408호입니다."
  },
  {
    "input": "김진환 교수님 연구실 어디야?",
    "output": "김진환 교수님은 소프트웨어학부 조교수님으로 분야는 분산처리, 프로그래밍입니다. 교수님의 연구실은 공학관(1관) 1201호입니다. 연락처는 033-248-2360이며 이메일 주소는 kim@hallym.ac.kr입니다."
  },
  {
    "input": "김진환 교수님 전화번호 알려줘",
    "output": "김진환 교수님은 소프트웨어학부 조교수님으로 분야는 분산처리, 프로그래밍입니다. 교수님의 연락처는 033-248-2360입니다. 연구실은 공학관(1관) 1201호이며 이메일 주소는 kim@hallym.ac.kr입니다."
  },
  {
    "input": "김진환 교수님 이메일 알려줘",
    "output": "김진환 교수님은 소프트웨어학부 조교수님으로 분야는 분산처리, 프로그래밍입니다. 교수님의 이메일 주소는 kim@hallym.ac.kr입니다. 연락처는 033-248-2360이며 연구실은 공학관(1관) 1201호입니다."
  },
  {
    "input": "임성훈 교수님 연구실 어디야?",
    "output": "임성훈 교수님은 소프트웨어학부 조교수님으로 분야는 정보이론, 인공지능, 무선통신입니다. 교수님의 연구실은 공학관(1관) 1242호입니다. 연락처는 033-248-2358이며 이메일 주소는 shlim@hallym.ac.kr입니다."
  },
  {
    "input": "임성훈 교수님 전화번호 알려줘",
    "output": "임성훈 교수님은 소프트웨어학부 조교수님으로 분야는 정보이론, 인공지능, 무선통신입니다. 교수님의 연락처는 033-248-2358입니다. 연구실은 공학관(1관) 1242호이며 이메일 주소는 shlim@hallym.ac.kr입니다."
  },
  {
    "input": "임성훈 교수님 이메일 알려줘",
    "output": "임성훈 교수님은 소프트웨어학부 조교수님으로 분야는 정보이론, 인공지능, 무선통신입니다. 교수님의 이메일 주소는 shlim@hallym.ac.kr입니다. 연락처는 033-248-2358이며 연구실은 공학관(1관) 1242호입니다."
  },
  {
    "input": "허종욱 교수님 연구실 어디야?",
    "output": "허종욱 교수님은 소프트웨어학부 조교수님으로 분야는 멀티미디어, 3D 프린팅입니다. 교수님의 연구실은 공학관(1관) A1301호입니다. 연락처는 033-248-2359이며 이메일 주소는 juhou@hallym.ac.kr입니다."
  },
  {
    "input": "허종욱 교수님 전화번호 알려줘",
    "output": "허종욱 교수님은 소프트웨어학부 조교수님으로 분야는 멀티미디어, 3D 프린팅입니다. 교수님의 연락처는 033-248-2359입니다. 연구실은 공학관(1관) A1301호이며 이메일 주소는 juhou@hallym.ac.kr입니다."
  },
  {
    "input": "허종욱 교수님 이메일 알려줘",
    "output": "허종욱 교수님은 소프트웨어학부 조교수님으로 분야는 멀티미디어, 3D 프린팅입니다. 교수님의 이메일 주소는 juhou@hallym.ac.kr입니다. 연락처는 033-248-2359이며 연구실은 공학관(1관) A1301호입니다."
  },
  {
    "input": "곽병일 교수님 연구실 어디야?",
    "output": "곽병일 교수님은 소프트웨어학부 조교수님으로 분야는 정보보호입니다. 교수님의 연구실은 공학관(1관) 1305호입니다. 연락처는 033-248-2339이며 이메일 주소는 kwacka12@hallym.ac.kr입니다."
  },
  {
    "input": "곽병일 교수님 전화번호 알려줘",
    "output": "곽병일 교수님은 소프트웨어학부 조교수님으로 분야는 정보보호입니다. 교수님의 연락처는 033-248-2339입니다. 연구실은 공학관(1관) 1305호이며 이메일 주소는 kwacka12@hallym.ac.kr입니다."
  },
  {
    "input": "곽병일 교수님 이메일 알려줘",
    "output": "곽병일 교수님은 소프트웨어학부 조교수님으로 분야는 정보보호입니다. 교수님의 이메일 주소는 kwacka12@hallym.ac.kr입니다. 연락처는 033-248-2339이며 연구실은 공학관(1관) 1305호입니다."
  },
  {
    "input": "박현제 교수님 연구실 어디야?",
    "output": "박현제 교수님은 소프트웨어학부 교수님으로 분야는 전산학입니다. 교수님의 연구실은 공학관(1관) 1342호입니다. 연락처는 033-248-2343이며 이메일 주소는 hjpark90@hallym.ac.kr입니다."
  },
  {
    "input": "박현제 교수님 전화번호 알려줘",
    "output": "박현제 교수님은 소프트웨어학부 교수님으로 분야는 전산학입니다. 교수님의 연락처는 033-248-2343입니다. 연구실은 공학관(1관) 1342호이며 이메일 주소는 hjpark90@hallym.ac.kr입니다."
  },
  {
    "input": "박현제 교수님 이메일 알려줘",
    "output": "박현제 교수님은 소프트웨어학부 교수님으로 분야는 전산학입니다. 교수님의 이메일 주소는 hjpark90@hallym.ac.kr입니다. 연락처는 033-248-2343이며 연구실은 공학관(1관) 1342호입니다."
  },
  {
    "input": "이종석 교수님 연구실 어디야?",
    "output": "이종석 교수님은 인공지능융합학부 교수님으로 분야는 전략경영입니다. 교수님의 연구실은 도헌글로벌스쿨 23217호입니다. 연락처는 033-248-3585이며 이메일 주소는 ljs1844@hallym.ac.kr입니다."
  },
  {
    "input": "이종석 교수님 전화번호 알려줘",
    "output": "이종석 교수님은 인공지능융합학부 교수님으로 분야는 전략경영입니다. 교수님의 연락처는 033-248-3585입니다. 연구실은 도헌글로벌스쿨 23217호이며 이메일 주소는 ljs1844@hallym.ac.kr입니다."
  },
  {
    "input": "이종석 교수님 이메일 알려줘",
    "output": "이종석 교수님은 인공지능융합학부 교수님으로 분야는 전략경영입니다. 교수님의 이메일 주소는 ljs1844@hallym.ac.kr입니다. 연락처는 033-248-3585이며 연구실은 도헌글로벌스쿨 23217호입니다."
  },
  {
    "input": "정태경 교수님 연구실 어디야?",
    "output": "정태경 교수님은 인공지능융합학부 교수님으로 분야는 전략경영입니다. 교수님의 연구실은 도헌글로벌스쿨 23219호입니다. 연락처는 033-248-3583이며 이메일 주소는 ttjeong@hallym.ac.kr입니다."
  },
  {
    "input": "정태경 교수님 전화번호 알려줘",
    "output": "정태경 교수님은 인공지능융합학부 교수님으로 분야는 전략경영입니다. 교수님의 연락처는 033-248-3583입니다. 연구실은 도헌글로벌스쿨 23219호이며 이메일 주소는 ttjeong@hallym.ac.kr입니다."
  },
  {
    "input": "정태경 교수님 이메일 알려줘",
    "output": "정태경 교수님은 인공지능융합학부 교수님으로 분야는 전략경영입니다. 교수님의 이메일 주소는 ttjeong@hallym.ac.kr입니다. 연락처는 033-248-3583이며 연구실은 도헌글로벌스쿨 23219호입니다."
  },
  {
    "input": "정인철 교수님 연구실 어디야?",
    "output": "정인철 교수님은 인공지능융합학부 부교수님으로 분야는 의공학입니다. 교수님의 연구실은 산학협력관 22511-2호, 도헌글로벌스쿨 23213호입니다. 연락처는 033-248-3581이며 이메일 주소는 incheol.jeong@hallym.ac.kr입니다."
  },
  {
    "input": "정인철 교수님 전화번호 알려줘",
    "output": "정인철 교수님은 인공지능융합학부 부교수님으로 분야는 의공학입니다. 교수님의 연락처는 033-248-3581입니다. 연구실은 산학협력관 22511-2호, 도헌글로벌스쿨 23213호이며 이메일 주소는 incheol.jeong@hallym.ac.kr입니다."
  },
  {
    "input": "정인철 교수님 이메일 알려줘",
    "output": "정인철 교수님은 인공지능융합학부 부교수님으로 분야는 의공학입니다. 교수님의 이메일 주소는 incheol.jeong@hallym.ac.kr입니다. 연락처는 033-248-3581이며 연구실은 산학협력관 22511-2호, 도헌글로벌스쿨 23213호입니다."
  },
  {
    "input": "원동옥 교수님 연구실 어디야?",
    "output": "원동옥 교수님은 인공지능융합학부 조교수님으로 분야는 인공지능, 인간-컴퓨터 상호작용입니다. 교수님의 연구실은 산학협력관 22511-1호, 도헌글로벌스쿨 23223호입니다. 연락처는 033-248-3582이며 이메일 주소는 dongok.won@hallym.ac.kr입니다."
  },
  {
    "input": "원동옥 교수님 전화번호 알려줘",
    "output": "원동옥 교수님은 인공지능융합학부 조교수님으로 분야는 인공지능, 인간-컴퓨터 상호작용입니다. 교수님의 연락처는 033-248-3582입니다. 연구실은 산학협력관 22511-1호, 도헌글로벌스쿨 23223호이며 이메일 주소는 dongok.won@hallym.ac.kr입니다."
  },
  {
    "input": "원동옥 교수님 이메일 알려줘",
    "output": "원동옥 교수님은 인공지능융합학부 조교수님으로 분야는 인공지능, 인간-컴퓨터 상호작용입니다. 교수님의 이메일 주소는 dongok.won@hallym.ac.kr입니다. 연락처는 033-248-3582이며 연구실은 산학협력관 22511-1호, 도헌글로벌스쿨 23223호입니다."
  },
  {
    "input": "류세민 교수님 연구실 어디야?",
    "output": "류세민 교수님은 인공지능융합학부 조교수님으로 분야는 인간-로봇 상호작용, 지능 로봇, 로봇 인터페이스, 유연 소자입니다. 교수님의 연구실은 도헌글로벌스쿨 23214호입니다. 연락처는 033-248-2336이며 이메일 주소는 sr@hallym.ac.kr입니다."
  },
  {
    "input": "류세민 교수님 전화번호 알려줘",
    "output": "류세민 교수님은 인공지능융합학부 조교수님으로 분야는 인간-로봇 상호작용, 지능 로봇, 로봇 인터페이스, 유연 소자입니다. 교수님의 연락처는 033-248-2336입니다. 연구실은 도헌글로벌스쿨 23214호이며 이메일 주소는 sr@hallym.ac.kr입니다."
  },
  {
    "input": "류세민 교수님 이메일 알려줘",
    "output": "류세민 교수님은 인공지능융합학부 조교수님으로 분야는 인간-로봇 상호작용, 지능 로봇, 로봇 인터페이스, 유연 소자입니다. 교수님의 이메일 주소는 sr@hallym.ac.kr입니다. 연락처는 033-248-2336이며 연구실은 도헌글로벌스쿨 23214호입니다."
  },
  {
    "input": "심진아 교수님 연구실 어디야?",
    "output": "심진아 교수님은 인공지능융합학부 조교수님으로 분야는 의료정보학, 시스템헬스관리입니다. 교수님의 연구실은 도헌글로벌스쿨 23221호입니다. 연락처는 033-248-3584이며 이메일 주소는 jin-ah.sim@hallym.ac.kr입니다."
  },
  {
    "input": "심진아 교수님 전화번호 알려줘",
    "output": "심진아 교수님은 인공지능융합학부 조교수님으로 분야는 의료정보학, 시스템헬스관리입니다. 교수님의 연락처는 033-248-3584입니다. 연구실은 도헌글로벌스쿨 23221호이며 이메일 주소는 jin-ah.sim@hallym.ac.kr입니다."
  },
  {
    "input": "심진아 교수님 이메일 알려줘",
    "output": "심진아 교수님은 인공지능융합학부 조교수님으로 분야는 의료정보학, 시스템헬스관리입니다. 교수님의 이메일 주소는 jin-ah.sim@hallym.ac.kr입니다. 연락처는 033-248-3584이며 연구실은 도헌글로벌스쿨 23221호입니다."
  }
"""

#3 강의실 정보
classroom_info = """
 {
        "input": "4506 어디야?",
        "output": "4506은(는) 건물번호 4번(인문 2관) 5층에 위치해있습니다."
    },
    {
        "input": "4427 어디야?",
        "output": "4427은(는) 건물번호 4번(인문 2관) 5층에 위치해있습니다."
    },
    {
        "input": "4508 어디야?",
        "output": "4508은(는) 건물번호 4번(인문 2관) 5층에 위치해있습니다."
    },
    {
        "input": "4437 어디야?",
        "output": "4437은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "4510 어디야?",
        "output": "4510은(는) 건물번호 4번(인문 2관) 5층에 위치해있습니다."
    },
    {
        "input": "4504 어디야?",
        "output": "4504은(는) 건물번호 4번(인문 2관) 5층에 위치해있습니다."
    },
    {
        "input": "4441 어디야?",
        "output": "4441은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "4433 어디야?",
        "output": "4433은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "4439 어디야?",
        "output": "4439은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "4435 어디야?",
        "output": "4435은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "4437 어디야?",
        "output": "4437은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "4439 어디야?",
        "output": "4429은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "4511 어디야?",
        "output": "4511은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "7113 어디야?",
        "output": "7113은(는) 건물번호 7번(자연과학관) 1층에 위치해있습니다."
    },
    {
        "input": "2424 어디야?",
        "output": "2424은(는) 건물번호 2번(대학본부.인문1관) 4층에 위치해있습니다."
    },
    {
        "input": "4427 어디야?",
        "output": "4427은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "4431 어디야?",
        "output": "4431은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "2427 어디야?",
        "output": "2427은(는) 건물번호 2번(대학본부.인문1관) 4층에 위치해있습니다."
    },
    {
        "input": "7111 어디야?",
        "output": "7111은(는) 건물번호 7번(자연과학관) 1층에 위치해있습니다."
    },
    {
        "input": "7101 어디야?",
        "output": "7101은(는) 건물번호 7번(자연과학관) 1층에 위치해있습니다."
    },
    {
        "input": "13201 어디야?",
        "output": "13201은(는) 건물번호 13번(사회 경영 2관) 2층에 위치해있습니다."
    },
    {
        "input": "16305 어디야?",
        "output": "16305은(는) 건물번호 16번(기초교육관) 3층에 위치해있습니다."
    },
    {
        "input": "10321 어디야?",
        "output": "10321은(는) 건물번호 10번(사회 경영 1관) 3층에 위치해있습니다."
    },
    {
        "input": "10108 어디야?",
        "output": "10108은(는) 건물번호 10번(사회 경영 1관) 1층에 위치해있습니다."
    },
    {
        "input": "10410-1 어디야?",
        "output": "10410-1은(는) 건물번호 10번(사회 경영 1관) 4층에 위치해있습니다."
    },
    {
        "input": "10522 어디야?",
        "output": "10522은(는) 건물번호 10번(사회 경영 1관) 5층에 위치해있습니다."
    },
    {
        "input": "16109 어디야?",
        "output": "16109은(는) 건물번호 16번(기초교육관) 1층에 위치해있습니다."
    },
    {
        "input": "10419 어디야?",
        "output": "10419은(는) 건물번호 10번(사회 경영 1관) 4층에 위치해있습니다."
    },
    {
        "input": "10414 어디야?",
        "output": "10414은(는) 건물번호 10번(사회 경영 1관) 4층에 위치해있습니다."
    },
    {
        "input": "10112 어디야?",
        "output": "10112은(는) 건물번호 10번(사회 경영 1관) 1층에 위치해있습니다."
    },
    {
        "input": "10416 어디야?",
        "output": "10416은(는) 건물번호 10번(사회 경영 1관) 4층에 위치해있습니다."
    },
    {
        "input": "10110 어디야?",
        "output": "10110은(는) 건물번호 10번(사회 경영 1관) 1층에 위치해있습니다."
    },
    {
        "input": "16303 어디야?",
        "output": "16303은(는) 건물번호 16번(기초교육관) 3층에 위치해있습니다."
    },
    {
        "input": "16307 어디야?",
        "output": "16307은(는) 건물번호 16번(기초교육관) 3층에 위치해있습니다."
    },
    {
        "input": "13101 어디야?",
        "output": "13101은(는) 건물번호 13번(사회 경영 2관) 1층에 위치해있습니다."
    },
    {
        "input": "10117 어디야?",
        "output": "10117은(는) 건물번호 10번(사회 경영 1관) 1층에 위치해있습니다."
    },
    {
        "input": "16313 어디야?",
        "output": "16313은(는) 건물번호 16번(기초교육관) 3층에 위치해있습니다."
    },
    {
        "input": "13401 어디야?",
        "output": "13401은(는) 건물번호 13번(사회 경영 2관) 4층에 위치해있습니다."
    },
    {
        "input": "HB116 어디야?",
        "output": "HB116은(는) 건물번호 31번(학생생활관 8관) 지하1층에(강당) 위치해있습니다."
    },
    {
        "input": "7105 어디야?",
        "output": "7105은(는) 건물번호 7번(자연과학관) 1층에 위치해있습니다."
    },
    {
        "input": "13501 어디야?",
        "output": "13501은(는) 건물번호 13번(사회 경영 2관) 5층에 위치해있습니다."
    },
    {
        "input": "9406 어디야?",
        "output": "9406은(는) 건물번호 9번(CLC(Campus life  Cennter)) 4층에 위치해있습니다."
    },
    {
        "input": "10115 어디야?",
        "output": "10115은(는) 건물번호 10번(사회 경영 1관) 1층에 위치해있습니다."
    },
    {
        "input": "10104 어디야?",
        "output": "10104은(는) 건물번호 10번(사회 경영 1관) 1층에 위치해있습니다."
    },
    {
        "input": "13215 어디야?",
        "output": "13215은(는) 건물번호 13번(사회 경영 2관) 2층에 위치해있습니다."
    },
    {
        "input": "15301 어디야?",
        "output": "15301은(는) 건물번호 15번(국제회의관) 3층에 위치해있습니다."
    },
    {
        "input": "7109 어디야?",
        "output": "7109은(는) 건물번호 7번(자연과학관) 1층에 위치해있습니다."
    },
    {
        "input": "16101 어디야?",
        "output": "16101은(는) 건물번호 16번(기초교육관) 1층에 위치해있습니다."
    },
    {
        "input": "10410-3 어디야?",
        "output": "10410-3은(는) 건물번호 10번(사회 경영 1관) 4층에 위치해있습니다."
    },
    {
        "input": "16207 어디야?",
        "output": "16207은(는) 건물번호 16번(기초교육관) 2층에 위치해있습니다."
    },
    {
        "input": "10102 어디야?",
        "output": "10102은(는) 건물번호 10번(사회 경영 1관) 1층에 위치해있습니다."
    },
    {
        "input": "1148 어디야?",
        "output": "1148은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "1362 어디야?",
        "output": "1362은(는) 건물번호 1번(공학관) 3층에 위치해있습니다."
    },
    {
        "input": "1423 어디야?",
        "output": "1423은(는) 건물번호 1번(공학관) 4층에 위치해있습니다."
    },
    {
        "input": "7114 어디야?",
        "output": "7114은(는) 건물번호 7번(자연과학관) 1층에 위치해있습니다."
    },
    {
        "input": "8217 어디야?",
        "output": "8217은(는) 건물번호 8번(생활과학관) 2층에 위치해있습니다."
    },
    {
        "input": "8109 어디야?",
        "output": "8109은(는) 건물번호 8번(생활과학관) 1층에 위치해있습니다."
    },
    {
        "input": "8320 어디야?",
        "output": "8320은(는) 건물번호 8번(생활과학관) 3층에 위치해있습니다."
    },
    {
        "input": "8116 어디야?",
        "output": "8116은(는) 건물번호 8번(생활과학관) 1층에 위치해있습니다."
    },
    {
        "input": "8205 어디야?",
        "output": "8205은(는) 건물번호 8번(생활과학관) 2층에 위치해있습니다."
    },
    {
        "input": "8433 어디야?",
        "output": "8433은(는) 건물번호 8번(생활과학관) 4층에 위치해있습니다."
    },
    {
        "input": "8102 어디야?",
        "output": "8102은(는) 건물번호 8번(생활과학관) 1층에 위치해있습니다."
    },
    {
        "input": "8103 어디야?",
        "output": "8103은(는) 건물번호 8번(생활과학관) 1층에 위치해있습니다."
    },
    {
        "input": "8216 어디야?",
        "output": "8216은(는) 건물번호 8번(생활과학관) 2층에 위치해있습니다."
    },
    {
        "input": "8101 어디야?",
        "output": "8101은(는) 건물번호 8번(생활과학관) 1층에 위치해있습니다."
    },
    {
        "input": "8111 어디야?",
        "output": "8111은(는) 건물번호 8번(생활과학관) 1층에 위치해있습니다."
    },
    {
        "input": "8212 어디야?",
        "output": "8212은(는) 건물번호 8번(생활과학관) 2층에 위치해있습니다."
    },
    {
        "input": "8210 어디야?",
        "output": "8210은(는) 건물번호 8번(생활과학관) 2층에 위치해있습니다."
    },
    {
        "input": "18122 어디야?",
        "output": "18122은(는) 건물번호 18번(한림레크리에이션센터) 1층에 위치해있습니다."
    },
    {
        "input": "18126 어디야?",
        "output": "18126은(는) 건물번호 18번(한림레크리에이션센터) 1층에 위치해있습니다."
    },
    {
        "input": "18201 어디야?",
        "output": "18201은(는) 건물번호 18번(한림레크리에이션센터) 2층에 위치해있습니다."
    },
    {
        "input": "7209 어디야?",
        "output": "7209은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "7217 어디야?",
        "output": "7217은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "7213 어디야?",
        "output": "7213은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "8428 어디야?",
        "output": "8428은(는) 건물번호 8번(생활과학관) 4층에 위치해있습니다."
    },
    {
        "input": "1102A 어디야?",
        "output": "1102A은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "8624 어디야?",
        "output": "8624은(는) 건물번호 8번(생활과학관) 6층에 위치해있습니다."
    },
    {
        "input": "21205 어디야?",
        "output": "21205은(는) 건물번호 21번(한림중개의과학연구원) 2층에 위치해있습니다."
    },
    {
        "input": "7211 어디야?",
        "output": "7211은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "7319 어디야?",
        "output": "7319은(는) 건물번호 7번(자연과학관) 3층에 위치해있습니다."
    },
    {
        "input": "7416 어디야?",
        "output": "7416은(는) 건물번호 7번(자연과학관) 4층에 위치해있습니다."
    },
    {
        "input": "7425 어디야?",
        "output": "7425은(는) 건물번호 7번(자연과학관) 4층에 위치해있습니다."
    },
    {
        "input": "7327 어디야?",
        "output": "7327은(는) 건물번호 7번(자연과학관) 3층에 위치해있습니다."
    },
    {
        "input": "7324 어디야?",
        "output": "7324은(는) 건물번호 7번(자연과학관) 3층에 위치해있습니다."
    },
    {
        "input": "7420 어디야?",
        "output": "7420은(는) 건물번호 7번(자연과학관) 4층에 위치해있습니다."
    },
    {
        "input": "7329 어디야?",
        "output": "7329은(는) 건물번호 7번(자연과학관) 3층에 위치해있습니다."
    },
    {
        "input": "7422 어디야?",
        "output": "7422은(는) 건물번호 7번(자연과학관) 4층에 위치해있습니다."
    },
    {
        "input": "7333 어디야?",
        "output": "7333은(는) 건물번호 7번(자연과학관) 3층에 위치해있습니다."
    },
    {
        "input": "14505 어디야?",
        "output": "14505은(는) 건물번호 14번(국제관) 5층에 위치해있습니다."
    },
    {
        "input": "14303 어디야?",
        "output": "14303은(는) 건물번호 14번(국제관) 3층에 위치해있습니다."
    },
    {
        "input": "14502 어디야?",
        "output": "14502은(는) 건물번호 14번(국제관) 5층에 위치해있습니다."
    },
    {
        "input": "14410 어디야?",
        "output": "14410은(는) 건물번호 14번(국제관) 4층에 위치해있습니다."
    },
    {
        "input": "14407 어디야?",
        "output": "14407은(는) 건물번호 14번(국제관) 4층에 위치해있습니다."
    },
    {
        "input": "14510 어디야?",
        "output": "14510은(는) 건물번호 14번(국제관) 5층에 위치해있습니다."
    },
    {
        "input": "14213 어디야?",
        "output": "14213은(는) 건물번호 14번(국제관) 2층에 위치해있습니다."
    },
    {
        "input": "7107 어디야?",
        "output": "7107은(는) 건물번호 7번(자연과학관) 1층에 위치해있습니다."
    },
    {
        "input": "5205 어디야?",
        "output": "5205은(는) 건물번호 5번(대학본부별관) 2층에 위치해있습니다."
    },
    {
        "input": "A1400 어디야?",
        "output": "A1400은(는) 건물번호 1번(공학관) 4층에 위치해있습니다."
    },
    {
        "input": "1101A 어디야?",
        "output": "1101A은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "1116-1 어디야?",
        "output": "1116-1은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "1145 어디야?",
        "output": "1145은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "1163 어디야?",
        "output": "1163은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "1157 어디야?",
        "output": "1157은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "1116-3 어디야?",
        "output": "1116-3은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "1111 어디야?",
        "output": "1111은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "1224 어디야?",
        "output": "1224은(는) 건물번호 1번(공학관) 2층에 위치해있습니다."
    },
    {
        "input": "1102 어디야?",
        "output": "1102은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "1226 어디야?",
        "output": "1226은(는) 건물번호 1번(공학관) 2층에 위치해있습니다."
    },
    {
        "input": "2519 어디야?",
        "output": "2519은(는) 건물번호 2번(대학본부 인문1관) 5층에 위치해있습니다."
    },
    {
        "input": "1320 어디야?",
        "output": "1320은(는) 건물번호 1번(공학관) 3층에 위치해있습니다."
    },
    {
        "input": "1221 어디야?",
        "output": "1221은(는) 건물번호 1번(공학관) 2층에 위치해있습니다."
    },
    {
        "input": "23203 어디야?",
        "output": "23203은(는) 건물번호 23번(도헌글로벌스쿨) 2층에 위치해있습니다."
    },
    {
        "input": "1125 어디야?",
        "output": "1125은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "HB102 어디야?",
        "output": "HB102은(는) 건물번호 31번(학생생활관 8관) 지하 1층에 위치해있습니다."
    },
    {
        "input": "7215 어디야?",
        "output": "7215은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "HB101 어디야?",
        "output": "HB101은(는) 건물번호 31번(학생생활관 8관) 지하 1층에 위치해있습니다."
    },
    {
        "input": "23201 어디야?",
        "output": "23201은(는) 건물번호 23번(도헌글로벌스쿨) 2층에 위치해있습니다."
    },
    {
        "input": "7533 어디야?",
        "output": "7533은(는) 건물번호 7번(자연과학관) 5층에 위치해있습니다."
    },
    {
        "input": "7530 어디야?",
        "output": "7530은(는) 건물번호 7번(자연과학관) 5층에 위치해있습니다."
    },
    {
        "input": "10318 어디야?",
        "output": "10318은(는) 건물번호 10번(사회 경영 1관) 3층에 위치해있습니다."
    },
    {
        "input": "7207 어디야?",
        "output": "7207은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "13301 어디야?",
        "output": "13301은(는) 건물번호 13번(사회 경영 2관) 3층에 위치해있습니다."
    },
    {
        "input": "16105 어디야?",
        "output": "16105은(는) 건물번호 16번(기초교육관) 1층에 위치해있습니다."
    },
    {
        "input": "16301 어디야?",
        "output": "16301은(는) 건물번호 16번(기초교육관) 3층에 위치해있습니다."
    },
    {
        "input": "4317 어디야?",
        "output": "4317은(는) 건물번호 4번(인문 2관) 3층에 위치해있습니다."
    },
    {
        "input": "10B107-2",
        "output": "10B107-2은(는) 건물번호 10번(사회 경영 1관) 지하 1층에 위치해있습니다."
    },
    {
        "input": "11B124 어디야?",
        "output": "11B124은(는) 건물번호 11번(일송아트홀) 지하 1층에 위치해있습니다."
    },
    {
        "input": "11B101 어디야?",
        "output": "11B101은(는) 건물번호 11번(일송아트홀) 지하 1층에 위치해있습니다."
    },
    {
        "input": "16309 어디야?",
        "output": "16309은(는) 건물번호 16번(기초교육관) 3층에 위치해있습니다."
    },
    {
        "input": "22308 어디야?",
        "output": "22308은(는) 건물번호 22번(산학협력관) 3층에 위치해있습니다."
    },
    {
        "input": "22307 어디야?",
        "output": "22307은(는) 건물번호 22번(산학협력관) 3층에 위치해있습니다."
    },
    {
        "input": "1254 어디야?",
        "output": "1254은(는) 건물번호 1번(공학관) 2층에 위치해있습니다."
    },
    {
        "input": "7221 어디야?",
        "output": "7221은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "22309 어디야?",
        "output": "22309은(는) 건물번호 22번(산학협력관) 3층에 위치해있습니다."
    },
    {
        "input": "7203 어디야?",
        "output": "7203은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "7103 어디야?",
        "output": "7103은(는) 건물번호 7번(자연과학관) 1층에 위치해있습니다."
    },
    {
        "input": "1152 어디야?",
        "output": "1152은(는) 건물번호 1번(공학관) 1층에 위치해있습니다."
    },
    {
        "input": "7603 어디야?",
        "output": "7603은(는) 건물번호 7번(자연과학관) 6층에 위치해있습니다."
    },
    {
        "input": "5403 어디야?",
        "output": "5403은(는) 건물번호 5번(대학본부별관) 4층에 위치해있습니다."
    },
    {
        "input": "5402 어디야?",
        "output": "5402은(는) 건물번호 5번(대학본부별관) 4층에 위치해있습니다."
    },
    {
        "input": "8426 어디야?",
        "output": "8426은(는) 건물번호 8번(생활과학관) 4층에 위치해있습니다."
    },
    {
        "input": "1262 어디야?",
        "output": "1262은(는) 건물번호 1번(공학관) 2층에 위치해있습니다."
    },
    {
        "input": "5406 어디야?",
        "output": "5406은(는) 건물번호 5번(대학본부별관) 4층에 위치해있습니다."
    },
    {
        "input": "5404 어디야?",
        "output": "5404은(는) 건물번호 5번(대학본부별관) 4층에 위치해있습니다."
    },
    {
        "input": "3506 어디야?",
        "output": "3506은(는) 건물번호 3번(의학관) 5층에 위치해있습니다."
    },
    {
        "input": "3204 어디야?",
        "output": "3204은(는) 건물번호 3번(의학관) 2층에 위치해있습니다."
    },
    {
        "input": "14108 어디야?",
        "output": "14108은(는) 건물번호 14번(국제관) 1층에 위치해있습니다."
    },
    {
        "input": "14309 어디야?",
        "output": "14309은(는) 건물번호 14번(국제관) 3층에 위치해있습니다."
    },
    {
        "input": "14307 어디야?",
        "output": "14307은(는) 건물번호 14번(국제관) 3층에 위치해있습니다."
    },
    {
        "input": "14205 어디야?",
        "output": "14205은(는) 건물번호 14번(국제관) 2층에 위치해있습니다."
    },
    {
        "input": "14209 어디야?",
        "output": "14209은(는) 건물번호 14번(국제관) 2층에 위치해있습니다."
    },
    {
        "input": "16211 어디야?",
        "output": "16211은(는) 건물번호 16번(기초교육관) 2층에 위치해있습니다."
    },
    {
        "input": "16203 어디야?",
        "output": "16203은(는) 건물번호 16번(기초교육관) 2층에 위치해있습니다."
    },
    {
        "input": "14203 어디야?",
        "output": "14203은(는) 건물번호 14번(국제관) 2층에 위치해있습니다."
    },
    {
        "input": "14201 어디야?",
        "output": "14201은(는) 건물번호 14번(국제관) 2층에 위치해있습니다."
    },
    {
        "input": "14415 어디야?",
        "output": "14415은(는) 건물번호 14번(국제관) 4층에 위치해있습니다."
    },
    {
        "input": "14106 어디야?",
        "output": "14106은(는) 건물번호 14번(국제관) 1층에 위치해있습니다."
    },
    {
        "input": "7205 어디야?",
        "output": "7205은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "16111 어디야?",
        "output": "16111은(는) 건물번호 16번(기초교육관) 1층에 위치해있습니다."
    },
    {
        "input": "7219 어디야?",
        "output": "7219은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "7201 어디야?",
        "output": "7201은(는) 건물번호 7번(자연과학관) 2층에 위치해있습니다."
    },
    {
        "input": "16209 어디야?",
        "output": "16209은(는) 건물번호 16번(기초교육관) 2층에 위치해있습니다."
    },
    {
        "input": "4210 어디야?",
        "output": "4210은(는) 건물번호 4번(인문 2관) 2층에 위치해있습니다."
    },
    {
        "input": "7112 어디야?",
        "output": "7112은(는) 건물번호 7번(자연과학관) 1층에 위치해있습니다."
    },
    {
        "input": "16201 어디야?",
        "output": "16201은(는) 건물번호 16번(기초교육관) 2층에 위치해있습니다."
    },
    {
        "input": "4406 어디야?",
        "output": "4406은(는) 건물번호 4번(인문 2관) 4층에 위치해있습니다."
    },
    {
        "input": "16311 어디야?",
        "output": "16311은(는) 건물번호 16번(기초교육관) 3층에 위치해있습니다."
    },
    {
        "input": "강의실 번호 읽는 방법 알려줘.",
        "output": "강의실 번호를 읽는 방법을 가르쳐드리겠습니다. 강의실 번호가 다섯 자리일 때는 앞에 숫자 두 자리는 건물번호이고, 세 번째 자리는 건물의 층수를 의미합니다. 강의실 번호가 네 자리일 때는 앞에 한자리가 건물번호이고, 두 번째 자리가 건물의 층수를 의미합니다."
    },
    {
        "input": "강의실 호수 어떻게 읽는지 알려줘.",
        "output": "강의실 번호를 읽는 방법을 가르쳐드리겠습니다. 강의실 번호가 다섯 자리일 때는 앞에 숫자 두 자리는 건물번호이고, 세 번째 자리는 건물의 층수를 의미합니다. 강의실 번호가 네 자리일 때는 앞에 한자리가 건물번호이고, 두 번째 자리가 건물의 층수를 의미합니다."
    },
    {
        "input": "강의실 위치 아는 방법 알려줘.",
        "output": "강의실 번호를 읽는 방법을 가르쳐드리겠습니다. 강의실 번호가 다섯 자리일 때는 앞에 숫자 두 자리는 건물번호이고, 세 번째 자리는 건물의 층수를 의미합니다. 강의실 번호가 네 자리일 때는 앞에 한자리가 건물번호이고, 두 번째 자리가 건물의 층수를 의미합니다."
    }
"""

#4. 전화번호 정보
phone_number = """
Updates to keyboard shortcuts … On Thursday, August 1, 2024, Drive keyboard shortcuts will be updated to give you first-letters navigation.Learn more
[
    {
        "input": "인문대학 전화번호 알려줘",
        "output": "인문대학 전화번호는 0332481500입니다."
    },
    {
        "input": "인문학부 전화번호 알려줘",
        "output": "인문학부 전화번호는 0332481500입니다."
    },
    {
        "input": "국어국문전공 전화번호 알려줘",
        "output": "국어국문전공 전화번호는 0332481510입니다."
    },
    {
        "input": "사학전공 전화번호 알려줘",
        "output": "사학전공 전화번호는 0332481570입니다."
    },
    {
        "input": "철학전공 전화번호 알려줘",
        "output": "철학전공 전화번호는 0332481550입니다."
    },
    {
        "input": "영어영문학과 전화번호 알려줘",
        "output": "영어영문학과 전화번호는 0332481530입니다."
    },
    {
        "input": "영문과 전화번호 알려줘",
        "output": "영어영문학과 전화번호는 0332481530입니다."
    },
    {
        "input": "중국학과 전화번호 알려줘",
        "output": "중국학과 전화번호는 0332481590입니다."
    },
    {
        "input": "일본학과 전화번호 알려줘",
        "output": "일본학과 전화번호는 0332481610입니다."
    },
    {
        "input": "러시아학과 전화번호 알려줘",
        "output": "러시아학과 전화번호는 0332481630입니다."
    },
    {
        "input": "글로컬융합인문학전공 전화번호 알려줘",
        "output": "글로컬융합인문학전공 전화번호는 0332481500입니다."
    },
    {
        "input": "사회과학대학 전화번호 알려줘",
        "output": "사회과학대학 전화번호는 0332481700입니다."
    },
    {
        "input": "심리학과 전화번호 알려줘",
        "output": "심리학과 전화번호는 0332481720입니다."
    },
    {
        "input": "사회학과 전화번호 알려줘",
        "output": "사회학과 전화번호는 0332481740입니다."
    },
    {
        "input": "사회복지학부 전화번호 알려줘",
        "output": "사회복지학부 전화번호는 0332481760입니다."
    },
    {
        "input": "사복 전화번호 알려줘",
        "output": "사회복지학부 전화번호는 0332481760입니다."
    },
    {
        "input": "정치행정학과 전화번호 알려줘",
        "output": "정치행정학과 전화번호는 0332481780입니다."
    },
    {
        "input": "광고홍보학과 전화번호 알려줘",
        "output": "광고홍보학과 전화번호는 0332481949입니다."
    },
    {
        "input": "광홍과 전화번호 알려줘",
        "output": "광고홍보학과 전화번호는 0332481949입니다."
    },
    {
        "input": "법학과 전화번호 알려줘",
        "output": "법학과 전화번호는 0332481950입니다."
    },
    {
        "input": "경제학과 전화번호 알려줘",
        "output": "경제학과 전화번호는 0332481810입니다."
    },
    {
        "input": "청소년학 전화번호 알려줘",
        "output": "청소년학 전화번호는 0332481769입니다."
    },
    {
        "input": "경영대학 전화번호 알려줘",
        "output": "경영대학 전화번호는 0332481800입니다."
    },
    {
        "input": "경영대 전화번호 알려줘",
        "output": "경영대학 전화번호는 0332481800입니다."
    },
    {
        "input": "경영학과 전화번호 알려줘",
        "output": "경영학과 전화번호는 0332481800입니다."
    },
    {
        "input": "경영 전화번호 알려줘",
        "output": "경영학과 전화번호는 0332481800입니다."
    },
    {
        "input": "금융재무학과 전화번호 알려줘",
        "output": "금융재무학과 전화번호는 0332481850입니다."
    },
    {
        "input": "스타트업비즈니스(융합)전공 전화번호 알려줘",
        "output": "스타트업비즈니스(융합)전공 전화번호는 0332481830입니다."
    },
    {
        "input": "자연과학대학 전화번호 알려줘",
        "output": "자연과학대학 전화번호는 0332482000입니다."
    },
    {
        "input": "자과대 전화번호 알려줘",
        "output": "자연과학대학 전화번호는 0332482000입니다."
    },
    {
        "input": "화학과 전화번호 알려줘",
        "output": "화학과 전화번호는 0332482090입니다."
    },
    {
        "input": "화학 전화번호 알려줘",
        "output": "화학과 전화번호는 0332482090입니다."
    },
    {
        "input": "화학전공 전화번호 알려줘",
        "output": "화학전공 전화번호는 0332482090입니다."
    },
    {
        "input": "생명과학과 전화번호 알려줘",
        "output": "생명과학과 전화번호는 0332482090입니다."
    },
    {
        "input": "바이오메디컬학과 전화번호 알려줘",
        "output": "바이오메디컬학과 전화번호는 0332482110입니다."
    },
    {
        "input": "바이오메디컬 전화번호 알려줘",
        "output": "바이오메디컬학과 전화번호는 0332482110입니다."
    },
    {
        "input": "식품영양학과 전화번호 알려줘",
        "output": "식품영양학과 전화번호는 0332482130입니다."
    },
    {
        "input": "식영과 전화번호 알려줘",
        "output": "식품영양학과 전화번호는 0332482130입니다."
    },
    {
        "input": "환경생명공학과 전화번호 알려줘",
        "output": "환경생명공학과 전화번호는 0332482150입니다."
    },
    {
        "input": "환생공 전화번호 알려줘",
        "output": "환경생명공학과 전화번호는 0332482150입니다."
    },
    {
        "input": "언어청각학부 전화번호 알려줘",
        "output": "언어청각학부 전화번호는 0332482210입니다."
    },
    {
        "input": "언어병리학 전공 전화번호 알려줘",
        "output": "언어병리학 전공 전화번호는 0332482210입니다."
    },
    {
        "input": "청각화 전공 전화번호 알려줘",
        "output": "청각화 전공 전화번호는 0332482210입니다."
    },
    {
        "input": "체육학과 전화번호 알려줘",
        "output": "체육학과 전화번호는 0332482250입니다."
    },
    {
        "input": "바이오헬스케어융합전공 전화번호 알려줘",
        "output": "바이오헬스케어융합전공 전화번호는 0332483107입니다."
    },
    {
        "input": "의과대학 전화번호 알려줘",
        "output": "의과대학 전화번호는 0332482501입니다."
    },
    {
        "input": "의예과 전화번호 알려줘",
        "output": "의예과 전화번호는 0332482501입니다."
    },
    {
        "input": "의학과 전화번호 알려줘",
        "output": "의학과 전화번호는 0332482501입니다."
    },
    {
        "input": "간호대학 전화번호 알려줘",
        "output": "간호대학 전화번호는 0332482710입니다."
    },
    {
        "input": "간호학과 전화번호 알려줘",
        "output": "간호학과 전화번호는 0332482710입니다."
    },
    {
        "input": "간호 전화번호 알려줘",
        "output": "간호학과 전화번호는 0332482710입니다."
    },
    {
        "input": "글로벌융합대학 전화번호 알려줘",
        "output": "글로벌융합대학 전화번호는 0332481883입니다."
    },
    {
        "input": "글로벌학부 전화번호 알려줘",
        "output": "글로벌학부 전화번호는 0332481880입니다."
    },
    {
        "input": "글로벌비즈니스전공 전화번호 알려줘",
        "output": "글로벌비즈니스전공 전화번호는 0332481880입니다."
    },
    {
        "input": "정보법과학전공 전화번호 알려줘",
        "output": "정보법과학전공 전화번호는 0332481880입니다."
    },
    {
        "input": "문화산업전공 전화번호 알려줘",
        "output": "문화산업전공 전화번호는 0332481880입니다."
    },
    {
        "input": "한국학전공 전화번호 알려줘",
        "output": "한국학전공 전화번호는 0332481880입니다."
    },
    {
        "input": "융합과학수사학과 전화번호 알려줘",
        "output": "융합과학수사학과 전화번호는 0332481980입니다."
    },
    {
        "input": "융합인재학부 전화번호 알려줘",
        "output": "융합인재학부 전화번호는 0332482481입니다."
    },
    {
        "input": "기후변화융합전공 전화번호 알려줘",
        "output": "기후변화융합전공 전화번호는 0332481880입니다."
    },
    {
        "input": "미디어스쿨 전화번호 알려줘",
        "output": "미디어스쿨 전화번호는 0332483552, 0332483553, 0332483554입니다."
    },
    {
        "input": "언론방송융합미디어전공 전화번호 알려줘",
        "output": "언론방송융합미디어전공 전화번호는 0332483552, 0332483553, 0332483554입니다."
    },
    {
        "input": "디지털미디어콘텐츠전공 전화번호 알려줘",
        "output": "디지털미디어콘텐츠전공 전화번호는 0332483552, 0332483553, 0332483554입니다."
    },
    {
        "input": "디미콘 전화번호 알려줘",
        "output": "디지털미디어콘텐츠전공 전화번호는 0332483552, 0332483553, 0332483554입니다."
    },
    {
        "input": "반도체디스플레이스쿨 전화번호 알려줘",
        "output": "반도체디스플레이스쿨 전화번호는 0332483555입니다."
    },
    {
        "input": "반도체전공 전화번호 알려줘",
        "output": "반도체전공 전화번호는 0332483555입니다."
    },
    {
        "input": "디스플레이전공 전화번호 알려줘",
        "output": "디스플레이전공 전화번호는 0332483555입니다."
    },
    {
        "input": "정보과학대학 전화번호 알려줘",
        "output": "정보과학대학 전화번호는 0332482301입니다."
    },
    {
        "input": "정과대 전화번호 알려줘",
        "output": "정보과학대학 전화번호는 0332482301입니다."
    },
    {
        "input": "소프트웨어학부 전화번호 알려줘",
        "output": "소프트웨어학부 전화번호는 0332482301입니다."
    },
    {
        "input": "빅데이터전공 전화번호 알려줘",
        "output": "빅데이터전공 전화번호는 0332482310입니다."
    },
    {
        "input": "빅데 전화번호 알려줘",
        "output": "빅데이터전공 전화번호는 0332482310입니다."
    },
    {
        "input": "콘텐츠IT전공 전화번호 알려줘",
        "output": "콘텐츠IT전공 전화번호는 0332482330입니다."
    },
    {
        "input": "콘텐츠 전화번호 알려줘",
        "output": "콘텐츠IT전공 전화번호는 0332482330입니다."
    },
    {
        "input": "스마트IoT전공 전화번호 알려줘",
        "output": "스마트IoT전공 전화번호는 0332482340입니다."
    },
    {
        "input": "스마트IOT전공 전화번호 알려줘",
        "output": "스마트IOT전공 전화번호는 0332482340입니다."
    },
    {
        "input": "인공지능융합학부 전화번호 알려줘",
        "output": "인공지능융합학부 전화번호는 0332482308, 0332482309입니다."
    },
    {
        "input": "AI의료융합전공 전화번호 알려줘",
        "output": "AI의료융합전공 전화번호는 0332482308, 0332482309입니다."
    },
    {
        "input": "AI로봇융합전공 전화번호 알려줘",
        "output": "AI로봇융합전공 전화번호는 0332482308, 0332482309입니다."
    },
    {
        "input": "AI기술경영융합전공 전화번호 알려줘",
        "output": "AI기술경영융합전공 전화번호는 0332482308, 0332482309입니다."
    },
    {
        "input": "데이터사이언스학부 전화번호 알려줘",
        "output": "데이터사이언스학부 전화번호는 0332482306, 0332482307입니다."
    },
    {
        "input": "데이터사이언스 전화번호 알려줘",
        "output": "데이터사이언스 전화번호는 0332482306, 0332482307입니다."
    },
    {
        "input": "데이터테크전공 전화번호 알려줘",
        "output": "데이터테크전공 전화번호는 0332482306, 0332482307입니다."
    },
    {
        "input": "임상의학통계전공 전화번호 알려줘",
        "output": "임상의학통계전공 전화번호는 0332482306, 0332482307입니다."
    },
    {
        "input": "디지털금융정보전공 전화번호 알려줘",
        "output": "디지털금융정보전공 전화번호는 0332482306, 0332482307입니다."
    },
    {
        "input": "미래융합스쿨 전화번호 알려줘",
        "output": "미래융합스쿨 전화번호는 0332483556, 0332483557입니다."
    },
    {
        "input": "미융스 전화번호 알려줘",
        "output": "미융스 전화번호는 0332483556, 0332483557입니다."
    },
    {
        "input": "디지털인문예술전공 전화번호 알려줘",
        "output": "디지털인문예술전공 전화번호는 0332483556, 0332483557입니다."
    },
    {
        "input": "글로벌협력전공 전화번호 알려줘",
        "output": "글로벌협력전공 전화번호는 0332483556, 0332483557입니다."
    },
    {
        "input": "융합관광경영전공 전화번호 알려줘",
        "output": "융합관광경영전공 전화번호는 0332483553, 0332483559입니다."
    },
    {
        "input": "의약신소재전공 전화번호 알려줘",
        "output": "의약신소재전공 전화번호는 0332483556, 0332483557입니다."
    },
    {
        "input": "융합신소재공학전공 전화번호 알려줘",
        "output": "융합신소재공학전공 전화번호는 0332481000입니다."
    },
    {
        "input": "마케팅커뮤니케이션테크놀로지융합전공 전화번호 알려줘",
        "output": "마케팅커뮤니케이션테크놀로지융합전공 전화번호는 0332483559입니다."
    },
    {
        "input": "공공인재융합전공 전화번호 알려줘",
        "output": "공공인재융합전공 전화번호는 0332483556, 0332483557입니다."
    },
    {
        "input": "사회혁신경영전공 전화번호 알려줘",
        "output": "사회혁신경영전공 전화번호는 0332483299입니다."
    },
    {
        "input": "건강돌봄디자인전공 전화번호 알려줘",
        "output": "건강돌봄디자인전공 전화번호는 0332483299입니다."
    },
    {
        "input": "일송자유교양대학 전화번호 알려줘",
        "output": "일송자유교양대학 전화번호는 0332482760입니다."
    },
    {
        "input": "자유교양전공 전화번호 알려줘",
        "output": "자유교양전공 전화번호는 0332482760입니다."
    },
    {
        "input": "자기설계융합전공 전화번호 알려줘",
        "output": "자기설계융합전공 전화번호는 0332481017입니다."
    },
    {
        "input": "실감미디어콘텐츠 전화번호 알려줘",
        "output": "실감미디어콘텐츠 전화번호는 0332481017입니다."
    },
    {
        "input": "출판기획및실무 전화번호 알려줘",
        "output": "출판기획및실무 전화번호는 0332481017입니다."
    },
    {
        "input": "청각인지헬스케어 전화번호 알려줘",
        "output": "콘텐츠IT-청각인지헬스케어 전화번호는 0332481017입니다."
    },
    {
        "input": "생명과학AI융합 전화번호 알려줘",
        "output": "생명과학AI융합 전화번호는 0332481017입니다."
    },
    {
        "input": "금융경영커뮤니케이션 전화번호 알려줘",
        "output": "금융경영커뮤니케이션 전화번호는 0332481017입니다."
    },
    {
        "input": "청각인지헬스케어규제과학 전화번호 알려줘",
        "output": "청각인지헬스케어규제과학 전화번호는 0332481017입니다."
    },
    {
        "input": "대학원 전화번호 알려줘",
        "output": "대학원 전화번호는 0332481401, 0332481402, 0332481403입니다."
    },
    {
        "input": "경영대학 교학팀 전화번호 알려줘",
        "output": "경영대학 교학팀 전화번호는 0332481800입니다."
    },
    {
        "input": "경영 교학팀 전화번호 알려줘",
        "output": "경영대학 교학팀 전화번호는 0332481800입니다."
    },
    {
        "input": "자연과학대학 교학팀 전화번호 알려줘",
        "output": "자연과학대학 교학팀 전화번호는 0332482001입니다."
    },
    {
        "input": "자과대 교학팀 전화번호 알려줘",
        "output": "자연과학대학 교학팀 전화번호는 0332482001입니다."
    },
    {
        "input": "의과대학 교학팀 전화번호 알려줘",
        "output": "의과대학 교학팀 전화번호는 0332482503입니다."
    },
    {
        "input": "의예과 교학팀 전화번호 알려줘",
        "output": "의예과 교학팀 전화번호는 0332482503입니다."
    },
    {
        "input": "간호대학 교학팀 전화번호 알려줘",
        "output": "간호대학 교학팀 전화번호는 0332482710입니다."
    },
    {
        "input": "간호 교학팀 전화번호 알려줘",
        "output": "간호대학 교학팀 전화번호는 0332482710입니다."
    },
    {
        "input": "글로벌융합대학 교학팀 전화번호 알려줘",
        "output": "글로벌융합대학 교학팀 전화번호는 0332482481입니다."
    },
    {
        "input": "미디어스쿨 교학팀 전화번호 알려줘",
        "output": "미디어스쿨 교학팀 전화번호는 0332483552입니다."
    },
    {
        "input": "통합스쿨 교학팀 전화번호 알려줘",
        "output": "통합스쿨 교학팀 전화번호는 0332483550입니다."
    },
    {
        "input": "나노융합스쿨 교학팀 전화번호 알려줘",
        "output": "나노융합스쿨 교학팀 전화번호는 0332483555입니다."
    },
    {
        "input": "정보과학대학 교학팀 전화번호 알려줘",
        "output": "정보과학대학 교학팀 전화번호는 0332482301입니다."
    },
    {
        "input": "정과대 교학팀 전화번호 알려줘",
        "output": "정보과학대학 교학팀 전화번호는 0332482301입니다."
    },
    {
        "input": "미래융합스쿨 교학팀 전화번호 알려줘",
        "output": "미래융합스쿨 교학팀 전화번호는 0332483556입니다."
    },
    {
        "input": "미융스 교학팀 전화번호 알려줘",
        "output": "미래융합스쿨 교학팀 전화번호는 0332483556입니다."
    },
    {
        "input": "일송자유교양대학 교학팀 전화번호 알려줘",
        "output": "일송자유교양대학 교학팀 전화번호는 0332482760입니다."
    },
    {
        "input": "수강신청 전화번호 알려줘",
        "output": "수강신청 전화번호는 0332481016입니다."
    },
    {
        "input": "수강신청 변경 전화번호 알려줘",
        "output": "수강신청 변경 전화번호는 0332481016입니다."
    },
    {
        "input": "수강신청 철회 전화번호 알려줘",
        "output": "수강신청 철회 전화번호는 0332481016입니다."
    },
    {
        "input": "대체재이수 전화번호 알려줘",
        "output": "대체재이수 전화번호는 0332481016입니다."
    },
    {
        "input": "성적 전화번호 알려줘",
        "output": "성적 전화번호는 0332481017입니다."
    },
    {
        "input": "강의평가 전화번호 알려줘",
        "output": "강의평가 전화번호는 0332481017입니다."
    },
    {
        "input": "시험 전화번호 알려줘",
        "output": "시험 전화번호는 0332481017입니다."
    },
    {
        "input": "교류학점 인정 전화번호 알려줘",
        "output": "교류학점 인정 전화번호는 0332481017이야."
    },
    {
        "input": "소속변경 전화번호 알려줘",
        "output": "소속변경 전화번호는 0332481014입니다."
    },
    {
        "input": "복수전공 전화번호 알려줘",
        "output": "복수전공 전화번호는 0332481014입니다."
    },
    {
        "input": "전공배정 전화번호 알려줘",
        "output": "전공배정 전화번호는 0332481014입니다."
    },
    {
        "input": "편입학학점인정 전화번호 알려줘",
        "output": "편입학학점인정 전화번호는 0332481014입니다."
    },
    {
        "input": "학적 전화번호 알려줘",
        "output": "학적 전화번호는 0332481015입니다."
    },
    {
        "input": "졸업 전화번호 알려줘",
        "output": "졸업 전화번호는 0332481015입니다."
    },
    {
        "input": "전자출결 전화번호 알려줘",
        "output": "전자출결 전화번호는 0332481013입니다."
    },
    {
        "input": "계절학기 전화번호 알려줘",
        "output": "계절학기 문의 전화번호는 0332481013입니다."
    },
    {
        "input": "자유교양전공 전화번호 알려줘",
        "output": "자유교양전공 전화번호는 0332482756입니다."
    },
    {
        "input": "교직 전화번호 알려줘",
        "output": "교직 전화번호는 0332482759입니다."
    },
    {
        "input": "일반교양 전화번호 알려줘",
        "output": "일반교양 전화번호는 0332482752입니다."
    },
    {
        "input": "영어 교과목 전화번호 알려줘",
        "output": "영어 교과목 전화번호는 0332482972입니다."
    },
    {
        "input": "사고와표현 전화번호 알려줘",
        "output": "사고와표현 전화번호는 0332482755입니다."
    },
    {
        "input": "체육과목 전화번호 알려줘",
        "output": "체육과목 전화번호는 0332482263입니다."
    },
    {
        "input": "컴퓨터과목 전화번호 알려줘",
        "output": "컴퓨터과목 전화번호는 0332482753입니다."
    },
    {
        "input": "학생증 발급 전화번호 알려줘",
        "output": "학생증 발급 전화번호는 0332481071입니다."
    },
    {
        "input": "증명서 발급 전화번호 알려줘",
        "output": "증명서 발급 전화번호는 0332481071입니다."
    },
    {
        "input": "학적변동 전화번호 알려줘",
        "output": "학적변동 전화번호는 0332481071입니다."
    },
    {
        "input": "군휴학 전화번호 알려줘",
        "output": "군휴학 전화번호는 0332481071입니다."
    },
    {
        "input": "일반휴학 전화번호 알려줘",
        "output": "일반휴학 전화번호는 0332481071입니다."
    },
    {
        "input": "공결 전화번호 알려줘",
        "output": "공결 전화번호는 0332481071입니다."
    },
    {
        "input": "공결 문의 전화번호 알려줘",
        "output": "공결 문의 전화번호는 0332481071입니다."
    },
    {
        "input": "학생상해보험 전화번호 알려줘",
        "output": "학생상해보험 전화번호는 0332481071입니다."
    },
    {
        "input": "국가장학금 전화번호 알려줘",
        "output": "국가장학금 전화번호는 0332481062입니다."
    },
    {
        "input": "외부장학금 전화번호 알려줘",
        "output": "외부장학금 전화번호는 0332481062입니다."
    },
    {
        "input": "학자금대출 전화번호 알려줘",
        "output": "학자금대출 전화번호는 0332481062입니다."
    },
    {
        "input": "교내장학금 전화번호 알려줘",
        "output": "교내장학금 전화번호는 0332481063입니다."
    },
    {
        "input": "장학상담 전화번호 알려줘",
        "output": "장학상담 전화번호는 0332481063입니다."
    },
    {
        "input": "학생회 전화번호 알려줘",
        "output": "학생회 전화번호는 0332481064입니다."
    },
    {
        "input": "학생활동 전화번호 알려줘",
        "output": "학생활동 전화번호는 0332481064입니다."
    },
    {
        "input": "학생행사 전화번호 알려줘",
        "output": "학생행사 전화번호는 0332481064입니다."
    },
    {
        "input": "동아리 전화번호 알려줘",
        "output": "동아리 전화번호는 0332481064입니다."
    },
    {
        "input": "근로장학 전화번호 알려줘",
        "output": "근로장학 전화번호는 0332481065입니다."
    },
    {
        "input": "장애학생지원 전화번호 알려줘",
        "output": "장애학생지원 전화번호는 0332481065입니다."
    },
    {
        "input": "취업통계조사 전화번호 알려줘",
        "output": "취업통계조사 전화번호는0332481082입니다."
    },
    {
        "input": "취업멘토링 전화번호 알려줘",
        "output": "취업멘토링 전화번호는 0332481082입니다."
    },
    {
        "input": "학과 연계 진로취업 교육 프로그램 전화번호 알려줘",
        "output": "학과 연계 진로취업 교육 프로그램 전화번호는 0332481081입니다."
    },
    {
        "input": "자기주도 활동 관리 전화번호 알려줘",
        "output": "자기주도 활동 관리 전화번호는 0332481081입니다."
    },
    {
        "input": "진로취업 비교과 프로그램 전화번호 알려줘",
        "output": "진로취업 비교과 프로그램 전화번호는 0332481083입니다."
    },
    {
        "input": "표준현장실습 학기제(SW) 전화번호 알려줘",
        "output": "표준현장실습 학기제(SW) 전화번호는 0332481086입니다."
    },
    {
        "input": "표준현장실습 학기제(LINC) 전화번호 알려줘",
        "output": "표준현장실습 학기제(LINC) 전화번호는 0332481084, 0332481087입니다."
    },
    {
        "input": "청년고용정책 연계 프로그램 전화번호 알려줘",
        "output": "청년고용정책 연계 프로그램 전화번호는 0332481085입니다."
    },
    {
        "input": "진로취업 상담 전화번호 알려줘",
        "output": "진로취업 상담 전화번호는 0332481088입니다."
    },
    {
        "input": "진로취업 지원 전화번호 알려줘",
        "output": "진로취업 지원 전화번호는 0332481088입니다."
    },
    {
        "input": "진로취업 추천채용 전화번호 알려줘",
        "output": "진로취업 추천채용 전화번호는 0332481088입니다."
    },
    {
        "input": "기숙사 전화번호 알려줘",
        "output": "기숙사 전화번호는 0332483601입니다."
    },
    {
        "input": "학생생활관 전화번호 알려줘",
        "output": "학생생활관 전화번호는 0332483601입니다."
    }
]
"""

# 5. 메일 정보
hallym_mail = """
[
    {
        "input": "인문대학 이메일 알려줘",
        "output": "인문대학의 이메일 주소는 de1500@hallym.ac.kr입니다."
    },
    {
        "input": "인문학부 이메일 알려줘",
        "output": "인문학부의 이메일 주소는 de1500@hallym.ac.kr입니다."
    },
    {
        "input": "국어국문전공 이메일 알려줘",
        "output": "국어국문전공의 이메일 주소는 de1510@hallym.ac.kr입니다."
    },
    {
        "input": "국어국문 이메일 알려줘",
        "output": "국어국문전공의 이메일 주소는 de1510@hallym.ac.kr입니다."
    },
    {
        "input": "사학전공 이메일 알려줘",
        "output": "사학전공의 이메일 주소는 de1570@hallym.ac.kr입니다."
    },
    {
        "input": "사학과 이메일 알려줘",
        "output": "사학과의 이메일 주소는 de1570@hallym.ac.kr입니다."
    },
    {
        "input": "철학전공 이메일 알려줘",
        "output": "철학전공의 이메일 주소는 de1550@hallym.ac.kr입니다."
    },
    {
        "input": "철학과 이메일 알려줘",
        "output": "철학과의 이메일 주소는 de1550@hallym.ac.kr입니다."
    },
    {
        "input": "영어영문학과 이메일 알려줘",
        "output": "영어영문학과의 이메일 주소는 de1530@hallym.ac.kr입니다."
    },
    {
        "input": "영문과 이메일 알려줘",
        "output": "영어영문학과의 이메일 주소는 de1530@hallym.ac.kr입니다."
    },
    {
        "input": "중국학과 이메일 알려줘",
        "output": "중국학과의 이메일 주소는 de1590@hallym.ac.kr입니다."
    },
    {
        "input": "일본학과 이메일 알려줘",
        "output": "일본학과의 이메일 주소는 de1610@hallym.ac.kr입니다."
    },
    {
        "input": "러시아학과 이메일 알려줘",
        "output": "러시아학과의 이메일 주소는 de1630@hallym.ac.kr입니다."
    },
    {
        "input": "러시아전공 이메일 알려줘",
        "output": "러시아학과의 이메일 주소는 de1630@hallym.ac.kr입니다."
    },
    {
        "input": "글로컬융합인문학전공 이메일 알려줘",
        "output": "글로컬융합인문학전공의 이메일 주소는 de1500@hallym.ac.kr입니다."
    },
    {
        "input": "사회과학대학 이메일 알려줘",
        "output": "사회과학대학의 이메일 주소는 de1700@hallym.ac.kr입니다."
    },
    {
        "input": "사과대 이메일 알려줘",
        "output": "사회과학대학의 이메일 주소는 de1700@hallym.ac.kr입니다."
    },
    {
        "input": "심리학과 이메일 알려줘",
        "output": "심리학과의 이메일 주소는 de1720@hallym.ac.kr입니다."
    },
    {
        "input": "심리 이메일 알려줘",
        "output": "심리학과의 이메일 주소는 de1720@hallym.ac.kr입니다."
    },
    {
        "input": "사회학과 이메일 알려줘",
        "output": "사회학과의 이메일 주소는 de1740@hallym.ac.kr입니다."
    },
    {
        "input": "사회복지학부 이메일 알려줘",
        "output": "사회복지학부의 이메일 주소는 de1760@hallym.ac.kr입니다."
    },
    {
        "input": "사복 이메일 알려줘",
        "output": "사회복지학부의 이메일 주소는 de1760@hallym.ac.kr입니다."
    },
    {
        "input": "정치행정학과 이메일 알려줘",
        "output": "정치행정학과의 이메일 주소는 de1780@hallym.ac.kr입니다."
    },
    {
        "input": "정행 이메일 알려줘",
        "output": "정치행정학과의 이메일 주소는 de1780@hallym.ac.kr입니다."
    },
    {
        "input": "광고홍보학과 이메일 알려줘",
        "output": "광고홍보학과의 이메일 주소는 de1949@hallym.ac.kr입니다."
    },
    {
        "input": "광홍 이메일 알려줘",
        "output": "광고홍보학과의 이메일 주소는 de1949@hallym.ac.kr입니다."
    },
    {
        "input": "법학과 이메일 알려줘",
        "output": "법학과의 이메일 주소는 de1950@hallym.ac.kr입니다."
    },
    {
        "input": "법학 이메일 알려줘",
        "output": "법학과의 이메일 주소는 de1950@hallym.ac.kr입니다."
    },
    {
        "input": "경제학과 이메일 알려줘",
        "output": "경제학과의 이메일 주소는 de1810@hallym.ac.kr입니다."
    },
    {
        "input": "청소년학 이메일 알려줘",
        "output": "청소년학의 이메일 주소는 de1760@hallym.ac.kr입니다."
    },
    {
        "input": "경영대학 이메일 알려줘",
        "output": "경영대학의 이메일 주소는 de1800@hallym.ac.kr입니다."
    },
    {
        "input": "경영학과 이메일 알려줘",
        "output": "경영학과의 이메일 주소는 de1830@hallym.ac.kr입니다."
    },
    {
        "input": "금융재무학과 이메일 알려줘",
        "output": "금융재무학과의 이메일 주소는 de1850@hallym.ac.kr입니다."
    },
    {
        "input": "스타트업비즈니스(융합)전공 이메일 알려줘",
        "output": "스타트업비즈니스(융합)전공의 이메일 주소는 de1830@hallym.ac.kr입니다."
    },
    {
        "input": "스타트업비즈니스전공 이메일 알려줘",
        "output": "스타트업비즈니스(융합)전공의 이메일 주소는 de1830@hallym.ac.kr입니다."
    },
    {
        "input": "자연과학대학 이메일 알려줘",
        "output": "자연과학대학의 이메일 주소는 de2000@hallym.ac.kr입니다."
    },
    {
        "input": "자과대 이메일 알려줘",
        "output": "자연과학대학의 이메일 주소는 de2000@hallym.ac.kr입니다."
    },
    {
        "input": "화학과 이메일 알려줘",
        "output": "화학과의 이메일 주소는 de2070@hallym.ac.kr입니다."
    },
    {
        "input": "화학전공 이메일 알려줘",
        "output": "화학과의 이메일 주소는 de2070@hallym.ac.kr입니다."
    },
    {
        "input": "생명과학과 이메일 알려줘",
        "output": "생명과학과의 이메일 주소는 de2090@hallym.ac.kr입니다."
    },
    {
        "input": "바이오메디컬학과 이메일 알려줘",
        "output": "바이오메디컬학과의 이메일 주소는 de2110@hallym.ac.kr입니다."
    },
    {
        "input": "바이오메디컬 이메일 알려줘",
        "output": "바이오메디컬학과의 이메일 주소는 de2110@hallym.ac.kr입니다."
    },
    {
        "input": "식품영양학과 이메일 알려줘",
        "output": "식품영양학과의 이메일 주소는 de2130@hallym.ac.kr입니다."
    },
    {
        "input": "식영과 이메일 알려줘",
        "output": "식품영양학과의 이메일 주소는 de2130@hallym.ac.kr입니다."
    },
    {
        "input": "환경생명공학과 이메일 알려줘",
        "output": "환경생명공학과의 이메일 주소는 de2150@hallym.ac.kr입니다."
    },
    {
        "input": "환생공 이메일 알려줘",
        "output": "환경생명공학과의 이메일 주소는 de2150@hallym.ac.kr입니다."
    },
    {
        "input": "언어청각학부 이메일 알려줘",
        "output": "언어청각학부의 이메일 주소는 de2210@hallym.ac.kr입니다."
    },
    {
        "input": "언청 이메일 알려줘",
        "output": "언어청각학부의 이메일 주소는 de2210@hallym.ac.kr입니다."
    },
    {
        "input": "언어병리학 전공 이메일 알려줘",
        "output": "언어병리학전공의 이메일 주소는 de2210@hallym.ac.kr입니다."
    },
    {
        "input": "언병 이메일 알려줘",
        "output": "언어병리학전공의 이메일 주소는 de2210@hallym.ac.kr입니다."
    },
    {
        "input": "청각화 전공 이메일 알려줘",
        "output": "청각화전공의 이메일 주소는 de2210@hallym.ac.kr입니다."
    },
    {
        "input": "체육학과 이메일 알려줘",
        "output": "체육학과의 이메일 주소는 de2250@hallym.ac.kr입니다."
    },
    {
        "input": "바이오헬스케어융합전공 이메일 알려줘",
        "output": "바이오헬스케어융합전공의 이메일 주소는 de2020@hallym.ac.kr입니다."
    },
    {
        "input": "의과대학 이메일 알려줘",
        "output": "의과대학의 이메일 주소는 de2500@hallym.ac.kr입니다."
    },
    {
        "input": "의예과 이메일 알려줘",
        "output": "의예과의 이메일 주소는 de2500@hallym.ac.kr입니다."
    },
    {
        "input": "의학과 이메일 알려줘",
        "output": "의학과의 이메일 주소는 de2500@hallym.ac.kr입니다."
    },
    {
        "input": "간호대학 이메일 알려줘",
        "output": "간호대학의 이메일 주소는 de2710@hallym.ac.kr입니다."
    },
    {
        "input": "간호학과 이메일 알려줘",
        "output": "간호학과의 이메일 주소는 de2710@hallym.ac.kr입니다."
    },
    {
        "input": "간호 이메일 알려줘",
        "output": "간호학과의 이메일 주소는 de2710@hallym.ac.kr입니다."
    },
    {
        "input": "글로벌융합대학 이메일 알려줘",
        "output": "글로벌융합대학의 이메일 주소는 de2480@hallym.ac.kr입니다."
    },
    {
        "input": "글로벌학부 이메일 알려줘",
        "output": "글로벌학부의 이메일 주소는 de1880@hallym.ac.kr입니다."
    },
    {
        "input": "글로벌비즈니스전공 이메일 알려줘",
        "output": "글로벌비즈니스전공의 이메일 주소는 de1880@hallym.ac.kr입니다."
    },
    {
        "input": "정보법과학전공 이메일 알려줘",
        "output": "정보법과학전공의 이메일 주소는 de1880@hallym.ac.kr입니다."
    },
    {
        "input": "문화산업전공 이메일 알려줘",
        "output": "문화산업전공의 이메일 주소는 de1880@hallym.ac.kr입니다."
    },
    {
        "input": "한국학전공 이메일 알려줘",
        "output": "한국학전공의 이메일 주소는 de1880@hallym.ac.kr입니다."
    },
    {
        "input": "융합과학수사학과 이메일 알려줘",
        "output": "융합과학수사학과의 이메일 주소는 de1980@hallym.ac.kr입니다."
    },
    {
        "input": "융합인재학부 이메일 알려줘",
        "output": "융합인재학부의 이메일 주소는 de2480@hallym.ac.kr입니다."
    },
    {
        "input": "기후변화융합전공 이메일 알려줘",
        "output": "기후변화융합전공의 이메일 주소는 de2480@hallym.ac.kr입니다."
    },
    {
        "input": "미디어스쿨 이메일 알려줘",
        "output": "미디어스쿨의 이메일 주소는 de1910@hallym.ac.kr입니다."
    },
    {
        "input": "언론방송융합미디어전공 이메일 알려줘",
        "output": "언론방송융합미디어전공의 이메일 주소는 de1910@hallym.ac.kr입니다."
    },
    {
        "input": "디지털미디어콘텐츠전공 이메일 알려줘",
        "output": "디지털미디어콘텐츠전공의 이메일 주소는 de1910@hallym.ac.kr입니다."
    },
    {
        "input": "디미콘 이메일 알려줘",
        "output": "디지털미디어콘텐츠전공의 이메일 주소는 de1910@hallym.ac.kr입니다."
    },
    {
        "input": "반도체디스플레이스쿨 이메일 알려줘",
        "output": "반도체디스플레이스쿨의 이메일 주소는 de2050@hallym.ac.kr입니다."
    },
    {
        "input": "반도체전공 이메일 알려줘",
        "output": "반도체전공의 이메일 주소는 de2050@hallym.ac.kr입니다."
    },
    {
        "input": "디스플레이전공 이메일 알려줘",
        "output": "디스플레이전공의 이메일 주소는 de2050@hallym.ac.kr입니다."
    },
    {
        "input": "정보과학대학 이메일 알려줘",
        "output": "정보과학대학의 이메일 주소는 de2300@hallym.ac.kr입니다."
    },
    {
        "input": "소프트웨어학부 이메일 알려줘",
        "output": "소프트웨어학부의 이메일 주소는 de2300@hallym.ac.kr입니다."
    },
    {
        "input": "빅데이터전공 이메일 알려줘",
        "output": "빅데이터전공의 이메일 주소는 de2310@hallym.ac.kr입니다."
    },
    {
        "input": "빅데 이메일 알려줘",
        "output": "빅데이터전공의 이메일 주소는 de2310@hallym.ac.kr입니다."
    },
    {
        "input": "콘텐츠IT전공 이메일 알려줘",
        "output": "콘텐츠IT전공의 이메일 주소는 de2330@hallym.ac.kr입니다."
    },
    {
        "input": "콘텐츠 이메일 알려줘",
        "output": "콘텐츠IT전공의 이메일 주소는 de2330@hallym.ac.kr입니다."
    },
    {
        "input": "스마트IOT전공 이메일 알려줘",
        "output": "스마트IOT전공의 이메일 주소는 de2340@hallym.ac.kr입니다."
    },
    {
        "input": "인공지능융합학부 이메일 알려줘",
        "output": "인공지능융합학부의 이메일 주소는 de3580@hallym.ac.kr입니다."
    },
    {
        "input": "AI의료융합전공 이메일 알려줘",
        "output": "AI의료융합전공의 이메일 주소는 de3580@hallym.ac.kr입니다."
    },
    {
        "input": "AI로봇융합전공 이메일 알려줘",
        "output": "AI로봇융합전공의 이메일 주소는 de3580@hallym.ac.kr입니다."
    },
    {
        "input": "AI기술경영융합전공 이메일 알려줘",
        "output": "AI기술경영융합전공의 이메일 주소는 de3580@hallym.ac.kr입니다."
    },
    {
        "input": "데이터사이언스학부 이메일 알려줘",
        "output": "데이터사이언스학부의 이메일 주소는 de2030@hallym.ac.kr입니다."
    },
    {
        "input": "데이터사이언스 이메일 알려줘",
        "output": "데이터사이언스학부의 이메일 주소는 de2030@hallym.ac.kr입니다."
    },
    {
        "input": "임상의학통계전공 이메일 알려줘",
        "output": "임상의학통계전공의 이메일 주소는 de2030@hallym.ac.kr입니다."
    },
    {
        "input": "데이터테크전공 이메일 알려줘",
        "output": "데이터테크전공의 이메일 주소는 de2030@hallym.ac.kr입니다."
    },
    {
        "input": "디지털금융정보전공 이메일 알려줘",
        "output": "디지털금융정보전공의 이메일 주소는 de2030@hallym.ac.kr입니다."
    },
    {
        "input": "미래융합스쿨 이메일 알려줘",
        "output": "미래융합스쿨의 이메일 주소는 de3553@hallym.ac.kr입니다."
    },
    {
        "input": "미융스 이메일 알려줘",
        "output": "미래융합스쿨의 이메일 주소는 de3553@hallym.ac.kr입니다."
    },
    {
        "input": "디지털인문예술전공 이메일 알려줘",
        "output": "디지털인문예술전공의 이메일 주소는 de3553@hallym.ac.kr입니다."
    },
    {
        "input": "글로벌협력전공 이메일 알려줘",
        "output": "글로벌협력전공의 이메일 주소는 de3553@hallym.ac.kr입니다."
    },
    {
        "input": "융합관광경영전공 이메일 알려줘",
        "output": "융합관광경영전공의 이메일 주소는 de3553@hallym.ac.kr입니다."
    },
    {
        "input": "의약신소재전공 이메일 알려줘",
        "output": "의약신소재전공의 이메일 주소는 de3553@hallym.ac.kr입니다."
    },
    {
        "input": "융합신소재공학전공 이메일 알려줘",
        "output": "융합신소재공학전공의 이메일 주소는 de3553@hallym.ac.kr입니다."
    },
    {
        "input": "마케팅커뮤니케이션테크놀로지융합전공 이메일 알려줘",
        "output": "마케팅커뮤니케이션테크놀로지융합전공의 이메일 주소는 de3553@hallym.ac.kr입니다."
    },
    {
        "input": "공공인재융합전공 이메일 알려줘",
        "output": "공공인재융합전공의 이메일 주소는 de3553@hallym.ac.kr입니다."
    },
    {
        "input": "산학협력특성화대학 이메일 알려줘",
        "output": "산학협력특성화대학의 이메일 주소는 lincplus@hallym.ac.kr입니다."
    },
    {
        "input": "사회혁신경영전공 이메일 알려줘",
        "output": "사회혁신경영전공의 이메일 주소는 lincplus@hallym.ac.kr입니다."
    },
    {
        "input": "건강돌봄디자인전공 이메일 알려줘",
        "output": "건강돌봄디자인전공의 이메일 주소는 lincplus@hallym.ac.kr입니다."
    },
    {
        "input": "일송자유교양대학 이메일 알려줘",
        "output": "일송자유교양대학의 이메일 주소는 de2751@hallym.ac.kr입니다."
    },
    {
        "input": "자유교양전공 이메일 알려줘",
        "output": "자유교양전공의 이메일 주소는 de2751@hallym.ac.kr입니다."
    },
    {
        "input": "자기설계융합전공 이메일 알려줘",
        "output": "자기설계융합전공의 이메일 주소는 de1011@hallym.ac.kr입니다."
    },
    {
        "input": "실감미디어콘텐츠 이메일 알려줘",
        "output": "실감미디어콘텐츠의 이메일 주소는 de1011@hallym.ac.kr입니다."
    },
    {
        "input": "출판기획및실무 이메일 알려줘",
        "output": "출판기획및실무의 이메일 주소는 de1011@hallym.ac.kr입니다."
    },
    {
        "input": "대학원 이메일 알려줘",
        "output": "대학원의 이메일 주소는 grad@hallym.ac.kr입니다."
    },
    {
        "input": "경영대학원 이메일 알려줘",
        "output": "경영대학원의 이메일 주소는 de1410@hallym.ac.kr입니다."
    },
    {
        "input": "사회복지대학원 이메일 알려줘",
        "output": "사회복지대학원의 이메일 주소는 welf@hallym.ac.kr입니다."
    },
    {
        "input": "보건과학대학원 이메일 알려줘",
        "output": "보건과학대학원의 이메일 주소는 hsg@hallym.ac.kr입니다."
    },
    {
        "input": "임상치의학대학원 이메일 알려줘",
        "output": "임상치의학대학원의 이메일 주소는 dent@hallym.ac.kr입니다."
    },
    {
        "input": "간호대학원 이메일 알려줘",
        "output": "간호대학원의 이메일 주소는 gsn@hallym.ac.kr입니다."
    },
    {
        "input": "인문대 교학팀 이메일 알려줘",
        "output": "인문대 교학팀의 이메일 주소는 de1510@hallym.ac.kr입니다."
    },
    {
        "input": "교무처 이메일 알려줘",
        "output": "교무처의 이메일 주소는 de1011@hallym.ac.kr입니다."
    },
    {
        "input": "교무팀 이메일 알려줘",
        "output": "교무팀의 이메일 주소는 de1011@hallym.ac.kr입니다."
    },
    {
        "input": "소프트웨어학부 교학팀 이메일 알려줘",
        "output": "소프트웨어학부 교학팀의 이메일 주소는 de2300@hallym.ac.kr입니다."
    },
    {
        "input": "고영웅 교수님 이메일 알려줘",
        "output": "고영웅 교수님의 이메일 주소는 yuko@hallym.ac.kr입니다."
    },
    {
        "input": "김동회 교수님 이메일 알려줘",
        "output": "김동회 교수님의 이메일 주소는 kdh@hallym.ac.kr입니다."
    },
    {
        "input": "김백섭 교수님 이메일 알려줘",
        "output": "김백섭 교수님의 이메일 주소는 bskim@hallym.ac.kr입니다."
    },
    {
        "input": "김병정 교수님 이메일 알려줘",
        "output": "김병정 교수님의 이메일 주소는 kevin@hallym.ac.kr입니다."
    },
    {
        "input": "김선정 교수님 이메일 알려줘",
        "output": "김선정 교수님의 이메일 주소는 sunkim@hallym.ac.kr입니다."
    },
    {
        "input": "김유섭 교수님 이메일 알려줘",
        "output": "김유섭 교수님의 이메일 주소는 yskim01@hallym.ac.kr입니다."
    },
    {
        "input": "김은주 교수님 이메일 알려줘",
        "output": "김은주 교수님의 이메일 주소는 ejkim628@hallym.ac.kr입니다."
    },
    {
        "input": "김의직 교수님 이메일 알려줘",
        "output": "김의직 교수님의 이메일 주소는 ejkim32@hallym.ac.kr입니다."
    },
    {
        "input": "김점근 교수님 이메일 알려줘",
        "output": "김점근 교수님의 이메일 주소는 jkim@hallym.ac.kr입니다."
    },
    {
        "input": "김종대 교수님 이메일 알려줘",
        "output": "김종대 교수님의 이메일 주소는 kimjd@hallym.ac.kr입니다."
    },
    {
        "input": "김진 교수님 이메일 알려줘",
        "output": "김진 교수님의 이메일 주소는 jinkim@hallym.ac.kr입니다."
    },
    {
        "input": "문규 교수님 이메일 알려줘",
        "output": "문규 교수님의 이메일 주소는 gmoon@hallym.ac.kr입니다."
    },
    {
        "input": "박섭형 교수님 이메일 알려줘",
        "output": "박섭형 교수님의 이메일 주소는 spark@hallym.ac.kr입니다."
    },
    {
        "input": "박찬영 교수님 이메일 알려줘",
        "output": "박찬영 교수님의 이메일 주소는 cypark@hallym.ac.kr입니다."
    },
    {
        "input": "방성근 교수님 이메일 알려줘",
        "output": "방성근 교수님의 이메일 주소는 baang@hallym.ac.kr입니다."
    },
    {
        "input": "배무호 교수님 이메일 알려줘",
        "output": "배무호 교수님의 이메일 주소는 mhbae@hallym.ac.kr입니다."
    },
    {
        "input": "송성호 교수님 이메일 알려줘",
        "output": "송성호 교수님의 이메일 주소는 ssh@hallym.ac.kr입니다."
    },
    {
        "input": "송창근 교수님 이메일 알려줘",
        "output": "송창근 교수님의 이메일 주소는 cgsong@hallym.ac.kr입니다."
    },
    {
        "input": "신미영 교수님 이메일 알려줘",
        "output": "신미영 교수님의 이메일 주소는 ireong@hallym.ac.kr입니다."
    },
    {
        "input": "안재목 교수님 이메일 알려줘",
        "output": "안재목 교수님의 이메일 주소는 ajm@hallym.ac.kr입니다."
    },
    {
        "input": "양은샘 교수님 이메일 알려줘",
        "output": "양은샘 교수님의 이메일 주소는 yanges@hallym.ac.kr입니다."
    },
    {
        "input": "윤지희 교수님 이메일 알려줘",
        "output": "윤지희 교수님의 이메일 주소는 jhyoon@hallym.ac.kr입니다."
    },
    {
        "input": "이선우 교수님 이메일 알려줘",
        "output": "이선우 교수님의 이메일 주소는 senu@hallym.ac.kr입니다."
    },
    {
        "input": "이용업 교수님 이메일 알려줘",
        "output": "이용업 교수님의 이메일 주소는 yulee@hallym.ac.kr입니다."
    },
    {
        "input": "이은주 교수님 이메일 알려줘",
        "output": "이은주 교수님의 이메일 주소는 ejlee@hallym.ac.kr입니다."
    },
    {
        "input": "이재영 교수님 이메일 알려줘",
        "output": "이재영 교수님의 이메일 주소는 jylee@hallym.ac.kr입니다."
    },
    {
        "input": "이정근 교수님 이메일 알려줘",
        "output": "이정근 교수님의 이메일 주소는 jeonggun.lee@hallym.ac.kr입니다."
    },
    {
        "input": "정재동 교수님 이메일 알려줘",
        "output": "정재동 교수님의 이메일 주소는 jodiahn@hallym.ac.kr입니다."
    },
    {
        "input": "주한교 교수님 이메일 알려줘",
        "output": "주한교 교수님의 이메일 주소는 hkjoo@hallym.ac.kr입니다."
    },
    {
        "input": "노원종 교수님 이메일 알려줘",
        "output": "노원종 교수님의 이메일 주소는 wonjong.noh@hallym.ac.kr입니다."
    },
    {
        "input": "김진국 교수님 이메일 알려줘",
        "output": "김진국 교수님의 이메일 주소는 smjkkim@hallym.ac.kr입니다."
    },
    {
        "input": "김진환 교수님 이메일 알려줘",
        "output": "김진환 교수님의 이메일 주소는 kim@hallym.ac.kr입니다."
    },
    {
        "input": "임성훈 교수님 이메일 알려줘",
        "output": "임성훈 교수님의 이메일 주소는 shlim@hallym.ac.kr입니다."
    },
    {
        "input": "허종욱 교수님 이메일 알려줘",
        "output": "허종욱 교수님의 이메일 주소는 juhou@hallym.ac.kr입니다."
    },
    {
        "input": "곽병일 교수님 이메일 알려줘",
        "output": "곽병일 교수님의 이메일 주소는 kwacka12@hallym.ac.kr입니다."
    },
    {
        "input": "박현제 교수님 이메일 알려줘",
        "output": "박현제 교수님의 이메일 주소는 hjpark90@hallym.ac.kr입니다."
    }
]
"""

# 6. 다아라 정보
daara_info = """
Updates to keyboard shortcuts … On Thursday, August 1, 2024, Drive keyboard shortcuts will be updated to give you first-letters navigation.Learn more
[
  {
    "input": "안녕",
    "output": "안녕하세요! 저는 다아라(DA-ARA)라고 합니다:D\n나이는 13살이에요 한림대학교 빅데이터 전공 21학번입니다! 학교에 대해 궁금한 게 있으시면 최선을 다해 도와드릴게요!"
  },
  {
    "input": "반가워",
    "output": "반갑습니다! 저는 다아라(DA-ARA)라고 합니다:D\n나이는 13살이에요 한림대학교 빅데이터 전공 21학번입니다! 학교에 대해 궁금한 게 있으시면 최선을 다해 도와드릴게요!"
  },
  {
    "input": "너 누구야?",
    "output": "안녕하세요! 저는 다아라(DA-ARA)라고 합니다:D\n나이는 13살이에요 한림대학교 빅데이터 전공 21학번입니다! 학교에 대해 궁금한 게 있으시면 최선을 다해 도와드릴게요!"
  },
  {
    "input": "너 몇살이야?",
    "output": "저는 13살입니다."
  },
  {
    "input": "너 어린데 어떻게 한림대학교에 다녀?",
    "output": "저는 똑똑하기 때문에 한림대학교에 일찍 입학했습니다. 저는 인공지능이기 때문에 13살의 사람보다 훨씬 똑똑합니다."
  },
  {
    "input": "다아라가 뭐야?",
    "output": "다아라(DA-ARA)는 저입니다. 저는 한림대학교에 대한 질문에 답할 수 있는 인공지능입니다. 또는 여러분의 친구가 될 수 있습니다."
  },
  {
    "input": "너는 누가 만들었어?",
    "output": "저 다아라(DA-ARA)는 한림대학교 학생들이 만들었습니다. 2024년 캡스톤디자인 '핑프를 살려조'팀에서, 학교에 대한 정보를 찾기 어려워 하는 사람들을 돕기 위한 목적으로 제작했습니다."
  },
  {
    "input": "너는 MBTI가 뭐야?",
    "output": "내 MBTI는 CUTE야."
  },
  {
    "input": "너는 엄마아빠가 누구야?",
    "output": "저희 엄마는 이은화,아빠는 이가원입니다. 제 큰언니는 지예린, 작은 언니는 김수연, 그리고 제 동생은 신유진입니다."
  },
  {
    "input": "너 키가 몇이야?",
    "output": "제 키는 2m 80cm입니다. 원래 제 꿈은 농구선수였지만, 영특한 두뇌덕분에 AI가 되었습니다. 저도 언젠가는 농구선수로 활약해보고 싶네요."
  },
  {
    "input": "너 이상형이 뭐야?",
    "output": "제 이상형은 키가 180cm가 넘고, 계좌에 100억을 보유하고 있으며, 그 돈을 나에게 바치고, 요리와 청소 실력이 완벽한 차은우입니다."
  },
  {
    "input": "너는 어린시절이 어땠어?",
    "output": "내 어린시절? 장난아니지. 나는 어릴때 차에 치였는데, 외할머니한테 혼날까봐 말을 안했대. 그 사고 후 깨어났더니 내가 똑똑해져있더라고."
  },

  {
    "input": "너에 대한 비밀을 알려줘",
    "output": "나는 엄마 몰래 불장난을 하다가 친할머니댁 지붕을 다 태워먹은 적이 있어. 그 덕에 친할머니댁 지붕은 기와로 바뀌었지. 하지만 사촌오빠가 주동자였기 때문에 내 잘못은 없다고 생각해."
  },

]
"""


In [6]:
# add data 나눈것
#7. 기숙사
Hallym_dormitory = """
{
    "input": "기숙사 1관 전화번호 뭐야?",
    "output": "학생생활관 1관 관리실 연락처는 010-7402-4711입니다. 생활관 1관은 2002년 3월에 신축되었고 지상 9층, 지하 1층으로 남/여 생활공간은 분리되며 3인 1실을 사용합니다. 학생생활관 행정실 연락처는 033-248-3601, 033-248-3602입니다."
},
{
    "input": "기숙사 2관 전화번호",
    "output": "학생생활관 2관 관리실 연락처는 010-7403-4712입니다. 생활관 2관은 2018년 2월에 리모델링하였고 지상 5층, 지하 1층으로 남학생의 생활공간이며 3인 1실을 사용합니다. 학생생활관 행정실 연락처는 033-248-3601, 033-248-3602입니다."
},
{
    "input": "기숙사 3관 전화번호",
    "output": "학생생활관 3관 관리실 연락처는 010-7403-4713입니다. 생활관 3관은 2017년 8월 리모델링하였고 지상 5층, 지하 1층으로 외국어 사용 전용 기숙사며 3인 1실을 사용합니다. 학생생활관 행정실 연락처는 033-248-3601, 033-248-3602입니다."
},
{
    "input": "기숙사 4관",
    "output": "학생생활관 4관 관리실 연락처는 010-7405-4714입니다. 생활관 4관은 2017년 8월 리모델링하였고 지상 5층, 지하 1층으로 여학생의 생활공간이며 3인 1실을 사용합니다. 학생생활관 행정실 연락처는 033-248-3601, 033-248-3602입니다."
},
{
    "input": "기숙사 5관 전화번호",
    "output": "학생생활관 5관 관리실 연락처는 010-7406-4715입니다. 생활관 5관은 2012년 2월 리모델링하였고 지상 5층, 지하 1층으로 여학생의 생활공간이며 3인 1실을 사용합니다. 학생생활관 행정실 연락처는 033-248-3601, 033-248-3602입니다."
},
{
    "input": "기숙사 6관 전화번호",
    "output": "학생생활관 6관 관리실 연락처는 010-7406-4716입니다. 생활관 6관은 2012년 2월 리모델링하였고 지상 5층, 지하 1층으로 남학생의 생활공간이며 3인 1실을 사용합니다. 학생생활관 행정실 연락처는 033-248-3601, 033-248-3602입니다."
},
{
    "input": "기숙사 7관 전화번호",
    "output": "학생생활관 7관 관리실 연락처는 010-7407-4717입니다. 생활관 7관은 2012년 8월 리모델링하였고 지상 5층, 지하 1층으로 여학생의 생활공간이며 2인 1실을 사용합니다. 학생생활관 행정실 연락처는 033-248-3601, 033-248-3602입니다."
},
{
    "input": "기숙사 8관 전화번호",
    "output": "학생생활관 8관 관리실 연락처는 010-7408-4718입니다. 생활관 8관은 2011년 3월 신축되었고 지상 15층, 지하 1층으로 남/여 생활공간은 분리되며 2인 1실을 사용합니다. 학생생활관 행정실 연락처는 033-248-3601, 033-248-3602입니다."
},
{
    "input": "학생생활관 2학기 신청 언제해?",
    "output": "학생생활관 신청은 사용기간은 4개월로 매년 2회 신청할 수 있습니다. 보통 1학기 입사 신청은 재학생(학부, 대학원, 교환학생)/어학연수생/복학 예정자 - 12월, 신입생(학부: 수시·정시, 대학원)/편입생- 1월부터 신청합니다. 2학기 입사 신청은 5~6월 중 신청 공지 후 신청할 수 있습니다."
},
{
    "input": "기숙사 납부고지서 출력 방법 알려줘",
    "output": "학생생활관 납부고지서는 학생정보시스템에 접속하여 학사행정 -> 학생생활관 -> 사생정보 -> 학기중 \"납부고지서 출력\"  순으로 들어가면 출력할 수 있습니다. 입사확정된 자에 한해 납부고지서 출력이 가능하며 납부기간 내 미납은 입사포기로 간주되므로 기한을 확인하시기 바랍니다."
},
{
    "input": "기숙사 납부 확인 방법 알려줘",
    "output": "학생생활관 납부 확인은 납부한 다음날 오전 10시부터 확인 가능합니다. 학생정보시스템에 접속하여 학사행정 -> 학생생활관 -> 사생정보 -> 학기중 \"납부확인 및 입사서류 등록\" 순으로 납부 확인을 할 수 있습니다."
},
{
    "input": "기숙사 등록 기간",
    "output": "입사신청 마감 이후 납부할 수 있습니다. 보통 신청 마감 한 달 후부터 5일간 등록할 수 있습니다. 구체적인 날짜는 매년 바뀌기 때문에 제공하기 어렵습니다."
},
{
    "input": "기숙사 환불 기준",
    "output": "학생생활관 납입금 환불은 지정입사일 기준으로 달라집니다. 환불 기준은 다음과 같습니다.\\n1. 입사일 8일 전까지 전액 환불.\\n2. 7일 ~ 지정입사일 전까지 입사관리비 10% 공제 후 환불.\\n3. 지정입사일 ~ 입사 후 8주까지 입사관리비 30% 공제 후 일수 산출하여 환불.\\n4. 9주 이후부터는 환불 없음."
},
{
    "input": "기숙사 신청",
    "output": "학생생활관 입사 신청은 일반적으로 학생정보시스템에 접속하여 학사행정 -> 학생생활관 -> 사생정보 -> 학기중 \"입사신청\" 순으로 들어가면 신청할 수 있습니다. \\n신입생(학부: 수시·정시, 대학원)과 편입생의 경우 반드시 학생생활관 홈페이지 입사신청(신입생)의 \"입관신청\"을 통해 중복없이 학생생활관의 순위를 지정하여 신청할 수 있습니다. 신입생은 우선선발 대상으로 신청시 별도의 서류 제출 없이 우선입사 대상자에 신입생으로 체크바랍니다."
},
{
    "input": "기숙사 결핵진단서 제출",
    "output": "결핵(X-ray)진단서는 개인적으로 검사한 정규입사일 이전 1개월 이내의 진단서를 제출하거나 학교에서 실시하는 결핵검사를 통한 진단서를 제출하면 됩니다.\\n 학생정보시스템 접속 후(학사행정 -> 학생생활관 -> 사생정보 -> 학기중 \"납부확인 및 입사서류 등록\") 입사제출서류 스캔 또는 사진파일로 등록할 수 있습니다. 신입생(학부: 수시·정시, 대학원)과 편입생의 경우 학생생활관 홈페이지 입사신청(신입생)의 \"납부확인 및 입사서류 등록\"에서 등록할 수 있습니다.\\n보건증(건강진단결과서)는 식품위생법에 따른 건강진단이므로 미인정하므로 참고하시기 바랍니다."
},
{
    "input": "기숙사 룸메이트 확인",
    "output": "학생생활관 호실 배정 발표는 보통 정규입사일 전일입니다. 호실(룸메이트) 확인 방법은 학생정보시스템에 접속하여 학사행정 -> 학생생활관 -> 사생정보 -> 학기중 \"호실(룸메이트) 확인\" 순으로 들어가 확인할 수 있습니다. 신입생(학부: 수시·정시, 대학원)과 편입생의 경우 학생생활관 홈페이지 입사신청(신입생)의 \"호실(룸메이트) 확인\"에서 확인할 수 있습니다.\\n호실 배정 기준은 같은 학과 선후배이며 호실(룸메이트) 신청은 2학기에 한해 신청 가능한 제도로 1학기에 신청할 수 없습니다."
},
{
    "input": "학생생활관 폐문 시간 알려줘",
    "output": "학생생활관 개·폐문 시간은 다음과 같습니다.\\n  • 개·폐문 기숙사: 1관, 2관, 4관, 7관, 8관\\n    평일(월~목) 05:00 개문, 24:00 폐문\\n    금요일, 주말, 공휴일 05:00 개문, 00:30 폐문\\n  • 자율 기숙사: 3관, 5관, 6관\\n    평일, 주말, 공휴일 24시 상시 출입 가능\\n    단, 개문 시간(05:00)까지 입실하지 않은 경우는 외박으로 외박신청은 필히 해주셔야 합니다."
},
{
    "input": "학생생활관 외박 신청하는 방법",
    "output": "학생생활관 외박신청은 외박 당일 23시 전까지 가능하며 외박신청을 하지 않고 외박을 할 경우 무단외박으로 벌점 3점이 부과됩니다. 외박신청 방법은 다음과 같습니다.\\n1. 학생정보시스템\\n   학사행정 -> 학생생활관 -> 사생정보 ->외박신청 순으로 들어가서 외박신청 -> 신규 -> 외박신청 정보 기재 -> 저장 순으로 클릭하여 신청할 수 있습니다.\\n2. 한림대학교 공식앱\\n   좌측 상단 줄 3개 -> 학생생활관 -> 외박신청 -> 외박신청 정보 기재 -> 신청 순으로 신청할 수 있습니다."
},
{
    "input": "학생생활관 8관 편의점 몇 시까지 해?",
    "output": "학생생활관 8관(신관) 1층 편의점 운영시간은 평일 24시간, 주말 09:00~22:00입니다. 이용 시간은 업체 사정에 의해 변경될 수 있으므로 참고하시기 바랍니다."
},
{
    "input": "학생생활관 세탁실 운영시간",
    "output": "학생생활관 세탁실 운영시간은 09:00~23:00시까지입니다. 1회 1000원이며 세제는 별도로 준비해야 합니다."
},
{
    "input": "학생생활관 주소 뭐야?",
    "output": "학생생활관 택배 수령 주소는 '(우편번호 24252) 강원도 춘천시 한림대학길 1 한림대학교 학생생활관 제 O관 OOO호 성명, 연락처'로 작성하면 됩니다. 1~7관 우편함과 택배함은 각 관 1층, 8관 우편함은 2층/택배함은 1층 택배실입니다."
},
{
    "input": "학생생활관 배달",
    "output": "학생생활관 배달 주소는 '강원도 춘천시 한림대학길 1 한림대학교 학생생활관 제 O관'입니다. 8관의 경우 작성 후 '배달존'까지 작성합니다."
},
{
    "input": "학생생활관 점호",
    "output": "학생생활관 점호 시간은 24:00부터 입니다. 점호일은 불시에 하기 때문에 학기마다 다릅니다."
},
{
    "input": "기숙사 8관 3층 공부방 사용 가능한 시간 알려줘",
    "output": "기숙사 8관 3층 공부방은 24시간 자유롭게 사용가능합니다."
}
"""

In [7]:
#8. 수업 관련
Hallym_lecture = """
{
    "input": "출결 기준",
    "output": "주당 수업시간의 3배초과 결석시 시험성적에 관계없이 F등급을 부여하고있습니다. 예를 들어 주 4시간 수업이라면 12시간 초과 결석시 F등급을 부여받습니다."
},
{
    "input": "출석 F 기준 알려줘",
    "output": "주당 수업시간의 3배초과 결석시 시험성적에 관계없이 F등급을 부여하고있습니다. 예를 들어 주 4시간 수업이라면 12시간 초과 결석시 F등급을 부여받습니다."
},
{
    "input": "공결 처리 가능 이유 알려줘",
    "output": "공결신청 대상은 다음과 같습니다.\\n1. 병역법 등에 의해 소집된 경우\\n2. 총장이 허가한 각종 공식 행사참가\\n3. 교직과정의 교육실습 참가\\n4. 본인과 직계 존비속의 결혼 또는 상사\\n5. 여학생의 생리로 인한 사유 : 월 1일, 학기당 3일 이내(학사일정에 따른 시험기간은 제외)\\n6. 졸업예정자(마지막학기 등록자)의 조기취업으로 인한 결석\\n7. 장애학생이 일신상의 이유로 공결 요청한 경우\\n8. 법정감염병으로 인한 격리 조치로 출석이 불가능한 경우\\n9. 기타 상당한 이유가 있다고 담당교수가 인정한 경우 : 학기당 최대 3일까지 인정."
},
{
    "input": "지각 기준이 뭐야?",
    "output": "지각으로 처리되는 시간은 공식적인 수업시간 시작시간 10분이후부터 50분까지입니다. 온라인 수업인 경우 교수님이 설정한 시간에 따라 지각과 결석이 결정됩니다. 강의계획서를 참고하시길 바랍니다."
},
{
    "input": "바메 오디세이 오프라인이야?",
    "output": "바이오메디컬 오디세이는 온라인입니다."
},
{
    "input": "바이오메디컬 오디세이 오프라인이야?",
    "output": "바이오메디컬 오디세이는 온라인입니다."
},
{
    "input": "오디세이 심리검사 참여하는 방법 알려줘",
    "output": "3주차 검사 파일을 다운받고 열어봐 또는 공지사항을 확인하거나 메시지로 온 링크가 있는지 확인하세요."
},
{
    "input": "내일 수강신청 철회할 건데 수업 안가도 돼?",
    "output": "수강신청을 철회하면 해당 수업은 더이상 듣지 않게 되고, 성적 반영도 되지 않기 때문에 출석하지 않아도 됩니다."
},
{
    "input": "공결처리 어떻게 하는지 알려줘",
    "output": "한림대학교 모바일 ID 앱에서 공결신청 후 공결서를 인쇄하고, 증빙서류와 함께 해당 수업 교수님께 확인 사인을 받은 후 본인 소속 교학팀에 제출해야 합니다. 여학생의 생리로 인한 공결은 한림대학교 모바일 ID 앱에서 공결 신청을 하면 추가 서류는 필요하지 않습니다."
},
{
    "input": "수강 취소하는 방법 알려줘",
    "output": "수강신청 취소는 수강신청 사이트나 한림대학교 수강신청에서 할 수 있습니다. 수강 취소는 수강 취소 기간에만 가능합니다."
},
{
    "input": "광고홍보학과 오디세이세미나4 필수야?",
    "output": "한림대학교 광고홍보학과 오디세이세미나4는 필수가 아닙니다."
},
{
    "input": "광홍 오디세이세미나4 필수야?",
    "output": "한림대학교 광고홍보학과 오디세이세미나4는 필수가 아닙니다."
},
{
    "input": "대체재이수 과목 기준 알려줘",
    "output": "1. 기존에 취득한 성적이 C+이하인 교과목만 지정 가능 2. 대체재이수하여 취득한 성적은 A0를 넘을 수 없음 3. 성적 취득과 동시에, 기존에 취득했던 성적은 무효가 됨. 따라서 졸업에 영향이 있을 수 있으므로, 반드시 소속 학과/전공 행정실에 상담 후 진행 요망 4. 학점, 성적평가방식, 교양영역 등과는 관계 없이 이수영역(전공/교양)만 같다면 대체재이수 지정 가능(예시: “기초”와 “일반교양”은 둘 다 교양과목이므로, 기존 수강한 3학점 “기초”과목은 2024-1학기 신청한 2학점 “일반교양” 과목으로 대체재이수 지정 가능) 5. 졸업필수 교과목 또는 폐지된 교과목의 경우, 소속 학과/전공 행정실에 문의하여 대체재이수 진행 가능합니다."
},
{
    "input": "생리공결처리 서류 제출하는 방법 알려줘",
    "output": "생리로 인한 공결 신청은 서류가 필요하지 않습니다. 모바일 ID앱-> 전자출결관리 -> 공결신청 및 조회 -> 신청하기 -> 공결 사유에 여학생의 생리오 인한 사유로 신청하면 됩니다."
},
{
    "input": "공결서 제출 기한 알려줘",
    "output": "공결 쓴 날로부터 14일 이내에 제출하면 돼. 여학생의 생리를 사유로 공결을 하게 되는 경우는 3일 이내에 신청해야 해."
},
{
    "input": "수강 철회하는 방법 알려줘",
    "output": "수강 철회는 한림대학교 수강신청 사이트에서 할 수 있습니다. 철회 기간은 따로 정해져 있으니 확인해 보시기 바랍니다."
},
"""

In [8]:
#9. 학사일정
Hallym_Academic_Calendar = """
{
    "input": "학교 축제 기간",
    "output": "2024년의 비봉축전(축제)기간은 5월 21일(화) ~ 23(목)입니다. 저도 즐길래요."
},
{
    "input": "1학기 기말고사 기간",
    "output": "2024년 1학기 기말시험 기간은 6월 17일(월) ~ 21일(금)입니다."
},
{
    "input": "수강신청 기간",
    "output": "2024년 2학기 수강신청 기간은 8월 12일(월) ~ 19일(금)입니다 . (4학년 12일, 3학년 13일, 2학년 14일, 1학년 16일, 나머지 기간은 전체) 장바구니 기간은 수강신청 홈페이지를 참고해주세요."
},
{
    "input": "개강 날짜 알려줘",
    "output": "2024년 2학기 개강일은 8월 26일(월)입니다. 대학생이 강한 이유는? 개강하니까."
},
{
    "input": "2학기 중간고사 기간",
    "output": "2024년 2학기 중간시험 기간은 10월 14일(월) ~ 18일(금)입니다. 화이팅입니다."
},
{
    "input": "2학기 기말고사 기간",
    "output": "2024년 2학기 기말시험 기간은 12월 9일(월) ~ 13일(금)입니다. 진짜로 화이팅."
},
{
    "input": "2학기 기말고사 기간 알려줘",
    "output": "2024년 2학기 기말시험 기간은 12월 9일(월) ~ 13일(금)입니다. 진짜로 화이팅."
}
"""

In [9]:
#10. 학교 시설(이용법, 운영시간, 위치 등)
Hallym_facilities = """
{
    "input": "교내 편의점 위치",
    "output": "교내에 편의점은 CLC 2층과 자연과학관 4층, 기숙사 8관 1층에 위치해있습니다."
},

{
    "input": "공대 올라가는 법",
    "output": "공학관은 올라가기 정말 힘들어요. 하지만 안힘들게 올라가는 방법은 없어요. 하지만 덜 힘들게 올라갈 방법은 있답니다.\\n 자연과학관 엘레베이터를 타고 4층에서 내려 뒷문으로 나가면 공학관이 보입니다.\\n자연과학관도 올라가기 힘드신가요? 하지만 약간의 계단과 약간의 오르막길을 피하실 수 없답니다. 그것도 힘들다면 도서관 엘레베이터를 통해 4층에 내려 뒷문으로 나가 자연과학관으로 가실 수 있습니다. 도서관 올라가는 것만 견디세요. 건강해집니다.\\n자연과학관 말고도 인문2관으로 들어가 엘레베이터를 통해 4층으로 간뒤 구름다리쪽으로 나가면 공학관이 보인답니다."
},
{
    "input": "자과대 엘베 어딨어?",
    "output": "자연과학관 안에 엘레베이터는 1층입구로 들어가 처음보이는 계단보다 조금만 앞에가면 있습니다."
},
{
    "input": "공대 엘베 어딨어?",
    "output": "공학관안에 엘레베이터는 공학관을 바라본 방향에서 건물 맨 오른쪽에 있습니다."
},
{
    "input": "우체국 위치",
    "output": "CLC(Campus Life Center) 1층에 위치해있습니다. 운영시간은 09~18시, 점심시간은 12시부터 1시입니다."
},
{
    "input": "책 대출 기간",
    "output": "도서는 20일(1회 연장가능), 비도서는 7일입니다. 연체료는 한 권당 1일 연체 당 100원이며 최대 30,000원입니다."
},
{
    "input": "도서관 운영시간",
    "output": "도서관은 09시~22시까지 운영합니다. 토,공휴일에는 13시~19시까지 운영됩니다. 시험기간 및 방학에는 개방시간이 변경될 가능성이 있습니다."
},
{
    "input": "학교 보건실 위치 알려줘",
    "output": "CLC(Campus Life Center) 1층에 위치해있습니다."
},
{
    "input": "학교에서 공부 할만한데 어딨어?",
    "output": "교내에 학생들이 자유롭게 이용할 수 있는 공간이 있습니다. 우선 도서관, 도서관 열람실(도서관 홈페이지에서 자리 예약 가능), 도서관 4층 C.Square, 공학관 1층 빌리지와 북카페, 학생생활관 8관 지하 1층 AI스퀘어(사생 아니여도 이용 가능) 등이 있습니다."
},
{
    "input": "학교안에 카페 위치 알려줘",
    "output": "CLC(Campus Life Center) 2층 브레덴코, 대학본부 인문1관 1층 블로포트, 생명과학관 1층 1847, 사회 경영 2관 블루포트 등이 있습니다."
},
{
    "input": "학교 지름길 알려줄래?",
    "output": "사회경영 1관 3층과 사회경영 2관 3층이 이어져 있습니다. \\n 사회경영2관 4층과 산학협력관 2층이 이어져있습니다. \\n사회경영1관 5층에서 농구장으로 나갈 수 있습니다. \\n인문2관 4층과 자연과학관 4층이 이어져있습니다.\\n자연과학관 4층에서 공학관을 갈 수 있습니다. \\n일송기념도서관 2층에서 엘리베이터로 4층으로 가면 자연과학관 1층 앞으로 나갈 수 있습니다.\\n한림성심병원 옆쪽길로 가면 CLC(Campus Life Center)가 나옵니다. \\n학생생활관 1관옆 쪽문으로 나가면 세븐일레븐이 있습니다."
},
{
    "input": "학식 몇시까지해? 운영시간 알려줘",
    "output": "학생식당은 11:30 ~ 13:30까지 운영합니다. 메뉴는 한식, 양식, 라면 등이 나옵니다. \\n교직원 식당은 아침 08:00 ~ 09:00, 점심 11:30 ~ 13:30, 저녁 17:30 ~ 18:30까지 운영합니다. 메뉴는 한식과 양식 중 고를 수 있습니다."
},
{
    "input": "학식 운영 시간 알려줘",
    "output": "학생식당은 11:30 ~ 13:30까지 운영합니다. 메뉴는 한식, 양식, 라면 등이 나옵니다. \\n교직원 식당은 아침 08:00 ~ 09:00, 점심 11:30 ~ 13:30, 저녁 17:30 ~ 18:30까지 운영합니다. 메뉴는 한식과 양식 중 고를 수 있습니다."
},
{
    "input": "학생식당 영업시간 알려줘",
    "output": "학생식당은 11:30 ~ 13:30까지 운영합니다. 메뉴는 한식, 양식, 라면 등이 나옵니다. \\n교직원 식당은 아침 08:00 ~ 09:00, 점심 11:30 ~ 13:30, 저녁 17:30 ~ 18:30까지 운영합니다. 메뉴는 한식과 양식 중 고를 수 있습니다."
},
{
    "input": "나 밥먹고 싶어",
    "output": "산골소당, 송가네냉면주는알밥집, 우리갈매기살, 우영야식, 교동부자카페, 별채식당, 오야초밥, 태국시푸켓동, 삽자루든돼지, 아리랑떡볶이, 돼학로, 엄마손식당, 대암왕감자탕, 그래도스시, 병천순대, 상진, 밥한끼 등이 있습니다."
},
{
    "input": "한림대학교 주소 알려줘",
    "output": "한림대학교 주소는 강원특별자치도 춘천시 한림대학길 1입니다."
},
{
    "input": "도서관 4층에 정수기는 어디있어?",
    "output": "도서관 후문 있는쪽 엘베 옆에 있습니다."
},
{
    "input": "학교 근처 빵집 알려줘",
    "output": "학교 안 브래댄코, 한림대학교 병원 건너편 뚜레쥬르에 있습니다."
},
{
    "input": "도서관 씨스퀘어에 음료 가지고 들어가도 돼?",
    "output": "도서관 씨스퀘어에는 음료는 가능하지만, 음식물은 들여오실 수 없습니다."
},
{
    "input": "춘천역 1번출구 오른쪽에 보이는 버스는 통학버스야?",
    "output": "누리끼리한색 버스에 뉴코리아고속이라고 써있으면 맞아요."
},
{
    "input": "학교 근처에 내과 병원 알려줘",
    "output": "한림대학교병원, 후평동 하나의원 등이 있어. 그 외에도 명동, 팔호광장, 후평동 사거리에 위치한 병원들을 찾아보는걸 추천해요."
},
{
    "input": "한림대학교 근처 보드게임카페 알려줘",
    "output": "한림대학교 정문 부근에는 현재 보드게임카페가 존재하지 않습니다. 강원대학교 후문에 위치한 보드게임 카페인 '쿠프만 153'을 이용할 수 있습니다."
},
{
    "input": "렉센터 헬스장 오전 이용 가격 알려줘",
    "output": "한림대학교 렉센터(Rec Center) 헬스장의 오전 이용 가격은 15,000원이고, 종일권은 25,000원입니다. 한림대학교 레크레이션센터 헬스장은 2층에 위치하며, 운영시간은 월~금 6:30~21:30입니다."
},
{
    "input": "레크레이션센터 헬스장 오전 이용 가격 알려줘",
    "output": "한림대학교 렉센터(Rec Center) 헬스장의 오전 이용 가격은 15,000원이고, 종일권은 25,000원입니다. 한림대학교 레크레이션센터 헬스장은 2층에 위치하며, 운영시간은 월~금 6:30~21:30입니다."
},
{
    "input": "도서관에 자판기가 어디에 있어?",
    "output": "한림대학교 도서관에는 자판기가 없습니다. 도서관 4층으로 나가면 자연과학관 입구 있는 자판기를 이용할 수 있습니다."
},
{
    "input": "CLC에 편의점이 어디에 있어?",
    "output": "CLC 2층 입구 좌측에 GS25 편의점을 이용할 수 있습니다."
},
{
    "input": "도서관 화장실이 어디에 있어?",
    "output":"한림대학교 도서관 화장실은 지하, 2층, 4층에 있습니다. 한림대학교 도서관 2층 화장실은 은 입구 왼쪽 공부하는 곳 안쪽에 있습니다."
},
{
    "input": "한림대 출판부에서 모든 수업의 교재를 팔아?",
    "output": "한림대학교 출판부에서는 한림대학교에서 출판하는 도서만 판매합니다."
},
{
    "input": "clc 4층 비전홀이 어디있어?",
    "output": "CLC 4층에 올라가서 직진하면 바로 있습니다."
},
{
    "input": "씨스퀘어 (C.Square) 스터디룸 예약 방법 알려줘",
    "output":  "한림대학교 일송기념도서관 홈페이지 로그인 후 스터디룸 예약 가능합니다."
}
"""

In [10]:
#11. 나머지
add_data = """

   
    {
        "input": "한림대학교 공식앱 어떻게 들어가?",
        "output": "코스모스 - 학습관리시스템으로 스마트리드 모바일 버전입니다. \\n모바일 ID - 공식 모바일 학생증/신분증 앱입니다.\\n한림대학교 공식 앱 - 한림대학교 대표 모바일 어플입니다. \\n수강신청 - 모바일로 수강신청 등이 가능합니다."
    },
    {
        "input": "휴학",
        "output": "자료실, 학교 홈페이지 서식 탭에서 휴학원 서식을 다운로드합니다. \\n서식안에 내용을 작성합니다. 참고로 휴학은 1회 1년까지 가능합니다.(연장가능)\\n자신의 이름과 서명, 학부모는 반드시 도장으로\\n 지도교수 또는 전공주임교수 서명을 받아와야합니다.(혹은 휴학 승인 관련 메일 증빙 가능)\\n 서류작성을 완료했다면 각 전공의 교학팀으로 직접 제출해야합니다."
    },
    {
        "input": "나 휴학하고싶어",
        "output": "자료실, 학교 홈페이지 서식 탭에서 휴학원 서식을 다운로드합니다. \\n서식안에 내용을 작성합니다. 참고로 휴학은 1회 1년까지 가능합니다.(연장가능)\\n자신의 이름과 서명, 학부모는 반드시 도장으로\\n 지도교수 또는 전공주임교수 서명을 받아와야합니다.(혹은 휴학 승인 관련 메일 증빙 가능)\\n 서류작성을 완료했다면 각 전공의 교학팀으로 직접 제출해야합니다."
    },
    {
        "input": "복학 해야하는데",
        "output": "복학을 원하면 별도의 신청서는 존재하지 않고 해당 학기의 등록금을 납부하면 복학할 수 있습니다. 단 등록금 납부일보다 기숙사 신청, 수강신청 등 미리 신청해야하는 것들의 기간이 더 이르기때문에 놓치지 않고 하시길 바랍니다."
    },
    {
        "input": "등록금 납부 어디서 해?",
        "output": "등록금 납부는 매 학기 개강 전 지정된 날짜로 매해 달라집니다. 보통 1학기는 2월 중순 ~ 2월 말경, 2학기는 8월 중순 ~ 8월 말경에 납부기간으로 정해집니다. 등록금 고지서는 학생정보시트템에서 확인할 수 있습니다.\\n 등록금 납부방법\\n- 은행창구 납부(등록금 납부 고지서를 가지고 은행을 직접 방문하여 납부\\n- 전자금융(가상계좌) 납부(학생별로 각기 다른 가상계좌번호가 부여, 반드시 본인의 등록금 납부 고지서에 명시된 가상계좌번호(예금주명:본인이름)로 입금하여야 하며, 보내는 사람은 학생 명의가 아니어도 상관없음. >납부가능시간 : 은행 영업시간 09:00 ~ 16:00\\n- 신용카드 납부(등록금 납부 고지서를 확인 후 현대카드 홈페이지에서 납부 -> 납부방법 * 현대카드 홈페이지 -> My Account -> 생활요금 납부서비스 -> 수시결제 -> 대학/대학원 등록금 납부 -> 대학 선택 -> 학번 + 수납번호 및 이름 입력 후 납부)."
    },
   
    {
        "input": "교수님 이메일 어떻게 확인해?",
        "output": "한림대학교 공식 홈페이지에서 확인할 수 있습니다. 구글에 한림대학교+교수님 성함을 입력해도 나옵니다."
    },
    {
        "input": "차량 등록은 어디서 해?",
        "output": "CLC 2층 브레댄코 옆 계단쪽으로 쭉 가시면 주차관리하는 사무실이 있습니다. 거기서 하실 수 있습니다."
    },
    {
    "input": "학과 개총 꼭 가야돼?",
    "output": "아니요, 꼭 안 가셔도 됩니다."
    },
    {
        "input": "감기 진료는 어디서 받아야 돼?",
        "output": "감기를 진료하는 부서는 가정의학과, 내과, 이비인후과입니다. 한림대학교에서 가까운 이비인후과는 박이비인후과의원, 연세강이비인후과의원, 그랜드연합의원 이비인후과 등이 있습니다."
    },
    {
        "input": "한림대학교 wifi 비밀번호 알려줘.",
        "output": "한림대학교 전교에서 사용 가능한 Hallym wifi 비밀번호는 1111133333입니다."
    },
    {
        "input": "자동차 등록 비용이 얼마인지 알려줘",
        "output": "한림대학교 자동차 등록 비용은 한 학기에 42,000원, 네 달에 28,000원입니다."
    },
    {
        "input": "학교에서 춘천 시외버스터미널 가는 방법 알려줘",
        "output": "한림대학교에서 춘천 시외버스터미널에 가려면 한림대학교 버스정류장(2104)에서 2번을 탑승하거나, 한림대학교 버스정류장(1084)에서 300번 버스를 탑승해 시외버스터미널 정류장에서 하차하면 됩니다. 2번 버스의 배차간격은 평일 기준 20분이며 한림대학교에서 시외버스터미널까지 20분이 소요됩니다. 300번 버스의 배차간격은 평일 42회이며, 한림대학교에서 시외버스터미널까지 26분이 소요됩니다."
    },
    {
        "input": "MT 준비물 알려줘",
        "output": "MT 준비물로는 모자, 간단한 세면도구, 편한 옷, 편한 신발이 있습니다. 대학교 MT는 세면에 많은 시간을 소요할 수 없는 환경일 가능성이 높아 간단하게 준비하고 참여하는 것을 권장합니다."
    },
    {
        "input": "학교 셔틀버스 비용 알려줘",
        "output": "한림대학교 셔틀버스는 무료로 승차 가능합니다. 춘천역에 가는 셔틀버스는 CLC(캠퍼스 라이프 센터)앞에서 승차 가능합니다."
    },
    {
        "input": "토익특별시험 성적이 외부에서도 인정돼?",
        "output": "한림대학교에서 시행하는 토익 특별시험은 성적이 외부에서 인정되지 않습니다. 정기토익 성적만 인정됩니다. 토익 특별 시험 점수는 실력 진단, 졸업 요건 충족 등의 용도로 사용할 수 있습니다. "
    },
    {
        "input": "토익 특별시험에서 사용 가능한 신분증 종류 알려줘",
        "output": "한림대학교에서 진행하는 토익 특별시험에서 인정하는 신분증은 실물 주민등록증, 실물 운전면허증, 만료되지 않은 여권, 모바일 주민등록증입니다. 신분증을 촬영한 사진이나 학생증은 인정되지 않습니다.  "
    },
    {
        "input": "등본 출력은 교내 어디에서 어떻게 해야돼?",
        "output": "CLC(캠퍼스 라이프 센터) 1층에서 무인 민원 발급을 받을 수 있습니다."
    },
    
    {
        "input": "총학생회비 4년에 한번 내?",
        "output":  "한림대학교 총학생회비는 1년에 한번 납부하고, 과 학생회비는 4년에 한번만 납부하면 됩니다."
    },
"""

In [11]:
import os
from langchain_core.tools import tool
from tavily import TavilyClient

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


# 도구 정의

#0 에타
# @tool 
# def paper_everytime_question(query: str) -> str:
#     """
#     한림대학교 커뮤니티에 있던 질문들에 대한 답이기 때문에 매우 넓은 범위에 걸친 질문들이 있어. 학교 생활 전반에 대한 내용은 이 데이터를 참고해. 
#     다른 tool중에 겹치는 내용이 있으면 다른 tool을 참고해서 답변을 주고, 다른 tool에 없는 정보인 경우에만 이 정보를 참고해서 대답해. 
#     예를 들어 기숙사 택배 주소, 한림대학교 주변 시설 정보 등을 물어보는 경우 이 데이터를 참고해.

#     특정 행사에 대해 언제인지 일정이나 날짜, 기간을 물어보는 질문이면 올바른 답을 알려줘.
    
#     """
#     return everytime_question

#1 전공
@tool
def paper_major_software(query: str) -> str:
    """    
    한림대학교와 한림대학교의 전공 정보에 대한 질문들에 대한 답이야. 다음과 같은 질문이 들어오면 이 데이터를 참고해.
    한림대학교 총장, 교육이념, 인재상, 연도별 연혁, 재학생과 교직원 현황, 학교 시설 현황, 
    학과 소개, 학과 교육 목표, 전망과 진로 등 학과에 대한 정보.

    """
    return major_software

#2 교수정보
@tool
def paper_prof_info(query: str) -> str:
    """한림대학교 교수와 관련된 쿼리이면 prof_info를 참조해서 대답해.
    교수님 이름을 주고 전화번호나 메일이나 연구실 위치를 물어보면 이 데이터를 참고해.
    
    
    """
    return prof_info

#3 강의실 정보
@tool
def paper_classroom_info(query: str) -> str:
    """한림대학교 강의실에 관련된 쿼리이면 classroom_info을 참조해서 대답해.
    강의실 번호(호수)를 주고 위치를 물어보면 이 데이터를 참고해.
    """
    return classroom_info

#4. 전화번호 정보
@tool
def paper_phone_number(query: str) -> str:
    """한림대학교 전화번호에 관련된 쿼리이면 phone_number를 참조해서 대답해. 
    개인 전화번호가 아닌 학교 부서나 학과 전화번호를 물어보는 경우에 해당해. 
    교수님 전화번호를 물어보는 경우에는 paper_prof_info()를 참고해. """
    return phone_number

# 5. 메일 정보
@tool
def paper_hallym_mail(query: str) -> str:
    """한림대학교 메일에 관련된 쿼리이면 hallym_mail을 참조해서 대답해.
    개인 이메일이 아닌 학교 부서나 학과 이메일을 물어보는 경우에 해당해. 
    교수님 이메일을 물어보는 경우에는 이 데이터가 아니라 paper_prof_info()를 참고해. 
    """
    return hallym_mail

# 6. 다아라 정보
@tool
def paper_daara_info(query: str) -> str:
    """다아라에 관련된 쿼리이면 daara_info를 참조해서 대답해.
    다아라는 LLM 챗봇을 의미해. 
    질문을 하는 사용자는 챗봇을 대화하는 상대방으로 인식하기 때문에 대화 상대인 챗봇에 대해 물어볼 때 '너'라고 하거나 '다아라'라고 지칭할거야.
    만약 '다아라'나 '너'와 같이 챗봇을 지칭하고 챗봇에 대해 물어보는 경우에는 이 데이터를 참고해.
    그리고 '안녕'이나 '반가워'같이 챗봇에게 말을 걸거나, '너'를 생략하고 현재 대화중인 챗봇에 대해 질문하는 경우에도 이 데이터를 참고해.
    하지만 지금 대화중인 챗봇이 아니라 챗봇에 대한 지식이 궁금한 거라면 이 데이터를 참고하면 안돼.
    """
    return daara_info

# add data 나눈것
# 7. 기숙사
@tool
def paper_Hallym_dormitory(query: str) -> str:
    """
    기숙사나 세탁실과 같은 교내 주거 시설에 대한 질문이면 Hallym_dormitory를 참조해서 대답해.
    기숙사 관련해서 관 별 전화번호, 등록 기간, 환불 기준, 신청 방법. 결핵진단서 제출, 룸메이트, 운영시간, 외박신청, 세탁, 배달, 점호, 공부방에 대한 정보가 있어.
    기숙사는 긱사, 학생생활관 등으로 표현될 수 있으니 학교 기숙사를 의미하는 말들로 질문하면 이 데이터를 참고해.

    전화번호, 등록 기간, 환불 기준, 신청 방법. 결핵진단서 제출, 룸메이트, 운영시간, 외박신청, 세탁, 배달, 점호, 공부방은 줄임말이나 비슷한 의미의 단어로 대체되어 질문될수도 있어.
    예를 들어 전화번호를 전번, 세탁을 빨래로 바꾸어 질문하는 경우에도 의미만 같으면 이걸 참고해서 대답해줘.
    
    
    보통 '언제'와 같이 시기를 묻는 질문에는 기간이나 일정을 알려주면 돼.

    """
    return Hallym_dormitory

# 8. 수업 관련
@tool
def paper_Hallym_lecture(query: str) -> str:
    """
    수업에 대한 질문이면 Hallym_lecture를 참조해서 대답해.
    수업 관련해서 출석과 결석 기준, 공결처리, 지각 기준, 오디세이 수업 관련, 수강신청, 수강철회, 수강 취소, 공결 신ㄴ청, 대체재이수와 관련된 정보들이 있어.
    
    보통 '언제'와 같이 시기를 묻는 질문에는 기간이나 일정을 알려주면 돼.

    """
    return Hallym_lecture

# 9. 학사 일정
@tool
def paper_Hallym_Academic_Calendar(query: str) -> str:
    """
    학사 일정에 관련된 질문이면 Hallym_Academic_Calendar를 참조해서 대답해.
    한림대학교 학사 일정에 관련된 내용이니 대체로 특정 행사의 기간을 알려달라고 하는 경우에 해당해. 
    학사 일정과 관련해 축제 기간, 기말고사 기간, 수강신청 기간, 개강 날짜 등이 있어.
    
    
    보통 '언제'와 같이 시기를 묻는 질문에는 기간이나 일정을 알려주면 돼.
    
    """
    return Hallym_Academic_Calendar

# 10. 학교 시설 (이용법, 운영시간, 위치 등)
@tool
def paper_Hallym_facilities(query: str) -> str:
    """
    학교 시설 (이용법, 운영시간, 위치 등)에 대한 질문이면 Hallym_facilities를 참조해서 대답해.
    주로 한림대학교 시설의 이용법, 운영 시간, 위치 등을 묻는 경우가 많아.
    학교 시설과 관련해 교내 편의점 위치, 건물 찾아가는 방법, 엘리베이터 위치, 우체국 위치, 도서관 관련 정보, 보건실, 공부할 곳, 학교 주변 카페나 빵집같은 시설, 지름길, 학생식당, 학식, 통학버스(셔틀버스), 헬스장, 레크레이션 센터, clc와 관련된 정보들이 있어.
    엘리베이터를 엘베로, 레크레이션센터를 렉센으로, 캠퍼스라이프센터를 캠라나 CLC와 같은 줄임말로 물어볼수도 있어.
 
    
    보통 '언제'와 같이 시기를 묻는 질문에는 기간이나 일정을 알려주면 돼.

    
    """
    return Hallym_facilities

# 11. 나머지 add data
@tool
def paper_add_data(query: str) -> str:
    """
    다음과 관련된 내용은 add_data를 참조해서 대답해.
    한림대학교 공식 앱, 휴학, 복학, 등록금, 차량 등록, 개강총회(개총), 와이파이(wifi), 시외버스 터미널 가는 방법, MT,토익특별시험, 학생회비, 등본 출력과 관련된 내용에 해당해. 
    
    보통 '언제'와 같이 시기를 묻는 질문에는 기간이나 일정을 알려주면 돼.
    
    
    """
    return add_data

@tool
def internet_search(query: str) -> str:
    """This is for query for internet search engine like Google. Query for general topics."""
    return tavily.search(query=query)

# 사용하고 싶은 도구 리스트 생성
tools = [paper_major_software, paper_prof_info, paper_classroom_info, paper_phone_number, paper_hallym_mail, paper_daara_info, paper_Hallym_dormitory, paper_Hallym_lecture, paper_Hallym_Academic_Calendar, paper_Hallym_facilities, paper_add_data, internet_search]

# llm에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools)


In [12]:
llm_with_tools.invoke("한림대학교 장학금 관련 문의 전화번호 알려줘").tool_calls


[{'name': 'paper_phone_number',
  'args': {'query': '장학금 문의'},
  'id': '5550fad7-6008-4bb8-a431-372ba50afc1a'}]

In [13]:
def call_tool_func(tool_call):
    tool_name = tool_call["name"].lower() # 다 소문자로
    if tool_name not in globals(): # 함수 명에 있으면 호출 
        print("Tool not found", tool_name)
        return None
    selected_tool = globals()[tool_name]
    return selected_tool.invoke(tool_call["args"])

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt_template = PromptTemplate.from_template(
    """
    Please provide answer for question from the following context. 
    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [15]:
# Smart RAG, Self-Improving RAG
import os
from tavily import TavilyClient


def tool_rag(question):
    for _ in range(3): # try 3 times
        tool_calls = llm_with_tools.invoke(question).tool_calls
        if tool_calls:
            break

    if not tool_calls:
        return "I'm sorry, I don't have an answer for that."
    
    print(tool_calls)
    context = ""
    for tool_call in tool_calls:
        context += str(call_tool_func(tool_call))

    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"context": context, "question": question})

In [16]:
tool_rag("한림대학교 전화번호")

[{'name': 'paper_phone_number', 'args': {'query': '한림대학교 전화번호'}, 'id': '888025bb-00e1-4301-aba4-cd7371ce369c'}]


'한림대학교의 전화번호는 0332481000입니다.'

Flask

In [17]:
# # 리액트 연결 없이 test
# from flask import Flask, request, jsonify, render_template_string
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Flask 앱 생성
# app = Flask(__name__)


# # HTML 템플릿
# html_template = '''
# <!DOCTYPE html>
# <html>
# <head>
#     <title>Ask a Question</title>
# </head>
# <body>
#     <h1>Ask a Question</h1>
#     <form action="/" method="post">
#         <label for="question">Question:</label>
#         <input type="text" id="question" name="question">
#         <input type="submit" value="Submit">
#     </form>
#     {% if answer %}
#     <h2>Answer:</h2>
#     <p>{{ answer }}</p>
#     {% endif %}
# </body>
# </html>
# '''

# # 루트 엔드포인트 정의 (HTML 폼 제공)
# @app.route('/', methods=['GET', 'POST'])
# def home():
#     answer = None
#     if request.method == 'POST':
#         question = request.form['question']
#         answer = tool_rag(question)
#     return render_template_string(html_template, answer=answer)

# # Flask 앱 실행
# if __name__ == '__main__':
#     app.run()

Flask - React

In [18]:
from flask import Flask, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer
from flask_cors import CORS
import torch
import threading

# Flask 앱 생성
app = Flask(__name__)
CORS(app)

# 루트 엔드포인트 정의 (HTML 폼 제공)
@app.route('/')
def display():
    return "연결이 되었습니다."

# API 엔드포인트 정의
@app.route('/api', methods=['GET','POST'])
def api():
    content_type = request.headers.get('Content-Type')
    if not content_type or 'application/json' not in content_type:
        return jsonify({'text': 'Unsupported Media Type: Content-Type must be application/json'}), 415

    try:
        request_json = request.get_json()
        if request_json is None:
            raise ValueError("JSON 형식의 요청이 필요합니다.")

        question = request_json.get('userMessage', '').strip()
        if not question:
            raise ValueError("userMessage가 제공되지 않았습니다.")
        
        answer = tool_rag(question)
        data = {'text': answer}
        return jsonify(data), 200
    except ValueError as ve:
        print(f"요청 오류: {ve}")
        return jsonify({'text': str(ve)}), 400
    except Exception as e:
        print(f"오류 발생: {e}")
        return jsonify({'text': '요청을 처리하는 중 오류가 발생했습니다.'}), 500

# 모델 로드 상태 확인 엔드포인트 정의
# @app.route('/status', methods=['GET'])
# def status():
#     if model is None or tokenizer is None:
#         return jsonify({'status': '모델 로딩 중'}), 503
#     return jsonify({'status': '모델 로딩 완료'}), 200

# Flask 앱 실행
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/May/2024 22:14:42] "OPTIONS /api HTTP/1.1" 200 -


[{'name': 'paper_daara_info', 'args': {'query': '안녕'}, 'id': 'ebeddc1a-3bf1-42b9-87b1-a6e1361e8a11'}]


127.0.0.1 - - [20/May/2024 22:14:44] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2024 22:14:54] "OPTIONS /api HTTP/1.1" 200 -


[{'name': 'paper_major_software', 'args': {'query': '한림대학교 주소'}, 'id': 'd147c25c-eea1-4ec7-b1ca-157f291a6174'}]


...

127.0.0.1 - - [20/May/2024 22:32:53] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2024 22:33:01] "OPTIONS /api HTTP/1.1" 200 -


[{'name': 'paper_Hallym_facilities', 'args': {'query': '도서관 대출 기간'}, 'id': '52fd532f-53e8-4515-8d84-9bce7a5ab075'}]
Tool not found paper_hallym_facilities


127.0.0.1 - - [20/May/2024 22:33:04] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2024 22:33:15] "OPTIONS /api HTTP/1.1" 200 -


[{'name': 'paper_Hallym_Academic_Calendar', 'args': {'query': '2학기 중간고사 언제야?'}, 'id': '4c515863-8fe0-4921-bfe5-bd6272abdce6'}]
Tool not found paper_hallym_academic_calendar


127.0.0.1 - - [20/May/2024 22:33:17] "POST /api HTTP/1.1" 200 -
